<a href="https://colab.research.google.com/github/gseetha04/LLMs-Medicaldata/blob/main/GPT_4_GD_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, DistilBertTokenizerFast
# sequence tagging model + training-related
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pandas as pd
import spacy
import re
import json
from nltk.tokenize import word_tokenize

data_train_df = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')
tagged_words = []
for para in data_train_df['final']:
   para=para.encode("ascii", "ignore")
   para=para.decode()
   connl_text= []
   label = []
   cause = re.findall(r'<C>(.*?)</C>', para)
   condition = re.findall(r'<CO>(.*?)</CO>', para)
   effect = re.findall(r'<E>(.*?)</E>', para)
   action = re.findall(r'<A>(.*?)</A>', para)
   signal = re.findall(r'<S>(.*?)</S>',para)
  # ARG0 = [re.sub('<(\w+)>(.*?)<\/\w+>','',i) for i in ARG0]
   splitted = re.split('<C>|</C>|<CO>|</CO>|<E>|</E>|<A>|</A>|<S>|</S>', para)  # splitted considering tags
   if cause or condition or effect or action or signal:  # if any tag in sentence
    for split in splitted:  # search each index
            if split in cause:
                for token in split.split():
                    connl_text.append(token)
                    label.append('C')
            elif split in effect:
                for token in split.split():
                    connl_text.append(token)
                    label.append('E')
            elif split in signal:
                for token in split.split():
                  connl_text.append(token)
                  label.append('S')
            elif split in action:
                for token in split.split():
                  connl_text.append(token)
                  label.append('A')
            elif split in condition:
                for token in split.split():
                  connl_text.append(token)
                  label.append('CO')

            else:  # current word is not an entity
                for token in split.split():
                    connl_text.append(token)
                    label.append('O')

    tagged_words.append({'Word': connl_text, 'Tag': label})

In [ ]:
data_train = pd.DataFrame(tagged_words)

In [ ]:
data_train

,Word,Tag
0,"[Pregnant, persons, with, gestational, diabete...","[S, S, C, C, C, O, O, E, E, E, E, E, E, E, E, ..."
1,"[Gestational, diabetes, has, also, been, assoc...","[C, C, O, O, O, O, O, O, E, E, E, E, E, E, E, ..."
2,"[EVIDENCE, ASSESSMENT, The, USPSTF, concludes,...","[O, O, O, O, O, O, O, O, O, O, O, O, S, S, S, ..."
3,"[RECOMMENDATION, The, USPSTF, recommends, scre...","[O, O, O, O, A, A, A, A, O, CO, CO, CO, CO, CO..."
4,"[IImportance, Gestational, diabetes, is, diabe...","[O, C, C, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
284,"[Clinical, Preventive, Service, Recommendation...","[O, O, O, O, O, O, O, O, O, O, A, A, A, A, A, ..."
285,"[The, AAFP, also, agrees, that, there, is, ins...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
286,"[While, AAFP, acknowledges, that, diabetes, ,,...","[O, O, O, O, E, O, O, O, O, O, O, O, O, O, O, ..."
287,"[The, evidence, did, show, a, reduction, in, t...","[O, O, O, O, O, E, E, E, E, E, E, O, O, O, O, ..."


In [ ]:
data_train.to_excel('/content/tagged_words_GD-data.xlsx')

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
text = f'''
ImportanceGestational diabetes is diabetes that develops during pregnancy.1-3 Prevalence of gestational diabetes in the US has been estimated at 5.8% to 9.2%, based on traditional diagnostic criteria, although it may be higher if more inclusive criteria are used.4-8 Pregnant persons with gestational diabetesincreased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia.3,9-11 Gestational diabetes has also been associated with an increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring.12-16Table 1.
The same tests may be used to screen for and diagnose diabetes and to detect individuals with prediabetes (Table 2.2 and Table 2.5).
Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG may be used to diagnose diabetes (Table 2.2).
A1CRecommendations2.1 To avoid misdiagnosis or missed diagnosis, the A1C test should be performed using a method that is certied by the NGSP and stan- dardized to the Diabetes Control and Complications Trial (DCCT) assay.
Race/Ethnicity/HemoglobinopathiesHemoglobin variants can interfere with the measurement of A1C, although most assays in use in theU.S. are unaffected by the most common variants.
For patients with a hemoglobin variant but normal red blood cell turnover, such as those with the sickle cell trait, an A1C assay without interference from hemoglobin variants should be used.
African Americans heterozygous for the common hemoglobin variant HbS may have, for any given level of mean glycemia, lower A1C by about 0.3% than those without the trait (20).
Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans, was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant (21).
Even in the absence of hemoglobin variants, A1C levels may vary with race/ ethnicity independently of glycemia (22-24).
For example, African Americans may have higher A1C levels than non- Hispanic whites with similar fasting and postglucose load glucose levels (25), and A1C levels may be higher for a given mean glucose concentration when measured with continuous glucose monitoring (26).
Be- cause of the potential for preanalytic variability, it is critical that samples for plasma glucose be spun and separated immediately after they are drawn.
TYPE 1 DIABETESRecommendationsScreening for type 1 diabetes risk with a panel of islet autoantibodies is currently recommended in the setting of a research trial or can be offered as an option for rst-degree family members of a proband with type 1 diabetes.
Although patients are not typically obese when they present with type 1 diabetes, obesity is increasingly common in the general population and there is evidence that it may also be a risk factor for type 1 diabetes.
People with type 1 diabetes are also prone to other autoimmune disorders such as Hashimoto thyroiditis, Graves disease, celiac disease, Addison disease, vitiligo, auto- immune hepatitis, myasthenia gravis, and pernicious anemia (see Section 4 "Comprehensive Medical Evaluation and Assessment of Comorbidities," https://doi.org/10.2337/dc20-S004).
Individuals with autoantibody-negative type 1 diabetes of African or Asian ancestry may suffer from episodic DKA and exhibit varying degrees of insulin deficiency between episodes.
Multiple studies indicate that measuring islet autoantibodies in individuals genetically at risk for type 1 diabetes (e.g., relatives of those with type 1 diabetes or indi- viduals from the general population with type 1 diabetes-associated genetic factors) identifies individuals who may develop type 1 diabetes (9).
Indeed, the risk of type 1 diabetes increases as the number of relevant autoantibodies detected increases (43-45).
PREDIABETES AND TYPE 2 DIABETES2.13 Risk based screening for prediabetes and/or type 2 diabetes should be considered after the onset of puberty or after 10 years of age, whichever occurs earlier, in children and adolescents with overweight (BMI >85th percen- tile) or obesity (BMI >95th per- centile) and who have one or more risk factor for diabetes.
If results are normal, testing should be repeated at a minimum of 3-year intervals, with consideration of more frequent testing depending on initial results and risk status.
It should be noted that theTable 2.4Risk-based screening for type 2 diabetes or prediabetes in asymptomatic children and adolescents in a clinical setting (163)Testingshould be considered in youth* who have overweight (>85th percentile) or obesity (>95th percentile) and who have one or more additional risk factors based on the strength of their association with diabetes:Maternal history of diabetes or GDM during the childs gestation AFamily history of type 2 diabetes in rst- or second-degree relative ARace/ethnicity (Native American, African American, Latino, Asian American, Pacic Islander) ASigns of insulin resistance or conditions associated with insulin resistance (acanthosis nigricans, hypertension, dyslipidemia, polycystic ovary syndrome, or small-for-gestational- age birth weight) BGDM, gestational diabetes mellitus.
If tests are normal, repeat testing at a minimum of 3-year intervals, or more frequently if BMI is increasing, is recommended.
Those with an A1C range of 6.0- 6.5% (42-48 mmol/mol) had a 5-year risk of developing diabetes between 25% and 50% and a relative risk 20 times higher compared with A1C of 5.0% (31 mmol/mol) (49).
Similar to those with IFG and/or IGT, individuals with A1C of 5.7-6.4% (39-47 mmol/mol) should be informed of their increased risk for diabetes and CVD and counseled about effective strategies to lower their risks (see Section 3 "Prevention or Delay of Type 2 Di- abetes," https://doi.org/10.2337/dc20- S003).
Similar to glucose measurements, the continuum of risk is curvilinear, so as A1C rises, the diabetes risk rises disproportionately (49).
Aggressive interventions and vigilant follow-up should be pursued for those considered at very high risk (e.g., those with A1C .6.0% [42 mmol/mol]).
Excess weight itself causes some degree of insulin resistance.
DKA seldom occurs spontaneously in type 2 diabetes; when seen, it usually arises in association with the stress of another illness such as infection or with the use of certain drugs (e.g., corticosteroids, atypical antipsychotics, and sodium-glucose cotransporter 2 inhibitors) (53,54).
The risk of developing type 2 diabetes increases with age, obesity, and lack of physical activity.
After 5.3 years of follow-up, CVD risk factors were modestly but significantly improved with intensive treatment compared with routine care, but the incidence of first CVD events or mortality was not significantly different between the groups (48).
Community ScreeningIdeally, testing should be carried out within a health care setting because of the need for follow-up and treatment.
CYSTIC FIBROSIS-RELATED DIABETESRecommendationsAnnual screening for cystic brosisrelated diabetes (CFRD) with an oral glucose tolerance test should begin by age 10 years in all patients with cystic brosis not previously diagnosed with CFRD.
BPatients with cystic brosis related diabetes should be treated with insulin to attain individualized glycemic goals.
ABeginning 5 years after the diagnosis of cystic brosisrelated diabetes, annual monitoring for complications of diabetes is recommended.
Diabetes in this population, compared with individuals with type 1 or type 2 diabetes, is associated with worse nutritional status, more severe infiammatory lung disease, and greater mortality.
Genetically determined b-cell function and insulin resistance associated with infection and infiammation may also contribute to the development of CFRD.
EThe oral glucose tolerance test is the preferred test to make a diagnosis of posttransplantation diabetes mellitus.
Most studies have reported that transplant patients with hyperglycemia and PTDM after transplantation have higher rates of rejection, infection, and rehospitalization (89,91,96).
Those with previously poor control or with persistent hyperglycemia should continue insulin with frequent home self-monitoring of blood glucose to determine when insulin dose reductions may be needed and when it may be appropriate to switch to non- insulin agents.
Drug dose adjustments may be required because of decreases in the glomerular filtration rate, a relatively common complication in transplant patients.
Thiazolidinediones have been used successfully in patients with liver and kidney transplants, but side effects include fiuid retention, heart failure, and osteopenia (98,99).
Well-designed intervention trials examining the efficacy and safety of these and other antihyperglycemic agents in patients with PTDM are needed.
MONOGENIC DIABETES SYNDROMESRecommendationsAll children diagnosed with diabetes in the rst 6 months of life should have immediate genetic testing for neonatal diabetes.
AChildren and those diagnosed in early adulthood who have diabetes not characteristic of type 1 or type 2 diabetes that occurs in successive generations (suggestive of an autosomal dominant pattern of inheri- tance) should have genetic test- ing for maturity-onset diabetes of the young.
AIn both instances, consultation with a center specializing in diabetes genetics is recommended to understand the signicance of these mutations and how best to approach further evaluation, treatment, and genetic counseling.
EMonogenic defects that cause b-cell dysfunction, such as neonatal diabetes and MODY, represent a small fraction of patients with diabetes (,5%).
Correct diagnosis has critical implications because mostTable 2.6-Most common causes of monogenic diabetes (102)Gene Inheritance Clinical featuresMODYNeonatal diabetesGCK AD GCK-MODY: stable, nonprogressive elevated fasting blood glucose; typically does notrequire treatment; microvascular complications are rare; small rise in 2-h PG level on OGTT (,54 mg/dL [3 mmol/L])HNF1A AD HNF1A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; lowered renal threshold for glucosuria; large rise in 2-h PG level on OGTT (.90 mg/dL [5 mmol/L]); sensitive to sulfonylureasHNF4A AD HNF4A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; may have large birth weight and transient neonatal hypoglycemia; sensitive to sulfonylureasHNF1B AD HNF1B-MODY: developmental renal disease (typically cystic); genitourinary abnormalities; atrophy of the pancreas; hyperuricemia; goutKCNJ11 AD Permanent or transient: IUGR; possible developmental delay and seizures; responsive toSulfonylureasINS AD Permanent: IUGR; insulin requiringABCC8 AD Permanent or transient: IUGR; rarely developmental delay; responsive to sulfonylureas6q24 (PLAGL1,HYMA1)AD for paternal duplicationsTransient: IUGR; macroglossia; umbilical hernia; mechanisms include UPD6, paternal duplication or maternal methylation defect; may be treatable with medications other than insulinGATA6 AD Permanent: pancreatic hypoplasia; cardiac malformations; pancreatic exocrine insufficiency;insulin requiringEIF2AK3 AR Permanent: Wolcott-Rallison syndrome: epiphyseal dysplasia; pancreatic exocrineinsufficiency; insulin requiringFOXP3 X-linked Permanent: immunodysregulation, polyendocrinopathy, enteropathy X-linked (IPEX)syndrome: autoimmune diabetes; autoimmune thyroid disease; exfoliative dermatitis; insulin requiringAD, autosomal dominant; AR, autosomal recessive; IUGR, intrauterine growth restriction; OGTT, oral glucose tolerance test; 2-h PG, 2-h plasma glucose.
patients with KATP-related neonatal diabetes will exhibit improved glycemic control when treated with high-dose oral sulfonylureas instead of insulin.
Insulin gene (INS) mutations are the second most common cause of permanent neonatal diabetes, and, while intensive insulin management is currently the preferred treatment strategy, there are important genetic counseling considerations, as most of the mutations that cause diabetes are dominantly inherited.
Clinically, patients with GCK-MODY exhibit mild, stable, fasting hyperglycemia and do not require antihyperglycemic therapy except sometimes during pregnancy.
Mutations or deletions in HNF1B are associated with renal cysts and uterine malformations (renal cysts and diabetes [RCAD] syndrome).
Additionally, diagnosis can lead to identification of other affected family members.
A diagnosis of MODY should be considered in individuals who have atypical diabetes and multiple family members with diabetes not characteristic of type 1 or type 2 diabetes, although admittedly "atypical diabetes" is becoming increasingly difficult to precisely define in the absence of a definitive set of tests for either type of diabetes (104-110).
In most cases, the presence of autoantibodies for type 1 diabetes precludes further testing for monogenic diabetes, but the presence of autoantibodies in patients with monogenic diabetes has been reported (111).
Individuals in whom monogenic diabetes is suspected should be referred to a specialist for further evaluation if available, and consultation is available from several centers.
It is critical to correctly diagnose one of the monogenic forms of diabetes because these patients may be incorrectly diagnosed with type 1 or type 2 diabetes, leading to suboptimal, even potentially harmful, treatment regimens and delays in diagnosing other family members (114).
Genetic counseling is recommended to ensure that affected individuals understand the patterns of inheritance and the importance of a correct diagnosis.
The diagnosis of monogenic diabetes should be considered in children and adults diagnosed with diabetes in early adulthood with the following findings: Diabetes diagnosed within the first 6 months of life (with occasional cases presenting later, mostly INS and ABCC8 mutations) (103,116) Diabetes without typical features of type 1 or type 2 diabetes (negative diabetes-associated autoantibodies, non- obese, lacking other metabolic features especially with strong family history of diabetes) Stable, mild fasting hyperglycemia (100-150 mg/dL [5.5-8.5 mmol/L]),stable A1C between 5.6 and 7.6%(between 38 and 60 mmol/mol), especially if nonobesePANCREATIC DIABETES OR DIABETES IN THE CONTEXT OF DISEASE OF THE EXOCRINE PANCREASPancreatic diabetes includes both structural and functional loss of glucose- normalizing insulin secretion in the context of exocrine pancreatic dysfunction and is commonly misdiagnosed as type 2 diabetes.
Hyperglycemia due to general pancreatic dysfunction has been called "type 3c diabetes" and, more recently, diabetes in the context of disease of the exocrine pancreas has been termedpancreoprivic diabetes (1).
In the context of pancreatectomy, islet autotransplantation can be done to retain insulin secretion (123,124).
In some cases, this can lead to insulin independence.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the first prenatal visit in those with risk factors using standard diagnostic criteria.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the rst prenatal visit in those with risk factors using standard diagnostic criteria.
BTest for gestational diabetes mellitus at 2428 weeks of gestation in pregnant women not previously found to have diabetes.
ATest women with gestational diabetes mellitus for prediabetes or diabetes at 412 weeks postpartum, using the 75-g oral glucose tolerance test and clinically appropriate nonpregnancy diagnostic criteria.
BWomen with a history of gestational diabetes mellitus should have lifelong screening for the development of diabetes or pre- diabetes at least every 3 years.
BWomen with a history of gestational diabetes mellitus found to have prediabetes should receive intensive lifestyle interventions and/or metformin to prevent diabetes.
Because of the number of pregnant women with undiagnosed type 2 diabetes, it is reasonable to test women with risk factors for type 2 diabetes (133) (Table 2.3) at their initial prenatal visit, using standard diagnostic criteria (Table 2.2).
As effective prevention interventions are available (138,139), women diagnosed with GDM should receive lifelong screening for pre- diabetes to allow interventions to reduce diabetes risk and for type 2 diabetes to allow treatment at the earliest possible time (140).
'''

In [ ]:
text = f'''
Summary of USPSTF RationaleAssessmentRationalePregnant persons at 24 weeks of gestation or afterPregnant persons before 24 weeks of gestationDetectionThere is adequate evidence that commonly used screening tests can accurately detect gestational diabetes.
There is inadequate evidence that commonly used screening tests can accurately detect gestational diabetes earlier than 24 weeks of gestation.
Benefits of early detection and intervention and treatmentThere is inadequate direct evidence that screening for gestational diabetes improves health outcomes.
There is adequate evidence that treatment of screen-detected gestational diabetes is associated with moderate improvements in maternal and fetal outcomes, including primary (first) cesarean delivery, macrosomia, large for gestational age infants, shoulder dystocia, birth injury, and neonatal intensive care unit admissions.
Harms of early detection and intervention and treatmentThere is adequate evidence that the harms of screening for gestational diabetes, such as anxiety, depression, and labeling, are small.
There is adequate evidence that the harms of treatment of gestational diabetes, such as maternal hypoglycemia and low birth weight, are small.
USPSTF Assessment of Magnitude of Net BenefitPregnant Persons at 24 Weeks of Gestation or AfterThe US Preventive Services Task Force (USPSTF) concludes with moderate certainty that there is a moderate net benefit to screening for gestational diabetes at 24 weeks of gestation or after to improve maternal and fetal outcomes.
Definitions of Gestational DiabetesDuring pregnancy, insulin resistance increases, leading to higher glucose intolerance and development of gestational diabetes in some pregnant persons.
Assessment of RiskSeveral factors increase a persons risk for developing gestational diabetes, including obesity, increased maternal age, history of gestational diabetes, family history of diabetes, and belonging to a racial/ethnic group that is at increased risk for developing type 2 diabetes (Hispanic, Native American, South or East Asian, or Pacific Islander descent).4,19,20 Factors associated with a lower risk for developing gestational diabetes include age younger than 25 to 30 years, White race, a body mass index (BMI) of 25 or less (calculated as weight in kilograms divided by height in meters squared), no family history (in a first-degree relative) of diabetes, and no history of glucose intolerance or adverse pregnancy outcomes related to gestational diabetes.21-23 The risk in different racial/ethnic groups may be due in part to social risk factors such as low socioeconomic status or structural racism, although these associations are not examined in the current evidence.
Although a higher BMI increases the risk of gestational diabetes across racial/ethnic groups, the association varies.
In Asian American persons, the prevalence of gestational diabetes at a BMI of 22 to less than 25 is similar to the prevalence in Hispanic persons, non-Hispanic White persons, and Black persons with a higher (>28) BMI.24,25Screening TestsScreening for gestational diabetes in asymptomatic persons involves either a 2-step (screening test followed by a diagnostic test) or 1-step (diagnostic test used for all patients) approach.
In the US, a 2-step approach is commonly used.8,26 A 50-g oral glucose chal- lenge test (OGCT) is performed between 24 and 28 weeks of gestation in a nonfasting state.
If the screening threshold is met or exceeded, patients receive the oral glucose tolerance test (OGTT).
During the OGTT, a fasting glucose level is obtained, followed by administration of a 75-g or 100-g glucose load, then evaluation of glucose levels after 1, 2, and often 3 hours.
Pregnant persons at 24 weeks of gestation or after\nScreen for gestational diabetes.Grade: BPregnant persons before 24 weeks of gestationThe evidence is insufficient to assess the balance of benefits and harms for screening for gestational diabetes.
Screen: If the person is pregnant and is at least 24 weeks of gestation, screen for gestational diabetes by using 1 of several methods:A 2-step process that involves a screening test (oral glucose challenge test) followed by a diagnostic test (oral glucose tolerance test).
A diagnosis of gestational diabetes is made when 2 or more glucose values fall at or above the specified glucose thresholds.27In the 1-step approach, a 75-g glucose load is administered after a fasting glucose level is obtained, and plasma glucose levels are evaluated after 1 and 2 hours.
The American College of Obstetricians and Gynecologists recommends testing between 24 and 28 weeks of gestation.1 Pregnant persons whose first prenatal visit happens after 28 weeks of gestation (ie, late entry into prenatal care) should bescreened as soon as possible.
Potential Preventable BurdenBetween 2006 and 2016, there was an absolute increase of 3.6% in the prevalence of gestational diabetes.7 Pregnant persons with gestational diabetes are at increased risk for maternal and fetal complications and may benefit from early identification and treatment.
They are also at increased risk for developing type 2 diabetes after pregnancy.29 Pregnant persons who are diagnosed with gestational diabetes before 24 weeks of gestation may be at even greater risk for maternal and fetal complications.29Potential HarmsPotential harms of screening for gestational diabetes include psychological harms (anxiety, depression), intensive medical interventions (induction of labor, cesarean delivery, or admission to the neonatal intensive care unit [NICU]), and negative hospital experiences related to labeling (reduction in breastfeeding and fewer newborns staying in the mothers room) that may be associated with a diagnosis of gestational diabetes.
Race/ethnicity was fairly diverse in several studies, although 2 of the largest studies with treatment at 24 weeks of gestation or after enrolled large proportions of White (75%) and Chinese (97%) persons.29,34,37,38Treatment of gestational diabetes at 24 weeks of gestation or after was associated with decreased risk of primary cesarean deliveries (relative risk [RR], 0.70 [95% CI, 0.54-0.91]; absolute risk difference [ARD], 5.3%; 3 trials) and preterm deliveries, although findings for the latter are not statistically significant (RR, 0.75 [95% CI, 0.56-1.01]; ARD, 2.3%; 4 trials).29,34 Treatment of gestational diabetes was not associated with reduced risk of preeclampsia in the included studies (RR, 0.99 [95% CI, 0.46-2.16]; 6 trials).29 Addi- tionally, treatment was not associated with reduced risk of gestational hypertension (2 trials), total cesarean deliveries (8 trials), emergency cesarean deliveries (1 trial), induction of labor (5 trials), or maternal birth trauma (2 trials).29,34For fetal/neonatal outcomes, treatment of gestational diabetes at 24 weeks of gestation or after was associated with reduced risk of shoulder dystocia (RR, 0.42 [95% CI, 0.23-0.77]; ARD, 1.3%;3 trials), macrosomia (RR, 0.53 [95% CI, 0.41-0.68]; ARD, 8.9%; 8 trials), LGA infants (RR, 0.56 [95% CI, 0.47-0.66]; ARD, 8.4%; 7 trials), birth injury (eg, fracture or nerve palsies) (odds ratio, 0.33 [95% CI, 0.11-0.99]; ARD, 0.2%), and NICU admissions (RR, 0.73 [95% CI, 0.53-0.99]; ARD, 2%; 5 trials).29,34 No association was found for several outcomes, including mortality (stillbirth, neona- tal, or perinatal), respiratory distress syndrome, hypoglycemia (any or clinical), and hyperbilirubinemia.
One study found that gestational diabetes diagnosis and labeling may be associated with higher rates of cesarean delivery.29,34Harms associated with the treatment of gestational diabetes were evaluated in 13 trials (11 RCTs and 2 nonrandomized clinical trials; n = 4235).29,34 Treatment at 24 weeks of gestation or after was not associated with an increased risk for severe maternal hypoglycemia, low birth weight, or small for gestational age infants.
Treatment of gestational diabetes was associated with a reduced risk of macrosomia (>4000 g) (RR, 0.53 [95% CI, 0.41 to 0.68]) but no difference in the risk of total number of cesarean deliveries (RR, 0.95 [95% CI, 0.83 to 1.08]).29,34 This suggests that a small proportion of pregnant persons may undergo cesarean delivery because of gestational diabetes diagnosis alone.
Pregnant persons who are unable to produce adequate insulin to overcome the increase in insulin resistance seen in pregnancy develop glucose intolerance and gestational diabetes.
'''

In [ ]:
text = f'''
If the screening threshold is met or exceeded, patients receive the oral glucose tolerance test (OGTT).
'''

In [ ]:
# import os
# import openai

# openai.api_key = "sk-****************"

# response = openai.Completion.create(
#   model="gpt-4",
#   prompt="Extract the cause/effect from sentence1 as given in the example:\nExample1: When a policyholder or insured gets sick or hurt the Company pays cash benefits fairly and promptly for eligible claims\nLabel: O O B-C I-C I-C I-C I-C I-C I-C O O B-CT B-E I-E I-E I-E I-E I-E I-E I-E\nExample2: During 2020 in response to the onset and development of the global COVID 19 pandemic the Company pivoted to digital sales methods and accelerated related digital investments\nLabel2: O O O O O B-C I-C I-C I-C I-C I-C I-C I-C I-C I-C O O B-CT B-E I-E I-E I-E O B-CT B-E I-E I-E\nExample3: Prolonged periods of low interest rates also heighten the risk associated with future increases in interest rates because an increasing proportion of the Company s investment portfolio include investments that bear lower rates of return than the embedded book yield of the investment portfolio\nLabel3: B-C I-C I-C I-C I-C I-C O B-CT B-E I-E O O O B-CT O O O B-CT O B-CT O O O O O O O O O O O B-E I-E I-E I-E O O O B-E I-E I-E I-E I-E I-E" + text,
#   temperature=0.0,
#   max_tokens=2048,
#   top_p=1,
#   frequency_penalty=0,
#   presence_penalty=0
# )

In [ ]:
import openai

openai.api_key = "sk-******************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. For each token in the sentence assign a label. Use A for action, C for cause, CO for condition, E for effect and O for others as given in the example. The number of predicted labels should be equal to the number of labels in the input.
Example1: Pregnant persons with gestational diabetes are at increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia .	Label1: S S C C C O O E E E E E E E E E E E E E E E E E E E E E E O
Example2: Gestational diabetes has also been associated with an increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring .	Label2: C C O O O O O O E E E E E E E E E E E E E E E E O
Example3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a moderate net benefit to screening for gestational diabetes at 24 weeks of gestation or after to improve maternal and fetal outcomes .	Label3: O O O O O O O O O O O O S S S O C C C C C C C C C C C O E E E E E O
Example4: RECOMMENDATION The USPSTF recommends screening for gestational diabetes in asymptomatic pregnant persons at 24 weeks of gestation or after .	Label4: O O O O A A A A O CO CO CO CO CO CO CO CO CO CO O
2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Sentence: If the screening threshold is met or exceeded, patients receive the oral glucose tolerance test (OGTT).
Prediction: CO CO CO CO CO CO CO CO CO, A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A A

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use A for action, C for cause, CO for condition, E for effect and O for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the sentences with the tags:

1. <C>Summary of USPSTF RationaleAssessmentRationalePregnant persons at 24 weeks of gestation or afterPregnant persons before 24 weeks of gestationDetectionThere is adequate evidence that commonly used screening tests can accurately detect gestational diabetes.</C>
2. <C>There is inadequate evidence that commonly used screening tests can accurately detect gestational diabetes earlier than 24 weeks of gestation.</C>
3. <C>Benefits of early detection and intervention and treatmentThere is inadequate direct evidence that screening for gestational diabetes improves health outcomes.</C>
4. <C>There is adequate evidence that treatment of screen-detected gestational diabetes is associated with moderate improvements in maternal and fetal outcomes, including primary (first) cesarean delivery, macrosomia, large for gestational age infants, shoulder dystocia, birth injury, and neonatal intensive care unit admissions.</C>
5. <C>Harms 

In [ ]:
text = f'''
The USPSTF found that several screening tests (OGCT, OGTT, fasting plasma glucose) and strategies (1- vs 2-step approach) can accurately detect gestational diabetes and does not recommend any single specific test.
Type 1 diabetes (due to autoimmune b-cell destruction, usually leading to absolute insulin deficiency)2.
Gestational diabetes mellitus (diabetes diagnosed in the second or third trimester of pregnancy that was not clearly overt diabetes prior to gestation)4.
Occasionally, patients with type 2 diabetes may present with DKA (6), particularly ethnic minorities (7).
In both type 1 and type 2 diabetes, various genetic and environmental factors can result in the progressive loss of b-cell mass and/or function that manifests clinically as hyperglycemia.
Once hyperglycemia occurs, patients with all forms of diabetes are at risk for developing the same chronic complications, although rates of progression may differ.
It is now clear from studies of first-degree relatives of patients with type 1 diabetes that the persistent presence of two or more islet autoantibodies is an almost certain predictor of clinical hyperglycemia and diabetes.
Type 2 diabetes is associated with insulin secretory defects related to infiammation and metabolic stress among other contributors, including genetic factors.
The same tests may be used to screen for and diagnose diabetes and to detect individuals with prediabetes (Table 2.2 and Table 2.5).
Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG may be used to diagnose diabetes (Table 2.2).
A1CRecommendations2.1 To avoid misdiagnosis or missed diagnosis, the A1C test should be performed using a method that is certied by the NGSP and stan- dardized to the Diabetes Control and Complications Trial (DCCT) assay.
Race/Ethnicity/HemoglobinopathiesHemoglobin variants can interfere with the measurement of A1C, although most assays in use in theU.S. are unaffected by the most common variants.
For patients with a hemoglobin variant but normal red blood cell turnover, such as those with the sickle cell trait, an A1C assay without interference from hemoglobin variants should be used.
African Americans heterozygous for the common hemoglobin variant HbS may have, for any given level of mean glycemia, lower A1C by about 0.3% than those without the trait (20).
Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans, was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant (21).
Even in the absence of hemoglobin variants, A1C levels may vary with race/ ethnicity independently of glycemia (22-24).
For example, African Americans may have higher A1C levels than non- Hispanic whites with similar fasting and postglucose load glucose levels (25), and A1C levels may be higher for a given mean glucose concentration when measured with continuous glucose monitoring (26).
Be- cause of the potential for preanalytic variability, it is critical that samples for plasma glucose be spun and separated immediately after they are drawn.
TYPE 1 DIABETESRecommendationsScreening for type 1 diabetes risk with a panel of islet autoantibodies is currently recommended in the setting of a research trial or can be offered as an option for rst-degree family members of a proband with type 1 diabetes.
Although patients are not typically obese when they present with type 1 diabetes, obesity is increasingly common in the general population and there is evidence that it may also be a risk factor for type 1 diabetes.
People with type 1 diabetes are also prone to other autoimmune disorders such as Hashimoto thyroiditis, Graves disease, celiac disease, Addison disease, vitiligo, auto- immune hepatitis, myasthenia gravis, and pernicious anemia (see Section 4 "Comprehensive Medical Evaluation and Assessment of Comorbidities," https://doi.org/10.2337/dc20-S004).
Individuals with autoantibody-negative type 1 diabetes of African or Asian ancestry may suffer from episodic DKA and exhibit varying degrees of insulin deficiency between episodes.
Multiple studies indicate that measuring islet autoantibodies in individuals genetically at risk for type 1 diabetes (e.g., relatives of those with type 1 diabetes or indi- viduals from the general population with type 1 diabetes-associated genetic factors) identifies individuals who may develop type 1 diabetes (9).
Indeed, the risk of type 1 diabetes increases as the number of relevant autoantibodies detected increases (43-45).
PREDIABETES AND TYPE 2 DIABETES2.13 Risk based screening for prediabetes and/or type 2 diabetes should be considered after the onset of puberty or after 10 years of age, whichever occurs earlier, in children and adolescents with overweight (BMI >85th percen- tile) or obesity (BMI >95th per- centile) and who have one or more risk factor for diabetes.
If results are normal, testing should be repeated at a minimum of 3-year intervals, with consideration of more frequent testing depending on initial results and risk status.
It should be noted that theTable 2.4Risk-based screening for type 2 diabetes or prediabetes in asymptomatic children and adolescents in a clinical setting (163)Testingshould be considered in youth* who have overweight (>85th percentile) or obesity (>95th percentile) and who have one or more additional risk factors based on the strength of their association with diabetes:Maternal history of diabetes or GDM during the childs gestation AFamily history of type 2 diabetes in rst- or second-degree relative ARace/ethnicity (Native American, African American, Latino, Asian American, Pacic Islander) ASigns of insulin resistance or conditions associated with insulin resistance (acanthosis nigricans, hypertension, dyslipidemia, polycystic ovary syndrome, or small-for-gestational- age birth weight) BGDM, gestational diabetes mellitus.
If tests are normal, repeat testing at a minimum of 3-year intervals, or more frequently if BMI is increasing, is recommended.
Those with an A1C range of 6.0- 6.5% (42-48 mmol/mol) had a 5-year risk of developing diabetes between 25% and 50% and a relative risk 20 times higher compared with A1C of 5.0% (31 mmol/mol) (49).
Similar to those with IFG and/or IGT, individuals with A1C of 5.7-6.4% (39-47 mmol/mol) should be informed of their increased risk for diabetes and CVD and counseled about effective strategies to lower their risks (see Section 3 "Prevention or Delay of Type 2 Di- abetes," https://doi.org/10.2337/dc20- S003).
Similar to glucose measurements, the continuum of risk is curvilinear, so as A1C rises, the diabetes risk rises disproportionately (49).
Aggressive interventions and vigilant follow-up should be pursued for those considered at very high risk (e.g., those with A1C .6.0% [42 mmol/mol]).
Excess weight itself causes some degree of insulin resistance.
DKA seldom occurs spontaneously in type 2 diabetes; when seen, it usually arises in association with the stress of another illness such as infection or with the use of certain drugs (e.g., corticosteroids, atypical antipsychotics, and sodium-glucose cotransporter 2 inhibitors) (53,54).
The risk of developing type 2 diabetes increases with age, obesity, and lack of physical activity.
After 5.3 years of follow-up, CVD risk factors were modestly but significantly improved with intensive treatment compared with routine care, but the incidence of first CVD events or mortality was not significantly different between the groups (48).
Community ScreeningIdeally, testing should be carried out within a health care setting because of the need for follow-up and treatment.
CYSTIC FIBROSIS-RELATED DIABETESRecommendationsAnnual screening for cystic brosisrelated diabetes (CFRD) with an oral glucose tolerance test should begin by age 10 years in all patients with cystic brosis not previously diagnosed with CFRD.
BPatients with cystic brosis related diabetes should be treated with insulin to attain individualized glycemic goals.
ABeginning 5 years after the diagnosis of cystic brosisrelated diabetes, annual monitoring for complications of diabetes is recommended.
Diabetes in this population, compared with individuals with type 1 or type 2 diabetes, is associated with worse nutritional status, more severe infiammatory lung disease, and greater mortality.
Genetically determined b-cell function and insulin resistance associated with infection and infiammation may also contribute to the development of CFRD.
EThe oral glucose tolerance test is the preferred test to make a diagnosis of posttransplantation diabetes mellitus.
Most studies have reported that transplant patients with hyperglycemia and PTDM after transplantation have higher rates of rejection, infection, and rehospitalization (89,91,96).
Those with previously poor control or with persistent hyperglycemia should continue insulin with frequent home self-monitoring of blood glucose to determine when insulin dose reductions may be needed and when it may be appropriate to switch to non- insulin agents.
Drug dose adjustments may be required because of decreases in the glomerular filtration rate, a relatively common complication in transplant patients.
Thiazolidinediones have been used successfully in patients with liver and kidney transplants, but side effects include fiuid retention, heart failure, and osteopenia (98,99).
Well-designed intervention trials examining the efficacy and safety of these and other antihyperglycemic agents in patients with PTDM are needed.
MONOGENIC DIABETES SYNDROMESRecommendationsAll children diagnosed with diabetes in the rst 6 months of life should have immediate genetic testing for neonatal diabetes.
AChildren and those diagnosed in early adulthood who have diabetes not characteristic of type 1 or type 2 diabetes that occurs in successive generations (suggestive of an autosomal dominant pattern of inheri- tance) should have genetic test- ing for maturity-onset diabetes of the young.
AIn both instances, consultation with a center specializing in diabetes genetics is recommended to understand the signicance of these mutations and how best to approach further evaluation, treatment, and genetic counseling.
EMonogenic defects that cause b-cell dysfunction, such as neonatal diabetes and MODY, represent a small fraction of patients with diabetes (,5%).
Correct diagnosis has critical implications because mostTable 2.6-Most common causes of monogenic diabetes (102)Gene Inheritance Clinical featuresMODYNeonatal diabetesGCK AD GCK-MODY: stable, nonprogressive elevated fasting blood glucose; typically does notrequire treatment; microvascular complications are rare; small rise in 2-h PG level on OGTT (,54 mg/dL [3 mmol/L])HNF1A AD HNF1A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; lowered renal threshold for glucosuria; large rise in 2-h PG level on OGTT (.90 mg/dL [5 mmol/L]); sensitive to sulfonylureasHNF4A AD HNF4A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; may have large birth weight and transient neonatal hypoglycemia; sensitive to sulfonylureasHNF1B AD HNF1B-MODY: developmental renal disease (typically cystic); genitourinary abnormalities; atrophy of the pancreas; hyperuricemia; goutKCNJ11 AD Permanent or transient: IUGR; possible developmental delay and seizures; responsive toSulfonylureasINS AD Permanent: IUGR; insulin requiringABCC8 AD Permanent or transient: IUGR; rarely developmental delay; responsive to sulfonylureas6q24 (PLAGL1,HYMA1)AD for paternal duplicationsTransient: IUGR; macroglossia; umbilical hernia; mechanisms include UPD6, paternal duplication or maternal methylation defect; may be treatable with medications other than insulinGATA6 AD Permanent: pancreatic hypoplasia; cardiac malformations; pancreatic exocrine insufficiency;insulin requiringEIF2AK3 AR Permanent: Wolcott-Rallison syndrome: epiphyseal dysplasia; pancreatic exocrineinsufficiency; insulin requiringFOXP3 X-linked Permanent: immunodysregulation, polyendocrinopathy, enteropathy X-linked (IPEX)syndrome: autoimmune diabetes; autoimmune thyroid disease; exfoliative dermatitis; insulin requiringAD, autosomal dominant; AR, autosomal recessive; IUGR, intrauterine growth restriction; OGTT, oral glucose tolerance test; 2-h PG, 2-h plasma glucose.
patients with KATP-related neonatal diabetes will exhibit improved glycemic control when treated with high-dose oral sulfonylureas instead of insulin.
Insulin gene (INS) mutations are the second most common cause of permanent neonatal diabetes, and, while intensive insulin management is currently the preferred treatment strategy, there are important genetic counseling considerations, as most of the mutations that cause diabetes are dominantly inherited.
Clinically, patients with GCK-MODY exhibit mild, stable, fasting hyperglycemia and do not require antihyperglycemic therapy except sometimes during pregnancy.
Mutations or deletions in HNF1B are associated with renal cysts and uterine malformations (renal cysts and diabetes [RCAD] syndrome).
Additionally, diagnosis can lead to identification of other affected family members.
A diagnosis of MODY should be considered in individuals who have atypical diabetes and multiple family members with diabetes not characteristic of type 1 or type 2 diabetes, although admittedly "atypical diabetes" is becoming increasingly difficult to precisely define in the absence of a definitive set of tests for either type of diabetes (104-110).
In most cases, the presence of autoantibodies for type 1 diabetes precludes further testing for monogenic diabetes, but the presence of autoantibodies in patients with monogenic diabetes has been reported (111).
Individuals in whom monogenic diabetes is suspected should be referred to a specialist for further evaluation if available, and consultation is available from several centers.
It is critical to correctly diagnose one of the monogenic forms of diabetes because these patients may be incorrectly diagnosed with type 1 or type 2 diabetes, leading to suboptimal, even potentially harmful, treatment regimens and delays in diagnosing other family members (114).
Genetic counseling is recommended to ensure that affected individuals understand the patterns of inheritance and the importance of a correct diagnosis.
The diagnosis of monogenic diabetes should be considered in children and adults diagnosed with diabetes in early adulthood with the following findings: Diabetes diagnosed within the first 6 months of life (with occasional cases presenting later, mostly INS and ABCC8 mutations) (103,116) Diabetes without typical features of type 1 or type 2 diabetes (negative diabetes-associated autoantibodies, non- obese, lacking other metabolic features especially with strong family history of diabetes) Stable, mild fasting hyperglycemia (100-150 mg/dL [5.5-8.5 mmol/L]),stable A1C between 5.6 and 7.6%(between 38 and 60 mmol/mol), especially if nonobesePANCREATIC DIABETES OR DIABETES IN THE CONTEXT OF DISEASE OF THE EXOCRINE PANCREASPancreatic diabetes includes both structural and functional loss of glucose- normalizing insulin secretion in the context of exocrine pancreatic dysfunction and is commonly misdiagnosed as type 2 diabetes.
Hyperglycemia due to general pancreatic dysfunction has been called "type 3c diabetes" and, more recently, diabetes in the context of disease of the exocrine pancreas has been termedpancreoprivic diabetes (1).
In the context of pancreatectomy, islet autotransplantation can be done to retain insulin secretion (123,124).
In some cases, this can lead to insulin independence.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the first prenatal visit in those with risk factors using standard diagnostic criteria.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the rst prenatal visit in those with risk factors using standard diagnostic criteria.
BTest for gestational diabetes mellitus at 2428 weeks of gestation in pregnant women not previously found to have diabetes.
ATest women with gestational diabetes mellitus for prediabetes or diabetes at 412 weeks postpartum, using the 75-g oral glucose tolerance test and clinically appropriate nonpregnancy diagnostic criteria.
BWomen with a history of gestational diabetes mellitus should have lifelong screening for the development of diabetes or pre- diabetes at least every 3 years.
BWomen with a history of gestational diabetes mellitus found to have prediabetes should receive intensive lifestyle interventions and/or metformin to prevent diabetes.
Because of the number of pregnant women with undiagnosed type 2 diabetes, it is reasonable to test women with risk factors for type 2 diabetes (133) (Table 2.3) at their initial prenatal visit, using standard diagnostic criteria (Table 2.2).
As effective prevention interventions are available (138,139), women diagnosed with GDM should receive lifelong screening for pre- diabetes to allow interventions to reduce diabetes risk and for type 2 diabetes to allow treatment at the earliest possible time (140).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <S>The USPSTF found that several screening tests (OGCT, OGTT, fasting plasma glucose) and strategies (1- vs 2-step approach)</S> can <A>accurately detect gestational diabetes</A> and does not recommend any single specific test.
2. <C>Type 1 diabetes (due to autoimmune b-cell destruction, usually leading to absolute insulin deficiency)2</C>.
3. <C>Gestational diabetes mellitus (diabetes diagnosed in the second or third trimester of pregnancy that was not clearly overt diabetes prior to gestation)4</C>.
4. Occasionally, <C>patients with type 2 diabetes</C> may present with <E>DKA (6), particularly ethnic minorities (7)</E>.
5. In both <C>type 1 and type 2 diabetes</C>, various genetic and environmental factors can result in the progressive loss of b-cell mass and/or function that manifests clinically as <E>hyperglycemia</E>.
6. Once <C>hyperglycemia occurs</C>, patien

In [ ]:
text = f'''
ImportanceGestational diabetes is diabetes that develops during pregnancy.1-3 Prevalence of gestational diabetes in the US has been estimated at 5.8% to 9.2%, based on traditional diagnostic criteria, although it may be higher if more inclusive criteria are used.4-8 Pregnant persons with gestational diabetesincreased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia.3,9-11 Gestational diabetes has also been associated with an increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring.12-16Table 1.
The same tests may be used to screen for and diagnose diabetes and to detect individuals with prediabetes (Table 2.2 and Table 2.5).
Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG may be used to diagnose diabetes (Table 2.2).
A1CRecommendations2.1 To avoid misdiagnosis or missed diagnosis, the A1C test should be performed using a method that is certied by the NGSP and stan- dardized to the Diabetes Control and Complications Trial (DCCT) assay.
Race/Ethnicity/HemoglobinopathiesHemoglobin variants can interfere with the measurement of A1C, although most assays in use in theU.S. are unaffected by the most common variants.
For patients with a hemoglobin variant but normal red blood cell turnover, such as those with the sickle cell trait, an A1C assay without interference from hemoglobin variants should be used.
African Americans heterozygous for the common hemoglobin variant HbS may have, for any given level of mean glycemia, lower A1C by about 0.3% than those without the trait (20).
Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans, was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant (21).
Even in the absence of hemoglobin variants, A1C levels may vary with race/ ethnicity independently of glycemia (22-24).
For example, African Americans may have higher A1C levels than non- Hispanic whites with similar fasting and postglucose load glucose levels (25), and A1C levels may be higher for a given mean glucose concentration when measured with continuous glucose monitoring (26).
Be- cause of the potential for preanalytic variability, it is critical that samples for plasma glucose be spun and separated immediately after they are drawn.
TYPE 1 DIABETESRecommendationsScreening for type 1 diabetes risk with a panel of islet autoantibodies is currently recommended in the setting of a research trial or can be offered as an option for rst-degree family members of a proband with type 1 diabetes.
Although patients are not typically obese when they present with type 1 diabetes, obesity is increasingly common in the general population and there is evidence that it may also be a risk factor for type 1 diabetes.
People with type 1 diabetes are also prone to other autoimmune disorders such as Hashimoto thyroiditis, Graves disease, celiac disease, Addison disease, vitiligo, auto- immune hepatitis, myasthenia gravis, and pernicious anemia (see Section 4 "Comprehensive Medical Evaluation and Assessment of Comorbidities," https://doi.org/10.2337/dc20-S004).
Individuals with autoantibody-negative type 1 diabetes of African or Asian ancestry may suffer from episodic DKA and exhibit varying degrees of insulin deficiency between episodes.
Multiple studies indicate that measuring islet autoantibodies in individuals genetically at risk for type 1 diabetes (e.g., relatives of those with type 1 diabetes or indi- viduals from the general population with type 1 diabetes-associated genetic factors) identifies individuals who may develop type 1 diabetes (9).
Indeed, the risk of type 1 diabetes increases as the number of relevant autoantibodies detected increases (43-45).
PREDIABETES AND TYPE 2 DIABETES2.13 Risk based screening for prediabetes and/or type 2 diabetes should be considered after the onset of puberty or after 10 years of age, whichever occurs earlier, in children and adolescents with overweight (BMI >85th percen- tile) or obesity (BMI >95th per- centile) and who have one or more risk factor for diabetes.
If results are normal, testing should be repeated at a minimum of 3-year intervals, with consideration of more frequent testing depending on initial results and risk status.
It should be noted that theTable 2.4Risk-based screening for type 2 diabetes or prediabetes in asymptomatic children and adolescents in a clinical setting (163)Testingshould be considered in youth* who have overweight (>85th percentile) or obesity (>95th percentile) and who have one or more additional risk factors based on the strength of their association with diabetes:Maternal history of diabetes or GDM during the childs gestation AFamily history of type 2 diabetes in rst- or second-degree relative ARace/ethnicity (Native American, African American, Latino, Asian American, Pacic Islander) ASigns of insulin resistance or conditions associated with insulin resistance (acanthosis nigricans, hypertension, dyslipidemia, polycystic ovary syndrome, or small-for-gestational- age birth weight) BGDM, gestational diabetes mellitus.
If tests are normal, repeat testing at a minimum of 3-year intervals, or more frequently if BMI is increasing, is recommended.
Those with an A1C range of 6.0- 6.5% (42-48 mmol/mol) had a 5-year risk of developing diabetes between 25% and 50% and a relative risk 20 times higher compared with A1C of 5.0% (31 mmol/mol) (49).
Similar to those with IFG and/or IGT, individuals with A1C of 5.7-6.4% (39-47 mmol/mol) should be informed of their increased risk for diabetes and CVD and counseled about effective strategies to lower their risks (see Section 3 "Prevention or Delay of Type 2 Di- abetes," https://doi.org/10.2337/dc20- S003).
Similar to glucose measurements, the continuum of risk is curvilinear, so as A1C rises, the diabetes risk rises disproportionately (49).
Aggressive interventions and vigilant follow-up should be pursued for those considered at very high risk (e.g., those with A1C .6.0% [42 mmol/mol]).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <O>ImportanceGestational diabetes is diabetes that develops during pregnancy.1-3 Prevalence of gestational diabetes in the US has been estimated at 5.8% to 9.2%, based on traditional diagnostic criteria, although it may be higher if more inclusive criteria are used.4-8</O> <C>Pregnant persons with gestational diabetes</C> <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia.3,9-11</E> <C>Gestational diabetes</C> has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring.12-16Table 1.</E>
2. <O>The same tests may be used to screen for and diagnose diabetes and to detect individuals with prediabetes (Table 2.2 and Table 2.5).</O> <O>Fasting and 2-Hour Pl

In [ ]:
text = f'''Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG may be used to diagnose diabetes (Table 2.2).
Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans, was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant (21).
For example, African Americans may have higher A1C levels than non- Hispanic whites with similar fasting and postglucose load glucose levels (25), and A1C levels may be higher for a given mean glucose concentration when measured with continuous glucose monitoring (26).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Sentence: <O>Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG</O> <C>may be used to diagnose diabetes</C> (Table 2.2).
Prediction: <O>Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG</O> <C>may be used to diagnose diabetes</C> (Table 2.2).

Sentence: <O>Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans,</O> <C>was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant</C> (21).
Prediction: <O>Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans,</O> <C>was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant</C> (21).

Sentence: <O>For example, African Americans may have higher A1C levels than non- Hispanic whites with similar fasting and postglucose load glucose levels (25),</

In [ ]:
text = f'''
Excess weight itself causes some degree of insulin resistance.
DKA seldom occurs spontaneously in type 2 diabetes; when seen, it usually arises in association with the stress of another illness such as infection or with the use of certain drugs (e.g., corticosteroids, atypical antipsychotics, and sodium-glucose cotransporter 2 inhibitors) (53,54).
The risk of developing type 2 diabetes increases with age, obesity, and lack of physical activity.
After 5.3 years of follow-up, CVD risk factors were modestly but significantly improved with intensive treatment compared with routine care, but the incidence of first CVD events or mortality was not significantly different between the groups (48).
Community ScreeningIdeally, testing should be carried out within a health care setting because of the need for follow-up and treatment.
CYSTIC FIBROSIS-RELATED DIABETESRecommendationsAnnual screening for cystic brosisrelated diabetes (CFRD) with an oral glucose tolerance test should begin by age 10 years in all patients with cystic brosis not previously diagnosed with CFRD.
BPatients with cystic brosis related diabetes should be treated with insulin to attain individualized glycemic goals.
ABeginning 5 years after the diagnosis of cystic brosisrelated diabetes, annual monitoring for complications of diabetes is recommended.
Diabetes in this population, compared with individuals with type 1 or type 2 diabetes, is associated with worse nutritional status, more severe infiammatory lung disease, and greater mortality.
Genetically determined b-cell function and insulin resistance associated with infection and infiammation may also contribute to the development of CFRD.
EThe oral glucose tolerance test is the preferred test to make a diagnosis of posttransplantation diabetes mellitus.
Most studies have reported that transplant patients with hyperglycemia and PTDM after transplantation have higher rates of rejection, infection, and rehospitalization (89,91,96).
Those with previously poor control or with persistent hyperglycemia should continue insulin with frequent home self-monitoring of blood glucose to determine when insulin dose reductions may be needed and when it may be appropriate to switch to non- insulin agents.
Drug dose adjustments may be required because of decreases in the glomerular filtration rate, a relatively common complication in transplant patients.
Thiazolidinediones have been used successfully in patients with liver and kidney transplants, but side effects include fiuid retention, heart failure, and osteopenia (98,99).
Well-designed intervention trials examining the efficacy and safety of these and other antihyperglycemic agents in patients with PTDM are needed.
MONOGENIC DIABETES SYNDROMESRecommendationsAll children diagnosed with diabetes in the rst 6 months of life should have immediate genetic testing for neonatal diabetes.
AChildren and those diagnosed in early adulthood who have diabetes not characteristic of type 1 or type 2 diabetes that occurs in successive generations (suggestive of an autosomal dominant pattern of inheri- tance) should have genetic test- ing for maturity-onset diabetes of the young.
AIn both instances, consultation with a center specializing in diabetes genetics is recommended to understand the signicance of these mutations and how best to approach further evaluation, treatment, and genetic counseling.
EMonogenic defects that cause b-cell dysfunction, such as neonatal diabetes and MODY, represent a small fraction of patients with diabetes (,5%).
Correct diagnosis has critical implications because mostTable 2.6-Most common causes of monogenic diabetes (102)Gene Inheritance Clinical featuresMODYNeonatal diabetesGCK AD GCK-MODY: stable, nonprogressive elevated fasting blood glucose; typically does notrequire treatment; microvascular complications are rare; small rise in 2-h PG level on OGTT (,54 mg/dL [3 mmol/L])HNF1A AD HNF1A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; lowered renal threshold for glucosuria; large rise in 2-h PG level on OGTT (.90 mg/dL [5 mmol/L]); sensitive to sulfonylureasHNF4A AD HNF4A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; may have large birth weight and transient neonatal hypoglycemia; sensitive to sulfonylureasHNF1B AD HNF1B-MODY: developmental renal disease (typically cystic); genitourinary abnormalities; atrophy of the pancreas; hyperuricemia; goutKCNJ11 AD Permanent or transient: IUGR; possible developmental delay and seizures; responsive toSulfonylureasINS AD Permanent: IUGR; insulin requiringABCC8 AD Permanent or transient: IUGR; rarely developmental delay; responsive to sulfonylureas6q24 (PLAGL1,HYMA1)AD for paternal duplicationsTransient: IUGR; macroglossia; umbilical hernia; mechanisms include UPD6, paternal duplication or maternal methylation defect; may be treatable with medications other than insulinGATA6 AD Permanent: pancreatic hypoplasia; cardiac malformations; pancreatic exocrine insufficiency;insulin requiringEIF2AK3 AR Permanent: Wolcott-Rallison syndrome: epiphyseal dysplasia; pancreatic exocrineinsufficiency; insulin requiringFOXP3 X-linked Permanent: immunodysregulation, polyendocrinopathy, enteropathy X-linked (IPEX)syndrome: autoimmune diabetes; autoimmune thyroid disease; exfoliative dermatitis; insulin requiringAD, autosomal dominant; AR, autosomal recessive; IUGR, intrauterine growth restriction; OGTT, oral glucose tolerance test; 2-h PG, 2-h plasma glucose.
patients with KATP-related neonatal diabetes will exhibit improved glycemic control when treated with high-dose oral sulfonylureas instead of insulin.
Insulin gene (INS) mutations are the second most common cause of permanent neonatal diabetes, and, while intensive insulin management is currently the preferred treatment strategy, there are important genetic counseling considerations, as most of the mutations that cause diabetes are dominantly inherited.
Clinically, patients with GCK-MODY exhibit mild, stable, fasting hyperglycemia and do not require antihyperglycemic therapy except sometimes during pregnancy.
Mutations or deletions in HNF1B are associated with renal cysts and uterine malformations (renal cysts and diabetes [RCAD] syndrome).
Additionally, diagnosis can lead to identification of other affected family members.
A diagnosis of MODY should be considered in individuals who have atypical diabetes and multiple family members with diabetes not characteristic of type 1 or type 2 diabetes, although admittedly "atypical diabetes" is becoming increasingly difficult to precisely define in the absence of a definitive set of tests for either type of diabetes (104-110).
In most cases, the presence of autoantibodies for type 1 diabetes precludes further testing for monogenic diabetes, but the presence of autoantibodies in patients with monogenic diabetes has been reported (111).
Individuals in whom monogenic diabetes is suspected should be referred to a specialist for further evaluation if available, and consultation is available from several centers.
It is critical to correctly diagnose one of the monogenic forms of diabetes because these patients may be incorrectly diagnosed with type 1 or type 2 diabetes, leading to suboptimal, even potentially harmful, treatment regimens and delays in diagnosing other family members (114).
Genetic counseling is recommended to ensure that affected individuals understand the patterns of inheritance and the importance of a correct diagnosis.
The diagnosis of monogenic diabetes should be considered in children and adults diagnosed with diabetes in early adulthood with the following findings: Diabetes diagnosed within the first 6 months of life (with occasional cases presenting later, mostly INS and ABCC8 mutations) (103,116) Diabetes without typical features of type 1 or type 2 diabetes (negative diabetes-associated autoantibodies, non- obese, lacking other metabolic features especially with strong family history of diabetes) Stable, mild fasting hyperglycemia (100-150 mg/dL [5.5-8.5 mmol/L]),stable A1C between 5.6 and 7.6%(between 38 and 60 mmol/mol), especially if nonobesePANCREATIC DIABETES OR DIABETES IN THE CONTEXT OF DISEASE OF THE EXOCRINE PANCREASPancreatic diabetes includes both structural and functional loss of glucose- normalizing insulin secretion in the context of exocrine pancreatic dysfunction and is commonly misdiagnosed as type 2 diabetes.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the sentences with the tags:

1. <C>Excess weight itself</C> <E>causes some degree of insulin resistance</E>.
2. <C>DKA seldom occurs spontaneously in type 2 diabetes; when seen, it usually arises in association with the stress of another illness such as infection or with the use of certain drugs (e.g., corticosteroids, atypical antipsychotics, and sodium-glucose cotransporter 2 inhibitors) (53,54)</C>.
3. <C>The risk of developing type 2 diabetes</C> <E>increases with age, obesity, and lack of physical activity</E>.
4. <C>After 5.3 years of follow-up, CVD risk factors were modestly but significantly improved with intensive treatment compared with routine care</C>, but <E>the incidence of first CVD events or mortality was not significantly different between the groups (48)</E>.
5. <A>Community ScreeningIdeally, testing should be carried out within a health care setting</A> because of <C>the need for follow-up and treatment</C>.
6. <A>CYSTIC FIBROSIS-RE

In [ ]:
text = f'''
Hyperglycemia due to general pancreatic dysfunction has been called "type 3c diabetes" and, more recently, diabetes in the context of disease of the exocrine pancreas has been termedpancreoprivic diabetes (1).
In the context of pancreatectomy, islet autotransplantation can be done to retain insulin secretion (123,124).
In some cases, this can lead to insulin independence.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the first prenatal visit in those with risk factors using standard diagnostic criteria.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the rst prenatal visit in those with risk factors using standard diagnostic criteria.
BTest for gestational diabetes mellitus at 2428 weeks of gestation in pregnant women not previously found to have diabetes.
ATest women with gestational diabetes mellitus for prediabetes or diabetes at 412 weeks postpartum, using the 75-g oral glucose tolerance test and clinically appropriate nonpregnancy diagnostic criteria.
BWomen with a history of gestational diabetes mellitus should have lifelong screening for the development of diabetes or pre- diabetes at least every 3 years.
BWomen with a history of gestational diabetes mellitus found to have prediabetes should receive intensive lifestyle interventions and/or metformin to prevent diabetes.
Because of the number of pregnant women with undiagnosed type 2 diabetes, it is reasonable to test women with risk factors for type 2 diabetes (133) (Table 2.3) at their initial prenatal visit, using standard diagnostic criteria (Table 2.2).
As effective prevention interventions are available (138,139), women diagnosed with GDM should receive lifelong screening for pre- diabetes to allow interventions to reduce diabetes risk and for type 2 diabetes to allow treatment at the earliest possible time (140).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the tagged sentences:

1. <C>Hyperglycemia due to general pancreatic dysfunction</C> has been called <O>"type 3c diabetes"</O> and, more recently, <O>diabetes in the context of disease of the exocrine pancreas</O> has been termed <O>pancreoprivic diabetes (1)</O>.
2. <C>In the context of pancreatectomy</C>, <A>islet autotransplantation</A> can be done to <E>retain insulin secretion (123,124)</E>.
3. <C>In some cases</C>, this can lead to <E>insulin independence</E>.
4. GESTATIONAL DIABETES MELLITUSRecommendations<A>Test for undiagnosed prediabetes and diabetes at the first prenatal visit in those with risk factors using standard diagnostic criteria</A>.
5. GESTATIONAL DIABETES MELLITUSRecommendations<A>Test for undiagnosed prediabetes and diabetes at the rst prenatal visit in those with risk factors using standard diagnostic criteria</A>.
6. <A>Test for gestational diabetes mellitus at 2428 weeks of gestation in pregnant women not previously found to h

In [ ]:
text = f'''
DiagnosisGDM carries risks for the mother, fetus, and neonate.
The Hyperglycemia and Adverse Pregnancy Outcome (HAPO) study (141), a large-scale multinational cohort study completed by more than 23,000 pregnant women, demonstrated that risk of adverse maternal, fetal, and neonatal outcomes continuously increased as a function of maternal glycemia at 24-28 weeks of gestation, even within ranges previously considerednormal for pregnancy.
This one-step strategy was anticipated to significantly increase the incidence of GDM (from 5-6% to 15- 20%), primarily because only one abnormal value, not two, became sufficient to make the diagnosis (142).
A recent follow-up study of women participating in a blinded study of pregnancy OGTTs found that 11 years after their pregnancies, women who would have been diagnosed with GDM by the one-step approach, as compared with those without, were at 3.4-fold higher risk of developing prediabetes and type 2 diabetes and had children with a higher risk of obesity and increased body fat, suggesting that the larger group of women identified by the one-step approach would benefit from increased screening for diabetes and prediabetes that would accompany a history of GDM(144).
Table 2.7Screening for and diagnosis of GDMOne-step strategyPerform a 75-g OGTT, with plasma glucose measurement when patient is fasting and at 1 and 2 h, at 2428 weeks of gestation in women not previously diagnosed with diabetes.
The OGTT should be performed in the morning after an overnight fast of at least 8 h.
Additional well-designed clinical studies are needed to determine the optimal intensity of monitoring and treatment of women with GDM diagnosed by the one-step strategy (147,148).
The panel recommended a two- step approach to screening that used a 1-h 50-g glucose load test (GLT) followed by a 3-h 100-g OGTT for those who screened positive.
The American College of Obstetricians and Gynecologists (ACOG) recommends any of the commonly used thresholds of 130, 135, or 140 mg/dL for the 1-h 50-g GLT (150).
Treatment of higher-threshold maternal hyperglycemia, as identified by the two-step approach, reduces rates of neonatal macrosomia, large-for- gestational-age births (153), and shoulder dystocia, without increasing small-for- gestational-age births.
ACOG currently supports the two-step approach but notes that one elevated value, as opposed to two, may be used for the diagnosis of GDM (150).
IMPORTANCE Gestational diabetes is associated with several poor health outcomes.
At or after 24 weeks of gestation, oral glucose challenge tests with 140- and 135-mg/dL cutoffs had sensitivities of 82% and 93%, respectively, and specificities of 82% and 79%, respectively, against Carpenter and Coustan criteria, and a test with a 140-mg/dL cutoff had sensitivity of 85% and specificity of 81% against the National Diabetes Group Data criteria.
Based on 8 RCTs and 1 nonrandomized study (n = 3982), treatment was significantly associated with decreased risk of primary cesarean deliveries (relative risk [RR], 0.70 [95% CI, 0.54-0.91]; absolute risk difference [ARD], 5.3%), shoulder dystocia (RR, 0.42 [95% CI, 0.23-0.77]; ARD, 1.3%),macrosomia (RR, 0.53 [95% CI, 0.41-0.68]; ARD, 8.9%), large for gestational age (RR, 0.56[95% CI, 0.47-0.66]; ARD, 8.4%), birth injuries (odds ratio, 0.33 [95% CI, 0.11-0.99]; ARD,0.2%), and neonatal intensive care unit admissions (RR, 0.73 [95% CI, 0.53-0.99]; ARD, 2.0%).
Outcomes related to hyper- tension in pregnancy were classified as preeclampsia, gestational hypertension, or hypertensive disorders in pregnancy (mixed).
One new study (n = 1012)41 found 1-step screening of at-risk women was significantly associated with a reduction in late (>28 weeks of gestation) stillbirth (adjusted odds ratio [OR], 0.68 [95% CI, 0.47 to 0.97]).
The other new study (n = 2780)39 found universal 2-step screening, with early screening offered to women with risk factors, significantly associated with fewer cesarean deliveries (RR, 0.78 [95% CI, 0.66 to 0.92]; ARD, -4.8% [95% CI, -8.2% to -1.5%), birth injuries (fracture or dislocation; RR, 0.47 [95% CI, 0.23 to 0.97]; ARD, -0.9%[95% CI, -1.9% to 0.10%]), and admissions to the neonatal intensive care unit (NICU) (RR, 0.67 [95% CI, 0.58 to 0.78]; ARD, -8.7% [95% CI, -12.3% to -5.2%]) compared with historical controls.
Pre- specified analyses found screening in first trimester was significantly associated with decreased likelihood of NICU admissions vs second-trimester screening but with no significant difference for other outcomes.
No significant differences were found in anxiety and depressive symptoms before and after screening for those with negative or false-positive results in 2 cohort studies (n = 1015).44,48 One study (n = 100)42 found that anxiety symptoms scores were slightly higher (6 points on 60-point scale; P = .007) for women with vs without gestational diabetes immediately after receiving results but not significantly higher at gestational week 36 or 6 weeks postpartum.
One good-quality cohort study (n = 3778)46 found that the association between macrosomia and cesarean delivery in women with normoglycemia or untreated borderline gestational diabetes was not observed in those with treated gestational diabetes, suggesting that a gestational diabetes diagnosis may have increased the propensity to perform cesarean deliveries.
IADPSG vs Carpenter and Coustan ScreeningFive RCTs (n = 25 772)20-24 examined universal screening at 24 to 28 weeks of gestation with the 1-step IADPSG vs 2-step Carpenter and Coustan criteria (Table 1).
In the largest trial, 1-step screening significantly increased risk for neonatal hypoglycemia vs 2-step screening, although this may have been in part due to the routine surveillance of neonates with risk factors including diagnosis of maternal gestational diabetes (eFigure 2 in the Supplement).
In 1 trial (n = 921)24 in which all women randomized to 2-step screening underwent the 100-g OGTT (to assist with blinding), 2-step screening was associated with significantly more testing- related adverse events than 1-step screening (eg, reactive hypoglycemia, vomiting, nausea).
Early vs Usual Timing for Carpenter and Coustan ScreeningOne good-quality RCT (n = 922)19 enrolling obese women found early (14 to 20 weeks) vs usual timing of screening with Carpenter and Coustan criteria potentially associated with increased risk of preeclampsia, but the difference was not statistically significant (RR, 1.42 [95% CI, 0.99 to 2.05]; ARD, 4.0% [95% CI, 0.0% to 8.0%]).
Treatment of gestational diabetes was significantly associated with lower risk of primary cesarean deliveries vs no treatment (3 studies; RR, 0.70 [95% CI, 0.54 to 0.91]; I2 = 0%; ARD, -5.3%[95% CI, -10.3% to -0.24%]).25,32,36 Treatment was also associated, though not significantly, with fewer preterm deliveries (4 studies; RR, 0.75 [95% CI, 0.56 to 1.01]; I2 = 0%; ARD, -2.6%[95% CI, -4.9% to 0.02%])31,32,35,36 (Figure 3).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the sentences with the tags:

1. <C>DiagnosisGDM carries risks for the mother, fetus, and neonate.</C>
2. <C>The Hyperglycemia and Adverse Pregnancy Outcome (HAPO) study (141), a large-scale multinational cohort study completed by more than 23,000 pregnant women, demonstrated that risk of adverse maternal, fetal, and neonatal outcomes continuously increased as a function of maternal glycemia at 24-28 weeks of gestation, even within ranges previously considerednormal for pregnancy.</C>
3. <C>This one-step strategy was anticipated to significantly increase the incidence of GDM (from 5-6% to 15- 20%), primarily because only one abnormal value, not two, became sufficient to make the diagnosis (142).</C>
4. <C>A recent follow-up study of women participating in a blinded study of pregnancy OGTTs found that 11 years after their pregnancies, women who would have been diagnosed with GDM by the one-step approach, as compared with those without, were at 3.4-fold 

In [ ]:
text = f'''
Treatment was not significantly associated with reduced risk of gestational hypertension (2 trials32,35; some imprecision), total cesarean deliveries (8 trials25-29,31,32,35), emergency cesarean deliveries (1 trial27), induction of labor (5 trials25,27,28,32,35), or maternal birth trauma (2 studies27,36).
At or after 24 weeks of gestation, treatment of gestational diabetes was significantly associated with improved health outcomes.
We recommend that preconception counseling be provided to all women with diabetes who are considering pregnancy.
We suggest that women with diabetes seeking to conceive strive to achieve blood glucose and hemoglobin A1C (HbA1C) levels as close to normal as possible when they can be safely achieved without undue hypoglycemia.
We recommend that insulin-treated women with diabetes seeking to conceive be treated with multiple daily doses of insulin or continuous sc insulin infusion in preference to split-dose, premixed insulin therapy, because the former are more likely to allow for the achievement and maintenance of target blood glucose levels preconceptionally and, in the event of pregnancy, are more likely to allow for sufficient flexibility or precise adjustment of insulin therapy.
We suggest that a change to a woman's insulin regimen, particularly when she starts continuous sc in- sulin infusion, be undertaken well in advance of withdrawing contraceptive measures or otherwise trying to conceive to allow the patient to acquire expertise in, and the optimization of, the chosen insulin regimen.
We suggest that insulin-treated women with diabetes seeking to conceive be treated with rapid-acting insulin analog therapy (with insulin aspart or insulin lispro) in preference to regular (soluble) insulin.
(2IQQEE)Folic acid supplementation  1.4.We recommend that beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive, a woman with diabetes take a daily folic acid supplement to reduce the risk of neural tube defects.
We recommend that all women with diabetes who are seeking pregnancy have a detailed ocular assessment by a suitably trained and qualified eye care professional in advance of withdrawing contraceptive measures or otherwise trying to conceive (1IQQQQ), and if retinopathy is documented, the patient should be apprised of the specific risks to her of this worsening during pregnancy.
If the degree of retinopathy warrants therapy, we recommend deferring conception until the retinopathy has been treated and found to have stabilized.
We recommend that women with established retinopathy be seen by their eye specialist every trimester, then within 3 months of delivering, and then as needed.
(1IQEEE)  1.5 c. We suggest that pregnant women with diabetes not known to have retinopathy have ocular assessment performed soon after conception and then periodically as indicated during pregnancy.
(1IQEEE)  1.5 c. We suggest that pregnant women with diabetes not known to have retinopathy have ocular assessment performed soon after conception and then periodically as indicated during pregnancy. (2IQQEE) Renal function (preconception and during pregnancy)  1.6 a.
We suggest that all women with diabetes considering pregnancy have their renal function assessed (by measuring their urine albumin to creatinine ratio, serum creatinine, and estimated glomerular filtration rate [GFR]) in advance of withdrawing contraceptive measures or otherwise trying to conceive.
We recommend that a woman with diabetes who is seeking conception while taking an angiotensin-converting enzyme (ACE) inhibitor or angiotensin-receptor blocker in almost all cases should discontinue the medication before withdrawing contraceptive measures or otherwise trying to conceive.
(Ungraded recommendation)  1.7 d. We recommend when ACE inhibitors or angiotensin-receptor blockers have been continued up to the time of conception that the medication should be withdrawn immediately upon the confirmation of pregnancy.
We recommend that if a woman with diabetes is seeking pregnancy and has CAD, its severity should be ascertained, treatment instituted, and counseling provided as to the potential risks of pregnancy to the woman and fetus before the woman withdraws contraception or otherwise tries to conceive.
We suggest that bile acid-binding resins may be used in women with diabetes to treat hypercholesterolemia; however, this is seldom warranted.
For women with type 1 diabetes seeking conception, we recommend measurement of serum TSH and, if their thyroid peroxidase status is unknown, measurement of thyroid peroxidase antibodies before withdrawing contraceptive measures or otherwise trying to conceive.
We recommend weight reduction before pregnancy for overweight and obese women with diabetes.
(1IQQEE) In the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT) must be performed in the absence of symptoms of hyperglycemia and found to be abnormal on another day to confirm the diagnosis.
We recommend that pregnant women not previously identified (either during testing performed as per recommendation 2.1 or at some other time before 24 weeks gestation) with overt diabetes or gestational diabe- tes be tested for gestational diabetes (see Table 2) by having a 2-hour, 75-g oral glucose tolerance test (OGTT) performed at 24 to 28 weeks gestation.
(1IQQQE)  The 75-g OGTT should be performed after an over- night fast of at least 8 hours (but not more than 14 hours) and without having reduced usual carbohydrate intake for the preceding several days.
The test should be performed with the patient seated, and the patient should not smoke during the test.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the sentences with the tags:

```
<O>Treatment</O> was not significantly <C>associated with reduced risk of gestational hypertension (2 trials32,35; some imprecision), total cesarean deliveries (8 trials25-29,31,32,35), emergency cesarean deliveries (1 trial27), induction of labor (5 trials25,27,28,32,35), or maternal birth trauma (2 studies27,36)</C>.
At or after 24 weeks of gestation, <C>treatment of gestational diabetes</C> was significantly <E>associated with improved health outcomes</E>.
We recommend that <A>preconception counseling be provided to all women with diabetes who are considering pregnancy</A>.
We suggest that <A>women with diabetes seeking to conceive strive to achieve blood glucose and hemoglobin A1C (HbA1C) levels as close to normal as possible when they can be safely achieved without undue hypoglycemia</A>.
We recommend that <A>insulin-treated women with diabetes seeking to conceive be treated with multiple daily doses of insulin or

In [ ]:
text = f'''
(1IQQEE) In the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT) must be performed in the absence of symptoms of hyperglycemia and found to be abnormal on another day to confirm the diagnosis.
We recommend that pregnant women not previously identified (either during testing performed as per recommendation 2.1 or at some other time before 24 weeks gestation) with overt diabetes or gestational diabe- tes be tested for gestational diabetes (see Table 2) by having a 2-hour, 75-g oral glucose tolerance test (OGTT) performed at 24 to 28 weeks gestation.
(1IQQQE)  The 75-g OGTT should be performed after an over- night fast of at least 8 hours (but not more than 14 hours) and without having reduced usual carbohydrate intake for the preceding several days.
The test should be performed with the patient seated, and the patient should not smoke during the test.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Sentence , Prediction

(1IQQEE) In the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT) must be performed in the absence of symptoms of hyperglycemia and found to be abnormal on another day to confirm the diagnosis. , <C>In the case of overt diabetes, but not gestational diabetes</C>, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT) <A>must be performed</A> in the absence of symptoms of hyperglycemia and found to be abnormal on another day <E>to confirm the diagnosis</E>.

We recommend that pregnant women not previously identified (either during testing performed as per recommendation 2.1 or at some other time before 24 weeks gestation) with overt diabetes or gestational diabe- tes be tested for gestational diabetes (see Table 2) by having a 2-hour, 75-g oral glucose tolerance test (OGTT) performed at 24 to 28 weeks g

In [ ]:
text = f'''
One or more abnormal values establishes the diagnosis, with the exception that in the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT), in the absence of symptoms of hyperglycemia, must be performed and found to be abnormal on another day to confirm the diagnosis of overt diabetes.
We recommend that women with gestational diabetes target blood glucose levels as close to normal as possible.
(1IQQQE)  2.3 c. We recommend using blood glucose-lowering pharmacological therapy if lifestyle therapy is insufficient to maintain normoglycemia in women with gestational diabetes.
We recommend that postpartum care for women who have had gestational diabetes should include measurement of fasting plasma glucose or fasting self-monitored blood glucose for 24 to 72 hours after delivery to rule out ongoing hyperglycemiaTable 1.
b Testing should use plasma glucose analyzed at a laboratory, not capillary blood glucose analyzed with a blood glucose meter.
We recommend that a 2-hour, 75-g OGTT should be undertaken 6 to 12 weeks after delivery in women with gestational diabetes to rule out prediabetes or diabetes.
(1IQQQE) If results are normal, we recommend this or other diagnostic tests for diabetes should be repeated periodically as well as before future pregnancies.
We recommend that all women who have had gestational diabetes receive counseling on lifestyle measures to reduce the risk of type 2 diabetes, the need for future pregnancies to be planned, and the need for regular diabetes screening, especially before any future pregnancies.
We recommend self-monitoring of blood glucose in all pregnant women with gestational or overt diabetes (1IQQQQ) and suggest testing before and either 1 or 2 hours after the start of each meal (choosing the postmeal time when it is estimated that peak postprandial blood glucose is most likely to occur) and, as indicated, at bed- time and during the night.
We recommend pregnant women with overt or gestational diabetes strive to achieve a target preprandial blood glucose .
We suggest pregnant women with overt or gestational diabetes strive to achieve target blood glucose levels 1 hour after the start of a meal  (2IQEEE) when these targets can be safely achieved without undue hypoglycemia.
We suggest that continuous glucose monitoring be used during pregnancy in women with overt or gestational diabetes when self-monitored blood glucose levels (or, in the case of the woman with overt diabetes, HbA1C values) are not sufficient to assess glycemic control (including both hyperglycemia and hypoglycemia).
We recommend medical nutrition therapy for all pregnant women with overt or gestational diabetes to help achieve and maintain desired glycemic control while pro- viding essential nutrient requirements.
We suggest that women with overt or gestational diabetes follow the Institute of Medicine revised guidelines for weight gain during pregnancy (1) (Table 4).
We suggest obese women with overt or gestational diabetes reduce their calorie intake by approximately one-third (compared with their usual intake before pregnancy) while maintaining a minimum intake of 1600 to 1800 kcal/d.
We suggest women with overt or gestational diabetes limit carbohydrate intake to 35% to 45% of total calories, distributed in 3 small- to moderate-sized meals and 2 to 4 snacks including an evening snack.
We recommend pregnant women with overt or gestational diabetes should follow the same guidelines for the intake of minerals and vitamins as for women without diabetes (1IQQEE), with the exception of taking folic acid 5 mg daily beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive (see Recommen- dation 1.4).
We suggest that at 12 weeks gestation, the dose of folic acid be reduced to 0.4 to 1.0 mg/d, which should be continued until the completion of breastfeeding.
We suggest that the long-acting insulin analog detemir may be initiated during pregnancy for those women who require basal insulin and for whom neutral protamine Hagedorn (NPH) insulin, in appropriate doses, has previously resulted in, or for whom it is thought NPH insulin may result in, problematic hypoglycemia; insulin detemir may be continued in those women with diabetes already successfully taking insulin detemir before pregnancy.
We suggest that those pregnant women success- fully using insulin glargine before pregnancy may continue it during pregnancy.
(2IQQQO)  5.1 d. We recommend the ongoing use of continuous sc insulin infusion during pregnancy in women with diabetes when this has been initiated before pregnancy (1IQQQE), but suggest that continuous sc insulin infusion not be initiated during pregnancy unless other insulin strategies including multiple daily doses of insulin have first been tried and proven unsuccessful.
We suggest that metformin therapy be used for glycemic control only for those women with gestational diabetes who do not have satisfactory glycemic control despite medical nutrition therapy and who refuse or can- not use insulin or glyburide and are not in the first trimester.
Evidence  Women with diabetes who receive preconception counselinghave better preconception glycemic control (8, 9) and are more likely to have favorable pregnancy outcomes, including lower rates of congenital anomalies (9, 10) and spontaneous abortions (11, 12).
Evidence  Maternal hyperglycemia in the first few weeks of pregnancy increases the risk of fetal malformations, spontaneous abortions, and perinatal mortality (14 -18).
We suggest that women with diabetes successfully using the long-acting insulin analogs insulin detemir or insulin glargine preconceptionally may continue with this therapy before and then during pregnancy.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <C>One or more abnormal values establishes the diagnosis</C>, with the exception that in the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT), in the absence of symptoms of hyperglycemia, must be performed and found to be abnormal on another day to <E>confirm the diagnosis of overt diabetes</E>.

2. <A>We recommend that women with gestational diabetes target blood glucose levels as close to normal as possible</A>.

3. <C>We recommend using blood glucose-lowering pharmacological therapy if lifestyle therapy is insufficient to maintain normoglycemia in women with gestational diabetes</C>.

4. <A>We recommend that postpartum care for women who have had gestational diabetes should include measurement of fasting plasma glucose or fasting self-monitored blood glucose for 24

In [ ]:
text = f'''
(2IQQEE)1.3a- d. Evidence  Rapid-acting insulin analogs are likely more able than regular human insulin to help a woman achieve postprandial blood glucose targets and are less likely to cause hypoglycemia; fetal outcomes, however, seem comparable (28 -30).
We recommend that beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive, a woman with diabetes take a daily folic acid supplement to reduce the risk of neural tube defects.
Evidence  Taking a daily folic acid supplement preconceptionally reduces the risk of neural tube defects (38).
The absence of retinopathy before conception confers very small risk of development of significant retinopathy during pregnancy; nonetheless, significant retinopathy can develop and progress during pregnancy, even if not identified preconceptionally (40).
Additional risk factors for progression of retinopathy during pregnancy include preexisting hypertension (43), poorly controlled hypertension during pregnancy (44), preeclampsia (45), and poor glycemic control at the onset of and during pregnancy (41).
We suggest that all women with diabetes considering pregnancy have their renal function assessed (by measuring their urine albumin to creatinine ratio, serum creatinine, and estimated GFR) in advance of withdrawing contraceptive measures or otherwise trying to conceive.
(Ungraded recommendation) We suggest that a woman with diabetes who has a significantly reduced GFR be assessed by a nephrologist before pregnancy, both for baseline renal assessment and to review the woman's specific risk of worsening renal function in the event of pregnancy.
We suggest that all women with diabetes and preconceptional renal dysfunction have their renal function monitored regularly during pregnancy.
Evidence  Renal dysfunction in a pregnant woman with type 1 diabetes is associated with an increased risk of adverse maternal and fetal outcomes, including an increased risk of preeclampsia (46 - 48).
(1IQQEE)  1.7 c. We suggest that in the exceptional case where the degree of renal dysfunction is severe and there is uncertainty about when conception will occur, physicians and patients be engaged in shared decision-making about whether to continue ACE inhibitors or angiotensin-receptor blockers.
(Ungraded recommendation)  1.7 d. We recommend that when ACE inhibitors or angiotensin-receptor blockers have been continued up to the time of conception that the medication should be withdrawn immediately upon the confirmation of pregnancy.
Hypertension in a preconceptional woman increases the risk of adverse outcomes during pregnancy, especially her risk of developing preeclampsia (55).
We recommend that if a woman with diabetes has sufficient numbers of vascular risk factors (particularly the duration of the woman's diabetes and her age), screening studies for CAD be undertaken in advance of withdrawing contraceptive measures or otherwise trying to conceive.
Evidence  Myocardial infarction during pregnancy is associated with adverse maternal and fetal outcomes including maternal and fetal demise (57, 58).
Evidence  Autoimmune thyroid disease is common among women of childbearing age with type 1 diabetes with prevalence rates as high as 44% (62).
A systematic review and meta-analysis (70) demonstrated that abnormal screening test results were associated with worse maternal and fetal outcomes.
Remarks  We acknowledge that with universal testing for diabetes in early pregnancy, there will be a high rate of false- positive results (70) and that women with positive testing may have anxiety and will suffer the burden of additional testing.
Nevertheless, we recommended universal testing because we place the highest value on preventing fetal complications.
We recommend that pregnant women not previously identified (either during testing performed as per recommendation 2.1 or at some other time before 24 weeks gestation) with overt diabetes or gestational diabetes be tested for gestational diabetes (see Table 2) by having a 2-hour, 75-g OGTT performed at 24 to 28 weeks gestation.
Evidence  Pregnant women who develop gestational diabetes are at risk of adverse pregnancy outcomes, which may be prevented by adequate treatment (6, 71).
To exclude from the definition of gestational diabetes those women with overt diabetes, most of our committee supports redefining gestational diabetes as defined in the Hyperglycemia and Adverse Pregnancy Outcome study; that is, gestational diabetes is "the condition associated with degrees of maternal hyperglycemia less severe than those found in overt diabetes but associated with an increased risk of adverse pregnancy outcomes" (6).
It is recognized that implementation of the IADPSG criteria will lead to a substantial increase in the numbers of pregnant women being diagnosed with gestational diabetes with the attendant medicalization of pregnancies and with a concomitant increase in healthcare costs both to individuals and to society.
We recommend using blood glucose-lowering pharmacological therapy if lifestyle therapy is insufficient to maintain normoglycemia in women with gestational diabetes.
Lifestyle therapy for gestational diabetes results in a lower incidence of reduced birth weight, large-for-gestational-age births, and preeclampsia (71, 82).
Both aerobic exercise (83- 86) and non-weight-bearing exercise (87) have been shown to lower blood glucose levels in women with gestational diabetes.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <C>Rapid-acting insulin analogs</C> are likely more able than regular human insulin to help a woman achieve postprandial blood glucose targets and are less likely to cause hypoglycemia; <E>fetal outcomes, however, seem comparable</E> (28 -30).
2. We recommend that beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive, <A>a woman with diabetes take a daily folic acid supplement</A> to <E>reduce the risk of neural tube defects</E>.
3. <C>Taking a daily folic acid supplement preconceptionally</C> <E>reduces the risk of neural tube defects</E> (38).
4. The absence of retinopathy before conception confers very small risk of development of significant retinopathy during pregnancy; nonetheless, <E>significant retinopathy can develop and progress during pregnancy, even if not identified preconceptionally</E> (40).
5. <C>Additional risk

In [ ]:
text = f'''
(2IQQEE)2.4a- e. Evidence  Women who have had gestational diabetes are at high risk for the later development of impaired fasting glucose, impaired glucose tolerance, overt diabetes (91-93), and the metabolic syndrome (91, 94 -106).
Infants born to mothers with gestational diabetes are at increased risk of the later development of obesity or type 2 diabetes (107).
Pregnant women with type 1 diabetes are at increased risk of hypoglycemia including severe hypoglycemia, especially during the first trimester (21-25).
There is also an increased risk of hypoglycemia in pregnant women with type 2 diabetes (26).
Nevertheless, nutrition therapy has been shown to improve glycemic control for people living with overt diabetes (119, 120) and for women with gestational diabetes (121).
Evidence  In the absence of definitive evidence regarding optimal weight gain for women with gestational or overt diabetes-and with evidence both that women who gain excess weight during pregnancy may retain it after childbirth(123) and that women who are overweight or obese before pregnancy are at an increased risk for complications during pregnancy (including hypertensive complications, stillbirth, and increased risk for cesarean section) (124 - 127)-and with the reassurance that limiting maternal weight gain is not associated with a decrease in fetal birth weight (128), we conclude that following the Institute of Medicine recommendations for weight gain during pregnancy, although not written specifically for women with overt or gestational diabetes, is nonetheless appropriate for women with these conditions (Table 4) (129).
Moderate energy restriction (1600 -1800 kcal/d) in pregnant women with overt diabetes improves mean glycemia and fasting insulinemia without inhibiting fetal growth or birth weight or inducing ketosis (128).
Energy intake of approximately 2050 kcal in all BMI categories in women with gestational diabetes has been reported to limit maternal weight gain, maintain euglycemia, avoid ketonuria, and maintain an average birth weight of 3542 g (130).
Some authorities suggest that a minimum of 175 g/d carbohydrate should be provided, which is higher than the 130 g/d recommended for nonpregnant women (1).
We recommend pregnant women with overt or gestational diabetes should follow the same guidelines for the intake of minerals and vitamins as for women without diabetes (1IQQEE), with the exception of taking folic acid 5 mg daily beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive (see Recommendation 1.4).
We suggest that at 12 weeks gestation, the dose of folic acid be reduced to 0.4 to 1.0 mg/d, which should be continued until the completion of breastfeeding (2IQQEE).
We suggest that the long-acting insulin analog detemir may be initiated during pregnancy for those women who require basal insulin and for whom NPH insulin, in appropriate doses, has previously resulted in, or for whom it is thought NPH insulin may result in, problematic hypoglycemia; insulin detemir may be continued in those women with diabetes already successfully taking insulin detemir before pregnancy.
We suggest that those pregnant women successfully using insulin glargine before pregnancy may continue it during pregnancy.
We suggest that the rapid-acting insulin analogs lispro and aspart be used in preference to regular (soluble) insulin in pregnant women with diabetes.
We recommend the ongoing use of continuous sc insulin infusion during pregnancy in women with diabetes when this has been initiated before pregnancy (1IQQQE) but suggest that continuous sc insulin infusion not be initiated during pregnancy unless other insulin strategies including multiple daily doses of insulin have first been tried and proven unsuccessful.
Evidence  In nonpregnant women, insulin detemir is associated with less hypoglycemia than NPH insulin (141-143).
Additionally, compared with multiple daily doses of insulin, continuous sc insulin in- fusion provides greater lifestyle flexibility, easier blood glucose management in women experiencing morning nausea, less blood glucose variability, and facilitates man- aging glucose control in the peridelivery setting (170).
5.1 d. Remarks  Owing to the potential risk of temporarily worsened blood glucose control, ketoacidosis, and hypoglycemia when continuous sc insulin infusion is initiated, its use during pregnancy should be limited to those patients already successfully using this method of insulin administration before pregnancy and to those women who, during pregnancy, have not succeeded with other insulin strategies including multiple daily doses of insulin.
We suggest that glyburide (glibenclamide) is a suitable alternative to insulin therapy for glycemic control in women with gestational diabetes who fail to achieve sufficient glycemic control after a 1-week trial of medical nutrition therapy and exercise except for those women with a diagnosis of gestational diabetes before 25 weeks gestation and for those women with fasting plasma glucose levels >110 mg/dL (6.1 mmol/L), in which case insulin therapy is preferred.
Evidence  In pregnant women taking glyburide, the umbilical cord glyburide concentration is undetectable (90, 173- 176) or, at most, very low (177).
Glyburide is effective in controlling blood glucose in women with gestational diabetes and has been associated with favorable neonatal outcomes including the rate of large-for-gestational-age infants, macrosomia, neonatal intensive care unit admission, and neonatal hypoglycemia (90).
Compared with women with gestational diabetes taking insulin, those taking metformin have no difference in maternal glycemic control, significantly lower rates of neonatal hypoglycemia, and no increased risk of congenital anomalies or other serious maternal or neonatal adverse events.
Additionally, metformin-treated women with gestational diabetes have increased rates of preterm birth (89).
Evidence  Elevated maternal blood glucose during labor and delivery increases the risk of neonatal hypoglycemia and fetal distress (195-200) as well as birth asphyxia and ab- normal fetal heart rate (200, 201), albeit with these associations having been mainly demonstrated in obser- vational studies of women with type 1 diabetes.
We recommend whenever possible women with overt or gestational diabetes should breastfeed their infant.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <C>Women who have had gestational diabetes</C> are at <E>high risk for the later development of impaired fasting glucose, impaired glucose tolerance, overt diabetes (91-93), and the metabolic syndrome (91, 94 -106)</E>.
2. <C>Infants born to mothers with gestational diabetes</C> are at <E>increased risk of the later development of obesity or type 2 diabetes (107)</E>.
3. <C>Pregnant women with type 1 diabetes</C> are at <E>increased risk of hypoglycemia including severe hypoglycemia, especially during the first trimester (21-25)</E>.
4. There is also an <E>increased risk of hypoglycemia</E> in <C>pregnant women with type 2 diabetes (26)</C>.
5. <C>Nutrition therapy</C> has been shown to <E>improve glycemic control for people living with overt diabetes (119, 120) and for women with gestational diabetes (121)</E>.
6. <C>In the absence of definitive evidence regarding 

In [ ]:
text = f'''
We recommend that breastfeeding women with overt diabetes successfully using metformin or glyburide therapy during pregnancy should continue to use these medications, when necessary, during breastfeeding.
Evidence  The increased risk of infants born to women with diabetes for childhood obesity and the later development of impaired glucose intolerance and diabetes (81) is reduced by breastfeeding (202-211).
Breastfeeding may also facilitate postpartum weight loss and reduce maternal and neonatal risk for the later development of type 2 diabetes (212, 213).
Evidence  Combined oral contraceptive use by women with type 1 diabetes does not affect their glycemic control or increase their risk of end-organ injury (217-219).
We suggest women with type 1 diabetes be screened for postpartum thyroiditis with a TSH at 3 and 6 months postpartum.
SUMMARY OF RECOMMENDATION AND EVIDENCE    The USPSTF recommends screening for abnormal blood glucose as part of cardiovascular risk assessment in adults aged 40 to 70 years who are overweight or obese.
Recommendation: The USPSTF recommends screening for abnormal blood glucose as part of cardiovascular risk assessment in adults aged 40 to 70 years who are overweight or obese.
Modifiable cardiovascular risk factors include abnormal blood glucose, hypertension, hyperlipidemia or dyslipidemia, smoking, overweight and obesity, physical inactivity, and an unhealthy diet.
PopulationAdults aged 40 to 70 y who are overweight or obeseRecommendationScreen for abnormal blood glucose.
Treatment and InterventionsEffective behavioral interventions combine counseling on a healthful diet and physical activity and involve multiple contacts over extended periods.
The USPSTF previously found adequate evidence that intensive behavioral counseling interventions for persons at increased risk for CVD have moderate benefits in lowering CVD risk.
Populations in which these benefits have been shown include persons who are obese or overweight and have hypertension, hyperlipidemia or dyslipidemia, and/or IFG or IGT.
Harms of Early Detection and Treatment    The USPSTF found that measuring blood glucose is associated with short-term anxiety but not long-term psychological harms.
The harms of drug therapy for the prevention of diabetes are small to moderate, depending on the drug and dosage used.
Persons who have a family history of diabetes, have a history of gestational diabetes or polycystic ovarian syndrome, or are members of certain racial/ ethnic groups (that is, African Americans, American In- dians or Alaskan Natives, Asian Americans, Hispanics or Latinos, or Native Hawaiians or Pacific Islanders) may be at increased risk for diabetes at a younger age or at a lower body mass index.
Although direct evidence that preventing a diagnosis of type 2 diabetes results in improved health outcomes is limited, primary prevention that reduces the chances of a diagnosis may reduce the adverse consequences of disease management.
Cohort and modeling studies suggest that re- screening every 3 years may be a reasonable approach for adults with normal blood glucose levels (3-7).
Other Approaches to Prevention    Because overweight and obesity, physical inactivity, abnormal lipid levels, high blood pressure, and smoking are all modifiable risk factors for cardiovascular events, the USPSTF recommends screening and appropriate interventions for these conditions (available at www.uspreventiveservicestaskforce.org).
The USPSTF recommends screening for obesity in adults and offering or referring those with a body mass index of 30 kg/m2 or greater to intensive, multicomponent behavioral interventions.
The USPSTF recommends offering or referring adults who are overweight (body mass index >25 kg/ m2) and have additional cardiovascular risk factors to intensive behavioral counseling interventions to promote a healthful diet and physical activity for CVD prevention.
The USPSTF recommends screening for lipid disorders in men aged 35 years or older and women aged 45 years or older who are at increased risk for coronary heart disease.
The USPSTF also recommends screening for hypertension in adults aged 18 years or older and that clinicians ask all adults about tobacco use and provide tobacco cessation interventions to those who use tobacco products.
Useful Resources    The Community Preventive Services Task Force recommends combined diet and physical activity promotion programs for persons who are at increased risk for type 2 diabetes.
Clinical trials and additional modeling studies are needed to better elucidate the optimal frequency of screening and the age at which to start screening.
More U.S. data are also needed on the benefits and harms of lifestyle interventions and medical treatments for screen-detected IFG, IGT, and diabetes over a longer follow-up period.
Uncontrolled diabetes is a leading cause of cardiovascular mortality and morbidity and may also result in other complications, such as vision loss, renal failure, and amputation.
The USPSTF determined that there is adequate evidence that lifestyle interventions can prevent or delay progression to type 2 diabetes.
Metformin, thiazolidinediones, and aglucosidase inhibitors were all found to be effective in preventing or delaying progression.
The Diabetes Prevention Program (11), conducted in the United States in 3234 participants, found that an intensive lifestyle modification intervention was associated with better quality of life at 3-year follow-up.
Intensive glycemic control was associated with reduced risk for nonfatal myocardial infarction and microvascular disease but increased risk for severe hypoglycemia (15, 16).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the tagged sentences:

1. <A>We recommend that breastfeeding women with overt diabetes successfully using metformin or glyburide therapy during pregnancy should continue to use these medications, when necessary, during breastfeeding.</A>
2. <C>Evidence  The increased risk of infants born to women with diabetes for childhood obesity and the later development of impaired glucose intolerance and diabetes (81)</C> is <E>reduced by breastfeeding (202-211)</E>.
3. <C>Breastfeeding</C> may also <E>facilitate postpartum weight loss and reduce maternal and neonatal risk for the later development of type 2 diabetes (212, 213)</E>.
4. <C>Evidence  Combined oral contraceptive use by women with type 1 diabetes</C> does not <E>affect their glycemic control or increase their risk of end-organ injury (217-219)</E>.
5. <A>We suggest women with type 1 diabetes be screened for postpartum thyroiditis with a TSH at 3 and 6 months postpartum.</A>
6. <A>SUMMARY OF RECOMMENDA

In [ ]:
text = f'''
However, a consistent benefit of stroke reduction was found in study groups with intensive versus standard blood pressure control (23, 24).
The ACCORD study (25) found that intensive lipid-lowering therapy was associated with reduced risk for a composite cardiovascular outcome in men with diabetes but possible harm in women (interaction P = 0.01) (25).
One trial found that acarbose was associated with higher risk for withdrawal because of adverse events compared with placebo (42).
One large trial found that nateglinide (43) was associated with increased risk for hypoglycemia compared with placebo (RR, 1.73 [CI, 1.57 to 1.92]); further, valsartan was associated (44) with hypotension related adverse events (RR, 1.16 [CI, 1.11 to 1.23]).
One trial (45) found that rosiglitazone was associated with increased risk for congestive heart failure (HR, 7.04 [CI, 1.6 to 31]), but the estimate was imprecise.
UPDATE OF PREVIOUS USPSTF RECOMMENDATION    This is an update of the 2008 USPSTF recommendation statement in which the USPSTF recommended screening for diabetes in asymptomatic adults with hypertension (defined as sustained blood pressure of>135/80 mm Hg) (B recommendation).
RECOMMENDATIONS OF OTHERS    The American Diabetes Association (46) recommends screening for diabetes in adults aged 45 years or older and screening in persons with multiple risk factors regardless of age.
(27) found that maternal plasma glucose concentration during pregnancy (measured 2 h after a 75-g load) had a continuous association with adverse pregnancy outcomes (LGA and cesarean section).
(16) found risk of severe macrosomia, neonatal hypoglycemia, and hyperbilirubinemia increased with increasing number of abnormal glucose values according to current American Diabetes Association cut points (2,5) among women who did not meet National Diabetes Data Group criteria for GDM (31).
Adiposity in offspring at 5-7 years of age was significantly associated with measures of maternal glycemia (50-g glucose challenge and/or 100-g OGTT) during pregnancy.
Fetal macrosomia (LGA) is a major indicator of the effects of hyperglycemia during pregnancy (12,35,36).
Long-term risks associated with fetal macrosomia in infants of women with GDM (independent of confounders) include childhood overweight (43,44) and metabolic factors that may increase risk of cardiovascular disease (CVD) (45).
Measuring FPG alone identified 8.3% of the cohort as having GDM.
In addition to the outcomes used to deter- mine the thresholds, there were strong associations between maternal glucose and preeclampsia (ORs 1.40 -1.57) and shoulder dystocia and/or birth injury (1.30 -1.43).
The frequency of birth weight, C-peptide, or percent infant body fat >90th percentile was approximately twofold greater when any of the glucose values were greater than or equal to the threshold.
The frequency of preeclampsia was twofold higher when one or more glucose values met or exceeded threshold, and frequencies of preterm delivery and primary cesarean section were >45% higher.
In both randomized controlled trials, treatment, achieved primarily by diet/lifestyle modification, resulted in reduced birth weight and frequency of LGA births and pre- eclampsia.
As ongoing epidemics of obesity and diabetes result in more type 2 diabetes in young women, the number who are undiagnosed (before pregnancy) is increasing (49,50).
IADPSG Consensus Panel members favored use of any available certified laboratory measure of glucose (FPG, random plasma glucose, or A1C) for initial detection of possible cases.
An expert committee recently recommended that an A1C value >6.5% (measured in a laboratory standardized/aligned with the Diabetes Control and Complications Trial [DCCT]/UK Prospective Diabetes Study [UKPDS] assay) be used for diagnosis of diabetes outside pregnancy (55).
It is desirable to detect overt diabetes in pregnancy as early as possible to provide an opportunity to optimize pregnancy outcome.
However, if enrollment is at 24 weeks' gestation or later and overt diabetes is not found, the initial test should be followed by a 75-g OGTT.
It was recognized that any assessment of glycemia in early pregnancy would also result in detection of milder degrees of hyperglycemia short of overt diabetes.
Recently, it was reported that higher first trimester FPG levels (lower than those diagnostic of diabetes) are associated with increased risks of later diagnosis of GDM and adverse pregnancy outcomes (57).
Table 2-Strategy for the detection and diagnosis of hyperglycemic disorders in pregnancy*First prenatal visitMeasure FPG, A1C, or random plasma glucose on all or only high-risk women If results indicate overt diabetes as per Table 1  Treatment and follow-up as for preexisting diabetes If results not diagnostic of overt diabetesand fasting plasma glucose >5.1 mmol/l (92 mg/dl) but , diagnose as GDMand fasting plasma glucose , test for GDM from 24 to 28 weeks' gestation with a 75-g OGTT24-28 weeks' gestation: diagnosis of GDM2-h 75-g OGTT: perform after overnight fast on all women not previously found to have overt diabetes or GDM during testing earlier in this pregnancyOvert diabetes if fasting plasma glucose >7.0 mmol/l (126 mg/dl)GDM if one or more values equals or exceeds thresholds indicated in Table 1 Normal if all values on OGTT less than thresholds indicated in Table 1*To be applied to women without known diabetes antedating pregnancy.
Universal early testing in populations with a high prevalence of type 2 diabetes is recommended, especially if metabolic testing in this age-group is not commonly performed outside of pregnancy.
Well-designed studies should be conducted to determine whether it is beneficial and cost-effective to perform an OGTT in women who do not have overt diabetes at early testing but have indeterminate nondiagnostic results.
Glucose testing early in pregnancy to detect overt diabetes and again with a 75-g OGTT at 24 -28 weeks' of gestation in all pregnancies not already diagnosed with overt diabetes or GDM by early testing represents fundamental changes in strategies for detection and diagnosis of hyperglycemia in pregnancy.
Clinical Preventive Service RecommendationDiabetes Screening, AdultsType 2 Diabetes, AdultsThe AAFP recommends screening for type 2 diabetes in adults aged 40 to 70 years who have overweight or obesity.
The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.1The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.2  Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.
While AAFP acknowledges that diabetes, like all chronic conditions, exists on a continuum and that insulin resistance is a risk factor for developing diabetes, the current evidence does not show improvement in long term health outcomes for screening for prediabetes in adults who have obesity or overweight.
The evidence did show a reduction in the incidence of diabetes in individuals at risk for diabetes who received intense lifestyle interventions or metformin, however there was not an associated improvement in health outcomes.
Lifestyle interventions to prevent progression for prediabetes to diabetes are effective in reducing weight, blood pressure and cholesterol and would likely be recommended and beneficial for these individuals, even in the absence of screening.
'''

In [ ]:
text = f'''
The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.1The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.2  Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.
While AAFP acknowledges that diabetes, like all chronic conditions, exists on a continuum and that insulin resistance is a risk factor for developing diabetes, the current evidence does not show improvement in long term health outcomes for screening for prediabetes in adults who have obesity or overweight.
The evidence did show a reduction in the incidence of diabetes in individuals at risk for diabetes who received intense lifestyle interventions or metformin, however there was not an associated improvement in health outcomes.
Lifestyle interventions to prevent progression for prediabetes to diabetes are effective in reducing weight, blood pressure and cholesterol and would likely be recommended and beneficial for these individuals, even in the absence of screening.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<Sentence>The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.</Sentence>
<Prediction>
<O>The AAFP also agrees that</O> <C>there is insufficient evidence for screening prior to 24 weeks gestation.</C>
</Prediction>

<Sentence>The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.</Sentence>
<Prediction>
<O>The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that</O> <CO>nonpregnant adults aged 40 to 70 who have obesity or overweight</CO> <A>be screened for type 2 diabetes</A> <O>as part of risk assessment for cardiovascular disease.</O>
</Prediction>

<Sentence>Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.</Se

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the sentences with the tags:

```
<O>However,</O> a <C>consistent benefit of stroke reduction was found in study groups with intensive versus standard blood pressure control (23, 24)</C>.
The <C>ACCORD study (25) found that intensive lipid-lowering therapy was associated with reduced risk for a composite cardiovascular outcome in men with diabetes but possible harm in women (interaction P = 0.01) (25)</C>.
One trial found that <C>acarbose was associated with higher risk for withdrawal because of adverse events compared with placebo (42)</C>.
One large trial found that <C>nateglinide (43) was associated with increased risk for hypoglycemia compared with placebo (RR, 1.73 [CI, 1.57 to 1.92]); further, valsartan was associated (44) with hypotension related adverse events (RR, 1.16 [CI, 1.11 to 1.23])</C>.
One trial (45) found that <C>rosiglitazone was associated with increased risk for congestive heart failure (HR, 7.04 [CI, 1.6 to 31]), but the estimate 

In [ ]:
text = f'''
Similar to glucose measurements, the continuum of risk is curvilinear, so as A1C rises, the diabetes risk rises disproportionately (49).
(1IQQEE)  1.7 c. We suggest that in the exceptional case where the degree of renal dysfunction is severe and there is uncertainty about when conception will occur, physicians and patients be engaged in shared decision-making about whether to continue ACE inhibitors or angiotensin-receptor blockers.
Remarks  We acknowledge that with universal testing for diabetes in early pregnancy, there will be a high rate of false- positive results (70) and that women with positive testing may have anxiety and will suffer the burden of additional testing.
Nevertheless, we recommended universal testing because we place the highest value on preventing fetal complications.
To exclude from the definition of gestational diabetes those women with overt diabetes, most of our committee supports redefining gestational diabetes as defined in the Hyperglycemia and Adverse Pregnancy Outcome study; that is, gestational diabetes is "the condition associated with degrees of maternal hyperglycemia less severe than those found in overt diabetes but associated with an increased risk of adverse pregnancy outcomes" (6).
It is recognized that implementation of the IADPSG criteria will lead to a substantial increase in the numbers of pregnant women being diagnosed with gestational diabetes with the attendant medicalization of pregnancies and with a concomitant increase in healthcare costs both to individuals and to society.
Additionally, compared with multiple daily doses of insulin, continuous sc insulin in- fusion provides greater lifestyle flexibility, easier blood glucose management in women experiencing morning nausea, less blood glucose variability, and facilitates man- aging glucose control in the peridelivery setting (170).
5.1 d. Remarks  Owing to the potential risk of temporarily worsened blood glucose control, ketoacidosis, and hypoglycemia when continuous sc insulin infusion is initiated, its use during pregnancy should be limited to those patients already successfully using this method of insulin administration before pregnancy and to those women who, during pregnancy, have not succeeded with other insulin strategies including multiple daily doses of insulin.
IADPSG Consensus Panel members favored use of any available certified laboratory measure of glucose (FPG, random plasma glucose, or A1C) for initial detection of possible cases.
However, if enrollment is at 24 weeks' gestation or later and overt diabetes is not found, the initial test should be followed by a 75-g OGTT.
It was recognized that any assessment of glycemia in early pregnancy would also result in detection of milder degrees of hyperglycemia short of overt diabetes.
Table 2-Strategy for the detection and diagnosis of hyperglycemic disorders in pregnancy*First prenatal visitMeasure FPG, A1C, or random plasma glucose on all or only high-risk women If results indicate overt diabetes as per Table 1  Treatment and follow-up as for preexisting diabetes If results not diagnostic of overt diabetesand fasting plasma glucose >5.1 mmol/l (92 mg/dl) but , diagnose as GDMand fasting plasma glucose , test for GDM from 24 to 28 weeks' gestation with a 75-g OGTT24-28 weeks' gestation: diagnosis of GDM2-h 75-g OGTT: perform after overnight fast on all women not previously found to have overt diabetes or GDM during testing earlier in this pregnancyOvert diabetes if fasting plasma glucose >7.0 mmol/l (126 mg/dl)GDM if one or more values equals or exceeds thresholds indicated in Table 1 Normal if all values on OGTT less than thresholds indicated in Table 1*To be applied to women without known diabetes antedating pregnancy.
Well-designed studies should be conducted to determine whether it is beneficial and cost-effective to perform an OGTT in women who do not have overt diabetes at early testing but have indeterminate nondiagnostic results.
Glucose testing early in pregnancy to detect overt diabetes and again with a 75-g OGTT at 24 -28 weeks' of gestation in all pregnancies not already diagnosed with overt diabetes or GDM by early testing represents fundamental changes in strategies for detection and diagnosis of hyperglycemia in pregnancy.
The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.1The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.2  Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.
While AAFP acknowledges that diabetes, like all chronic conditions, exists on a continuum and that insulin resistance is a risk factor for developing diabetes, the current evidence does not show improvement in long term health outcomes for screening for prediabetes in adults who have obesity or overweight.
The evidence did show a reduction in the incidence of diabetes in individuals at risk for diabetes who received intense lifestyle interventions or metformin, however there was not an associated improvement in health outcomes.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <C>It is now clear from studies of first-degree relatives of patients with type 1 diabetes that the persistent presence of two or more islet autoantibodies</C> is an almost certain predictor of <E>clinical hyperglycemia and diabetes</E>.
2. <C>The same tests</C> may be used to <A>screen for and diagnose diabetes and to detect individuals with prediabetes</A> (Table 2.2 and Table 2.5).
3. <C>"A1CRecommendations2.1	To avoid misdiagnosis or missed diagnosis, the A1C test</C> should be performed using a method that is certied by the NGSP and stan- dardized to the Diabetes Control and Complications Trial (DCCT) assay."
4. <C>Race/Ethnicity/HemoglobinopathiesHemoglobin variants</C> can interfere with the measurement of A1C, although most assays in use in theU.S. are unaffected by the most common variants.
5. <C>African Americans heterozygous for the common hemoglobin vari

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <C>Risk based screening for prediabetes and/or type 2 diabetes</C> should be considered <CO>after the onset of puberty or after 10 years of age, whichever occurs earlier, in children and adolescents with overweight (BMI >85th percen- tile) or obesity (BMI >95th per- centile) and who have one or more risk factor for diabetes</CO>.
2. <A>If results are normal, testing should be repeated at a minimum of 3-year intervals, with consideration of more frequent testing depending on initial results and risk status</A>.
3. <C>Risk-based screening for type 2 diabetes or prediabetes in asymptomatic children and adolescents in a clinical setting</C> should be considered <CO>in youth* who have overweight (>85th percentile) or obesity (>95th percentile) and who have one or more additional risk factors based on the strength of their association with diabetes</CO>.
4. <A>If tests ar

In [ ]:
text = f'''
The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.1The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.2  Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.
While AAFP acknowledges that diabetes, like all chronic conditions, exists on a continuum and that insulin resistance is a risk factor for developing diabetes, the current evidence does not show improvement in long term health outcomes for screening for prediabetes in adults who have obesity or overweight.
The evidence did show a reduction in the incidence of diabetes in individuals at risk for diabetes who received intense lifestyle interventions or metformin, however there was not an associated improvement in health outcomes.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.</C>1The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that <C>nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.</C>2  <A>Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.</A>

While AAFP acknowledges that <C>diabetes, like all chronic conditions, exists on a continuum and that insulin resistance is a risk factor for developing diabetes</C>, the current evidence does not show <E>improvement in long term health outcomes for screening for prediabetes in adults who have obesity or overweight.</E>

The evidence did show a <E>reduction in the incidence of diabetes in individuals at risk for diabetes who received intense lifestyle interventions or metfor

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags as given in the examples below. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.
Example 1: <S>Pregnant persons</S> <C>with gestational diabetes </C> are at <E>increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia</E>.
Example 2: <C>Gestational diabetes</C>  has also been associated with an <E>increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring</E> .
Example 3: EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a <S>moderate net benefit</S>  to <C>screening for gestational diabetes at 24 weeks of gestation or after</C>  to <E>improve maternal and fetal outcomes</E> .
Example 4: RECOMMENDATION The USPSTF recommends <A>screening for gestational diabetes</A>  in <CO>asymptomatic pregnant persons at 24 weeks of gestation or after</CO> .

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <O>Similar to glucose measurements, the continuum of risk is curvilinear, so as</O> <C>A1C rises</C>, <E>the diabetes risk rises disproportionately (49)</E>.
2. <O>(1IQQEE)  1.7 c. We suggest that in the exceptional case where</O> <CO>the degree of renal dysfunction is severe and there is uncertainty about when conception will occur</CO>, <A>physicians and patients be engaged in shared decision-making about whether to continue ACE inhibitors or angiotensin-receptor blockers</A>.
3. <O>Remarks  We acknowledge that with universal testing for diabetes in early pregnancy,</O> <C>there will be a high rate of false- positive results (70) and that women with positive testing may have anxiety and will suffer the burden of additional testing</C>.
4. <O>Nevertheless,</O> <A>we recommended universal testing</A> <C>because we place the highest value on preventing fetal complica

In [ ]:
import pandas as pd

In [ ]:
data_gold = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')

In [ ]:
data_gold

,final
0,<S>Pregnant persons</S> <C>with gestational di...
1,<C>Gestational diabetes</C> has also been ass...
2,EVIDENCE ASSESSMENT The USPSTF concludes with ...
3,RECOMMENDATION The USPSTF recommends <A>screen...
4,Importance<C>Gestational diabetes</C> is diab...
...,...
284,Clinical Preventive Service RecommendationDiab...
285,The AAFP also agrees that there is insufficien...
286,"While AAFP acknowledges that <E>diabetes</E> ,..."
287,The evidence did show a <E>reduction in the in...


In [ ]:
def clean_text(text):
    text = str(text).lower()
    remove = string.punctuation
    remove = remove.replace("<", "")
    remove = remove.replace(">", "")
    remove = remove.replace("/", "")
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % remove, '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ')]
    # text=" ".join(text)
    #text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [ ]:
import re
import string
import nltk
stemmer = nltk.SnowballStemmer("english")
data_gold['clean_final']=data_gold['final'].apply(clean_text)

In [ ]:
data_gold

,final,clean_final
0,<S>Pregnant persons</S> <C>with gestational di...,<s>pregnant persons</s> <c>with gestational di...
1,<C>Gestational diabetes</C> has also been ass...,<c>gestational diabetes</c> has also been ass...
2,EVIDENCE ASSESSMENT The USPSTF concludes with ...,evidence assessment the uspstf concludes with ...
3,RECOMMENDATION The USPSTF recommends <A>screen...,recommendation the uspstf recommends <a>screen...
4,Importance<C>Gestational diabetes</C> is diab...,importance<c>gestational diabetes</c> is diab...
...,...,...
284,Clinical Preventive Service RecommendationDiab...,clinical preventive service recommendationdiab...
285,The AAFP also agrees that there is insufficien...,the aafp also agrees that there is insufficien...
286,"While AAFP acknowledges that <E>diabetes</E> ,...",while aafp acknowledges that <e>diabetes</e> ...
287,The evidence did show a <E>reduction in the in...,the evidence did show a <e>reduction in the in...


In [ ]:
data_gold.to_excel('/content/data_gold.xlsx')

In [ ]:
import pandas as pd
import spacy
import re
import json
from nltk.tokenize import word_tokenize

data_train_df = pd.read_excel('/content/data_gold.xlsx') #,sheet_name='Sheet2'
tagged_words = []
for para in data_train_df['clean_final']:
   para=para.encode("ascii", "ignore")
   para=para.decode()
   connl_text= []
   label = []
   cause = re.findall(r'<c>(.*?)</c>', para)
   condition = re.findall(r'<co>(.*?)</co>', para)
   effect = re.findall(r'<e>(.*?)</e>', para)
   action = re.findall(r'<a>(.*?)</a>', para)
   signal = re.findall(r'<s>(.*?)</s>',para)
  # ARG0 = [re.sub('<(\w+)>(.*?)<\/\w+>','',i) for i in ARG0]
   splitted = re.split('<c>|</c>|<co>|</co>|<e>|</e>|<a>|</a>|<s>|</s>', para)  # splitted considering tags
   if cause or condition or effect or action or signal:  # if any tag in sentence
    for split in splitted:  # search each index
            if split in cause:
                for token in split.split():
                    connl_text.append(token)
                    label.append('C')
            elif split in effect:
                for token in split.split():
                    connl_text.append(token)
                    label.append('E')
            elif split in signal:
                for token in split.split():
                  connl_text.append(token)
                  label.append('S')
            elif split in action:
                for token in split.split():
                  connl_text.append(token)
                  label.append('A')
            elif split in condition:
                for token in split.split():
                  connl_text.append(token)
                  label.append('CO')

            else:  # current word is not an entity
                for token in split.split():
                    connl_text.append(token)
                    label.append('O')

    tagged_words.append({'Word': connl_text, 'Tag': label})

In [ ]:
data_gold = pd.DataFrame(tagged_words)

In [ ]:
data_gold

,Word,Tag
0,"[pregnant, persons, with, gestational, diabete...","[S, S, C, C, C, O, O, E, E, E, E, E, E, E, E, ..."
1,"[gestational, diabetes, has, also, been, assoc...","[C, C, O, O, O, O, O, O, E, E, E, E, E, E, E, ..."
2,"[evidence, assessment, the, uspstf, concludes,...","[O, O, O, O, O, O, O, O, O, O, O, O, S, S, S, ..."
3,"[recommendation, the, uspstf, recommends, scre...","[O, O, O, O, A, A, A, A, O, CO, CO, CO, CO, CO..."
4,"[importance, gestational, diabetes, is, diabet...","[O, C, C, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
284,"[clinical, preventive, service, recommendation...","[O, O, O, O, O, O, O, O, O, A, A, A, A, A, CO,..."
285,"[the, aafp, also, agrees, that, there, is, ins...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
286,"[while, aafp, acknowledges, that, diabetes, li...","[O, O, O, O, E, O, O, O, O, O, O, O, O, O, O, ..."
287,"[the, evidence, did, show, a, reduction, in, t...","[O, O, O, O, O, E, E, E, E, E, E, O, O, O, O, ..."


In [ ]:
data_gold_final = data_gold.iloc[4:,:]

In [ ]:
data_gold_final

,Word,Tag
4,"[importance, gestational, diabetes, is, diabet...","[O, C, C, O, O, O, O, O, O, O, O, O, O, O, O, ..."
5,"[summary, of, uspstf, rationaleassessmentratio...","[O, O, O, O, CO, CO, CO, CO, CO, CO, CO, CO, C..."
6,"[there, is, inadequate, evidence, that, common...","[O, O, S, S, O, C, C, C, C, O, E, E, E, E, E, ..."
7,"[benefits, of, early, detection, and, interven...","[O, O, O, O, O, O, O, O, O, S, S, S, O, C, C, ..."
8,"[there, is, adequate, evidence, that, treatmen...","[O, O, S, S, O, C, C, C, C, C, O, O, O, E, E, ..."
...,...,...
284,"[clinical, preventive, service, recommendation...","[O, O, O, O, O, O, O, O, O, A, A, A, A, A, CO,..."
285,"[the, aafp, also, agrees, that, there, is, ins...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
286,"[while, aafp, acknowledges, that, diabetes, li...","[O, O, O, O, E, O, O, O, O, O, O, O, O, O, O, ..."
287,"[the, evidence, did, show, a, reduction, in, t...","[O, O, O, O, O, E, E, E, E, E, E, O, O, O, O, ..."


In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
#from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import itertools
import json
import glob

In [ ]:
# def clean_text(text):
#     text = str(text).lower()
#     text = re.sub('\[.*?\]', '', text)
#     text = re.sub('https?://\S+|www\.\S+', '', text)
#     text = re.sub('<.*?>+', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
#     text = [word for word in text.split(' ')]
#     text=" ".join(text)
#     text = [stemmer.stem(word) for word in text.split(' ')]
#     text=" ".join(text)
#     return text

In [ ]:
#data_gold_final['Word']=data_gold_final['Word'].apply(clean_text)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_gold_final[["Word", "Tag"]].rename(columns={"Word": "X", "Tag": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

In [ ]:
X_text_new = pd.DataFrame(X_raw,columns=['sentences'])
y_text_new = pd.DataFrame(y_raw,columns=['label'])

In [ ]:
df_gold_final = pd.concat([X_text_new,y_text_new],axis=1)

In [ ]:
df_gold_final['token_count'] = df_gold_final.label.str.split().str.len()

In [ ]:
df_gold_final

,sentences,label,token_count
0,"[""importance"", ""gestational"", ""diabetes"", ""is""...","[""O"", ""C"", ""C"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",86
1,"[""summary"", ""of"", ""uspstf"", ""rationaleassessme...","[""O"", ""O"", ""O"", ""O"", ""CO"", ""CO"", ""CO"", ""CO"", ""...",32
2,"[""there"", ""is"", ""inadequate"", ""evidence"", ""tha...","[""O"", ""O"", ""S"", ""S"", ""O"", ""C"", ""C"", ""C"", ""C"", ...",19
3,"[""benefits"", ""of"", ""early"", ""detection"", ""and""...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",20
4,"[""there"", ""is"", ""adequate"", ""evidence"", ""that""...","[""O"", ""O"", ""S"", ""S"", ""O"", ""C"", ""C"", ""C"", ""C"", ...",41
...,...,...,...
280,"[""clinical"", ""preventive"", ""service"", ""recomme...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",24
281,"[""the"", ""aafp"", ""also"", ""agrees"", ""that"", ""the...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",68
282,"[""while"", ""aafp"", ""acknowledges"", ""that"", ""dia...","[""O"", ""O"", ""O"", ""O"", ""E"", ""O"", ""O"", ""O"", ""O"", ...",48
283,"[""the"", ""evidence"", ""did"", ""show"", ""a"", ""reduc...","[""O"", ""O"", ""O"", ""O"", ""O"", ""E"", ""E"", ""E"", ""E"", ...",34


In [ ]:
toke_sentences = []
toke_label = []
for i,j in zip(df_gold_final['sentences'],df_gold_final['label']):
  for tok_sent,tok_lab in zip(i.split(','),j.split(',')):
    toke_sentences.append(tok_sent)
    toke_label.append(tok_lab)

In [ ]:
toke_sentences

['["importance"',
 ' "gestational"',
 ' "diabetes"',
 ' "is"',
 ' "diabetes"',
 ' "that"',
 ' "develops"',
 ' "during"',
 ' "prevalence"',
 ' "of"',
 ' "gestational"',
 ' "diabetes"',
 ' "in"',
 ' "the"',
 ' "us"',
 ' "has"',
 ' "been"',
 ' "estimated"',
 ' "at"',
 ' "to"',
 ' "based"',
 ' "on"',
 ' "traditional"',
 ' "diagnostic"',
 ' "criteria"',
 ' "although"',
 ' "it"',
 ' "may"',
 ' "be"',
 ' "higher"',
 ' "if"',
 ' "more"',
 ' "inclusive"',
 ' "criteria"',
 ' "are"',
 ' "pregnant"',
 ' "persons"',
 ' "with"',
 ' "gestational"',
 ' "diabetes"',
 ' "increased"',
 ' "risk"',
 ' "for"',
 ' "maternal"',
 ' "and"',
 ' "fetal"',
 ' "complications"',
 ' "including"',
 ' "preeclampsia"',
 ' "fetal"',
 ' "macrosomia"',
 ' "which"',
 ' "can"',
 ' "cause"',
 ' "shoulder"',
 ' "dystocia"',
 ' "and"',
 ' "birth"',
 ' "injury"',
 ' "and"',
 ' "neonatal"',
 ' "hypoglycemia"',
 ' "gestational"',
 ' "diabetes"',
 ' "has"',
 ' "also"',
 ' "been"',
 ' "associated"',
 ' "with"',
 ' "an"',
 ' "increas

In [ ]:
sents_df = pd.DataFrame(toke_sentences)
label_df = pd.DataFrame(toke_label)

In [ ]:
sents_df

,0
0,"[""importance"""
1,"""gestational"""
2,"""diabetes"""
3,"""is"""
4,"""diabetes"""
...,...
9190,"""in"""
9191,"""the"""
9192,"""absence"""
9193,"""of"""


In [ ]:
label_df

,0
0,"[""O"""
1,"""C"""
2,"""C"""
3,"""O"""
4,"""O"""
...,...
9190,"""O"""
9191,"""O"""
9192,"""O"""
9193,"""O"""


In [ ]:
df_final = pd.concat([sents_df,label_df],axis=1)

In [ ]:
df_final

,0,0
0,"[""importance""","[""O"""
1,"""gestational""","""C"""
2,"""diabetes""","""C"""
3,"""is""","""O"""
4,"""diabetes""","""O"""
...,...,...
9190,"""in""","""O"""
9191,"""the""","""O"""
9192,"""absence""","""O"""
9193,"""of""","""O"""


In [ ]:
df_final.to_excel('/content/GPT4_gold_tokens.xlsx')

In [ ]:
df_gold_final.to_excel('/content/gold.xlsx')

In [ ]:
import pandas as pd
data_GPT4_preds = pd.read_excel('/content/GPT-4-predictions-GD-data.xlsx',sheet_name='four-shot-removed')

In [ ]:
data_GPT4_preds

,final
0,Importance Gestational diabetes is diabetes th...
1,<C>Summary of USPSTF RationaleAssessmentRation...
2,<C>There is inadequate evidence that commonly ...
3,<C>Benefits of early detection and interventio...
4,<C>There is adequate evidence that treatment o...
...,...
280,Clinical Preventive Service RecommendationDiab...
281,<C>The AAFP also agrees that there is insuffic...
282,"While AAFP acknowledges that <C>diabetes, like..."
283,The evidence did show a <E>reduction in the in...


In [ ]:
def clean_text(text):
    text = str(text).lower()
    remove = string.punctuation
    remove = remove.replace("<", "")
    remove = remove.replace(">", "")
    remove = remove.replace("/", "")
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % remove, '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ')]
    # text=" ".join(text)
    #text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [ ]:
import re
import string
import nltk
stemmer = nltk.SnowballStemmer("english")
data_GPT4_preds['clean_final']=data_GPT4_preds['final'].apply(clean_text)

In [ ]:
data_GPT4_preds

,final,clean_final
0,Importance Gestational diabetes is diabetes th...,importance gestational diabetes is diabetes th...
1,<C>Summary of USPSTF RationaleAssessmentRation...,<c>summary of uspstf rationaleassessmentration...
2,<C>There is inadequate evidence that commonly ...,<c>there is inadequate evidence that commonly ...
3,<C>Benefits of early detection and interventio...,<c>benefits of early detection and interventio...
4,<C>There is adequate evidence that treatment o...,<c>there is adequate evidence that treatment o...
...,...,...
280,Clinical Preventive Service RecommendationDiab...,clinical preventive service recommendationdiab...
281,<C>The AAFP also agrees that there is insuffic...,<c>the aafp also agrees that there is insuffic...
282,"While AAFP acknowledges that <C>diabetes, like...",while aafp acknowledges that <c>diabetes like ...
283,The evidence did show a <E>reduction in the in...,the evidence did show a <e>reduction in the in...


In [ ]:
data_GPT4_preds.to_excel('/content/final_GPT4preds.xlsx')

In [ ]:
import pandas as pd
import spacy
import re
import json
from nltk.tokenize import word_tokenize

data_train_df = pd.read_excel('/content/final_GPT4preds.xlsx')
tagged_words = []
for para in data_train_df['clean_final']:
   para=para.encode("ascii", "ignore")
   para=para.decode()
   connl_text= []
   label = []
   cause = re.findall(r'<c>(.*?)</c>', para)
   condition = re.findall(r'<co>(.*?)</co>', para)
   effect = re.findall(r'<e>(.*?)</e>', para)
   action = re.findall(r'<a>(.*?)</a>', para)
   signal = re.findall(r'<s>(.*?)</s>',para)
  # ARG0 = [re.sub('<(\w+)>(.*?)<\/\w+>','',i) for i in ARG0]
   splitted = re.split('<c>|</c>|<co>|</co>|<e>|</e>|<a>|</a>|<s>|</s>', para)  # splitted considering tags
   if cause or condition or effect or action or signal:  # if any tag in sentence
    for split in splitted:  # search each index
            if split in cause:
                for token in split.split():
                    connl_text.append(token)
                    label.append('C')
            elif split in effect:
                for token in split.split():
                    connl_text.append(token)
                    label.append('E')
            elif split in signal:
                for token in split.split():
                  connl_text.append(token)
                  label.append('S')
            elif split in action:
                for token in split.split():
                  connl_text.append(token)
                  label.append('A')
            elif split in condition:
                for token in split.split():
                  connl_text.append(token)
                  label.append('CO')

            else:  # current word is not an entity
                for token in split.split():
                    connl_text.append(token)
                    label.append('O')

    tagged_words.append({'Word': connl_text, 'Tag': label})

In [ ]:
data_preds = pd.DataFrame(tagged_words)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_preds[["Word", "Tag"]].rename(columns={"Word": "X", "Tag": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

In [ ]:
X_text_new = pd.DataFrame(X_raw,columns=['sentences'])
y_text_new = pd.DataFrame(y_raw,columns=['label'])

In [ ]:
df_preds_final = pd.concat([X_text_new,y_text_new],axis=1)

In [ ]:
df_preds_final['token_count'] = df_preds_final.label.str.split().str.len()

In [ ]:
df_preds_final

,sentences,label,token_count
0,"[""importance"", ""gestational"", ""diabetes"", ""is""...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",84
1,"[""summary"", ""of"", ""uspstf"", ""rationaleassessme...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ...",29
2,"[""there"", ""is"", ""inadequate"", ""evidence"", ""tha...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ...",19
3,"[""benefits"", ""of"", ""early"", ""detection"", ""and""...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ...",20
4,"[""there"", ""is"", ""adequate"", ""evidence"", ""that""...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ...",41
...,...,...,...
280,"[""clinical"", ""preventive"", ""service"", ""recomme...","[""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ""O"", ...",24
281,"[""the"", ""aafp"", ""also"", ""agrees"", ""that"", ""the...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ...",70
282,"[""while"", ""aafp"", ""acknowledges"", ""that"", ""dia...","[""O"", ""O"", ""O"", ""O"", ""C"", ""C"", ""C"", ""C"", ""C"", ...",47
283,"[""the"", ""evidence"", ""did"", ""show"", ""a"", ""reduc...","[""O"", ""O"", ""O"", ""O"", ""O"", ""E"", ""E"", ""E"", ""E"", ...",34


In [ ]:
toke_sentences_preds = []
toke_label_preds = []
for i,j in zip(df_preds_final['sentences'],df_preds_final['label']):
  for tok_sent,tok_lab in zip(i.split(','),j.split(',')):
    toke_sentences_preds.append(tok_sent)
    toke_label_preds.append(tok_lab)

In [ ]:
sents_df_preds = pd.DataFrame(toke_sentences_preds)
label_df_preds = pd.DataFrame(toke_label_preds)

In [ ]:
df_preds = pd.concat([sents_df_preds,label_df_preds],axis=1)

In [ ]:
df_preds

,0,0
0,"[""importance""","[""O"""
1,"""gestational""","""O"""
2,"""diabetes""","""O"""
3,"""is""","""O"""
4,"""diabetes""","""O"""
...,...,...
9128,"""in""","""O"""
9129,"""the""","""O"""
9130,"""absence""","""O"""
9131,"""of""","""O"""


In [ ]:
df_preds.to_excel('/content/GPT4_preds.xlsx')

In [ ]:
df_preds_final.to_excel('/content/preds.xlsx')

In [ ]:
text = []
gold = []
for i,j in zip(data_gold_final['Word'],data_gold_final['Tag']):
  for p,r in zip(i,j):
    text.append(p)
    gold.append(r)


In [ ]:
preds = []
for k in data_preds['Tag']:
  for z in k:
    preds.append(z)

In [ ]:
import numpy as np
print(np.shape(text))
print(np.shape(gold))
print(np.shape(preds))

(10192,)
(10192,)
(10102,)


In [ ]:
# preds = []

# for i in data_preds['Tag']:
#   for p in zip(i):
#     preds.append(p)
#    # gold.append(r)

In [ ]:
text_df = pd.DataFrame(text,columns=['text'])
gold_df = pd.DataFrame(gold,columns=['gold'])
preds_df = pd.DataFrame(preds,columns =['preds'])

In [ ]:
df_goldcombined = pd.concat([text_df,preds_df,gold_df],axis=1)

In [ ]:
df_goldcombined = df_goldcombined.dropna()

In [ ]:
df_goldcombined

,text,preds,gold
0,IImportance,O,O
1,Gestational,O,C
2,diabetes,O,C
3,is,O,O
4,diabetes,O,O
...,...,...,...
10097,",",O,O
10098,the,O,O
10099,current,O,S
10100,evidence,O,S


In [ ]:
#four-shot
from sklearn.metrics import classification_report
print(classification_report(y_true=df_goldcombined['gold'], y_pred=df_goldcombined['preds'], zero_division = 1)) #id2tag.keys()

              precision    recall  f1-score   support

           A       0.32      0.44      0.37      2246
           C       0.14      0.52      0.22      1128
          CO       0.20      0.03      0.05      1666
           E       0.21      0.10      0.13      1945
           O       0.31      0.18      0.23      3041
           S       0.00      0.00      0.00        76

    accuracy                           0.23     10102
   macro avg       0.20      0.21      0.17     10102
weighted avg       0.26      0.23      0.21     10102



In [ ]:
import pandas as pd
data_gold = pd.read_excel('/content/data_GD_final.xlsx',sheet_name = 'Sheet2')

In [ ]:
data_gold

,final
0,<S>Pregnant persons</S> <C>with gestational di...
1,<C>Gestational diabetes</C> has also been ass...
2,EVIDENCE ASSESSMENT The USPSTF concludes with ...
3,RECOMMENDATION The USPSTF recommends <A>screen...
4,Importance<C>Gestational diabetes</C> is diab...
...,...
284,Clinical Preventive Service RecommendationDiab...
285,The AAFP also agrees that there is insufficien...
286,"While AAFP acknowledges that <E>diabetes</E> ,..."
287,The evidence did show a <E>reduction in the in...


In [ ]:
import pandas as pd

In [ ]:
data_gold = pd.read_excel('/content/LLAMA-preds_transfer.xlsx',sheet_name = 'gold-original')

In [ ]:
import re
#ce-rela = []
fullText = []
causal_relation = []
variables = []
effect=[]
cause=[]
signal=[]
action=[]
condition=[]
clean = re.compile('<.*?>')
for para in data_gold['final']:
  para=para.encode("ascii", "ignore")
  para=para.decode()
  #print(para)
  #tags = re.compile('<.*?>(.*?)</.*?>').findall(para)
  sent = re.sub(clean, '', para)
  tags = re.compile('(<C>|<E>|<A>|<S>|<CO>)(.*?)(</C>|</E>|</A>|</S>|</CO>)').findall(para)
 # print(tags)
  for i in tags:
    if i[0] == '<C>':
      variables.append([sent,i[1],'cause'])
    elif i[0] == '<E>':
      variables.append([sent, i[1], 'effect'])
    elif i[0] == '<A>':
      variables.append([sent,i[1], 'action'])
    elif i[0] == '<S>':
      variables.append([sent,i[1],'signal'])
    elif i[0] == '<CO>':
      variables.append([sent,i[1],'condition'])

In [ ]:
import re
#ce-rela = []
fullText = []
causal_relation = []
variables = []
effect=[]
cause=[]
signal=[]
action=[]
condition=[]
clean = re.compile('<.*?>')
for para in data_gold['final']:
  para=para.encode("ascii", "ignore")
  para=para.decode()
  #print(para)
  #tags = re.compile('<.*?>(.*?)</.*?>').findall(para)
  sent = re.sub(clean, '', para)
  tags = re.compile('(<C>|<E>)(.*?)(</C>|</E>)').findall(para)
 # print(tags)
  for i in tags:
    if i[0] == '<C>':
      variables.append([sent,i[1],'cause'])
    elif i[0] == '<E>':
      variables.append([sent, i[1], 'outcome'])

In [ ]:
#sent_id, unique = pd.factorize(variables['text'])

In [ ]:
df = pd.DataFrame((variables),columns =['text','variables','role'])

In [ ]:
#df_final = df.iloc[10:,:]
df_final = df

In [ ]:
df_final

,text,variables,role
0,Pregnant persons with gestational diabetes ar...,with gestational diabetes,cause
1,Pregnant persons with gestational diabetes ar...,increased risk for maternal and fetal complica...,outcome
2,Gestational diabetes has also been associated...,Gestational diabetes,cause
3,Gestational diabetes has also been associated...,increased risk of several long-term health out...,outcome
4,EVIDENCE ASSESSMENT The USPSTF concludes with ...,screening for gestational diabetes at 24 weeks...,cause
...,...,...,...
361,"While AAFP acknowledges that diabetes , like a...",diabetes,outcome
362,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,outcome
363,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause
364,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause


In [ ]:
#df_final_nocondition = df_final[df_final.role != 'condition']

In [ ]:
#df_final_nocondition

In [ ]:
sent_id, unique = pd.factorize(df['text'])

In [ ]:
df['id'] = sent_id

In [ ]:
df

,text,variables,role,id
0,Pregnant persons with gestational diabetes ar...,with gestational diabetes,cause,0
1,Pregnant persons with gestational diabetes ar...,increased risk for maternal and fetal complica...,outcome,0
2,Gestational diabetes has also been associated...,Gestational diabetes,cause,1
3,Gestational diabetes has also been associated...,increased risk of several long-term health out...,outcome,1
4,EVIDENCE ASSESSMENT The USPSTF concludes with ...,screening for gestational diabetes at 24 weeks...,cause,2
...,...,...,...,...
361,"While AAFP acknowledges that diabetes , like a...",diabetes,outcome,174
362,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,outcome,175
363,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,175
364,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,176


In [ ]:
df['variables_combined'] = df['variables'] + '-' + df['role']

In [ ]:
df

,text,variables,role,id,variables_combined
0,Pregnant persons with gestational diabetes ar...,with gestational diabetes,cause,0,with gestational diabetes -cause
1,Pregnant persons with gestational diabetes ar...,increased risk for maternal and fetal complica...,outcome,0,increased risk for maternal and fetal complica...
2,Gestational diabetes has also been associated...,Gestational diabetes,cause,1,Gestational diabetes-cause
3,Gestational diabetes has also been associated...,increased risk of several long-term health out...,outcome,1,increased risk of several long-term health out...
4,EVIDENCE ASSESSMENT The USPSTF concludes with ...,screening for gestational diabetes at 24 weeks...,cause,2,screening for gestational diabetes at 24 weeks...
...,...,...,...,...,...
361,"While AAFP acknowledges that diabetes , like a...",diabetes,outcome,174,diabetes-outcome
362,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,outcome,175,reduction in the incidence of diabetes-outcome
363,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,175,received intense lifestyle interventions or me...
364,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,176,Lifestyle interventions to prevent progression...


In [ ]:
df_partial = df.iloc[:,1:5]

In [ ]:
df_partial

,variables,role,id,variables_combined
0,with gestational diabetes,cause,0,with gestational diabetes -cause
1,increased risk for maternal and fetal complica...,outcome,0,increased risk for maternal and fetal complica...
2,Gestational diabetes,cause,1,Gestational diabetes-cause
3,increased risk of several long-term health out...,outcome,1,increased risk of several long-term health out...
4,screening for gestational diabetes at 24 weeks...,cause,2,screening for gestational diabetes at 24 weeks...
...,...,...,...,...
361,diabetes,outcome,174,diabetes-outcome
362,reduction in the incidence of diabetes,outcome,175,reduction in the incidence of diabetes-outcome
363,received intense lifestyle interventions or me...,cause,175,received intense lifestyle interventions or me...
364,Lifestyle interventions to prevent progression...,cause,176,Lifestyle interventions to prevent progression...


In [ ]:
data_gold_sor = df.sort_values(by=['id', 'role', 'variables'])

In [ ]:
data_gold_sor.to_excel('/content/data_gold.xlsx')

In [ ]:
data_GD_gold = df_partial.groupby('id').agg(lambda x: x.tolist())

In [ ]:
data_GD_gold

,variables,role,variables_combined
id,,,
0,"[with gestational diabetes , increased risk fo...","[cause, outcome]","[with gestational diabetes -cause, increased r..."
1,"[Gestational diabetes, increased risk of sever...","[cause, outcome]","[Gestational diabetes-cause, increased risk of..."
2,[screening for gestational diabetes at 24 week...,"[cause, outcome]",[screening for gestational diabetes at 24 week...
3,"[Gestational diabetes, Pregnant persons with g...","[cause, cause, outcome, cause, outcome]","[Gestational diabetes-cause, Pregnant persons ..."
4,"[commonly used screening tests, accurately det...","[cause, outcome]","[commonly used screening tests-cause, accurate..."
...,...,...,...
172,[detection and diagnosis of hyperglycemic diso...,"[cause, cause, outcome, cause, outcome]",[detection and diagnosis of hyperglycemic diso...
173,[ 75-g OGTT at 24 -28 weeks' of gestation in a...,"[cause, outcome]",[ 75-g OGTT at 24 -28 weeks' of gestation in a...
174,"[diabetes, insulin resistance, diabetes, does ...","[outcome, cause, outcome, outcome, outcome]","[diabetes-outcome, insulin resistance-cause, d..."


In [ ]:
data_GD_gold.to_excel('/content/partial.xlsx')

In [ ]:
data_text = df['text']

In [ ]:
type(data_text)

pandas.core.series.Series

In [ ]:
data_text = pd.DataFrame(data_text)

In [ ]:
data_text = data_text.drop_duplicates()

In [ ]:
data_text

,text
0,Pregnant persons with gestational diabetes ar...
2,Gestational diabetes has also been associated...
4,EVIDENCE ASSESSMENT The USPSTF concludes with ...
6,ImportanceGestational diabetes is diabetes th...
11,Summary of USPSTF RationaleAssessmentRationale...
...,...
350,Table 2-Strategy for the detection and diagnos...
355,Glucose testing early in pregnancy to detect ...
357,"While AAFP acknowledges that diabetes , like a..."
362,The evidence did show a reduction in the incid...


In [ ]:
data_text.to_excel('/content/data_text.xlsx')

In [ ]:
data_fi = pd.concat([data_text['text'],data_GD_gold],axis=1)

In [ ]:
data_fi

,text,variables,role,variables_combined
0,Pregnant persons with gestational diabetes ar...,"[with gestational diabetes , increased risk fo...","[cause, outcome]","[with gestational diabetes -cause, increased r..."
2,Gestational diabetes has also been associated...,[screening for gestational diabetes at 24 week...,"[cause, outcome]",[screening for gestational diabetes at 24 week...
4,EVIDENCE ASSESSMENT The USPSTF concludes with ...,"[commonly used screening tests, accurately det...","[cause, outcome]","[commonly used screening tests-cause, accurate..."
6,ImportanceGestational diabetes is diabetes th...,"[screening for gestational diabetes, improves ...","[cause, outcome]","[screening for gestational diabetes-cause, imp..."
11,Summary of USPSTF RationaleAssessmentRationale...,"[insulin resistance increases, higher glucose ...","[cause, outcome]","[insulin resistance increases-cause, higher gl..."
...,...,...,...,...
168,NaN,[for initial detection of possible cases],[cause],[for initial detection of possible cases-cause]
170,NaN,"[assessment of glycemia in early pregnancy, de...","[cause, outcome]",[assessment of glycemia in early pregnancy-cau...
172,NaN,[detection and diagnosis of hyperglycemic diso...,"[cause, cause, outcome, cause, outcome]",[detection and diagnosis of hyperglycemic diso...
174,NaN,"[diabetes, insulin resistance, diabetes, does ...","[outcome, cause, outcome, outcome, outcome]","[diabetes-outcome, insulin resistance-cause, d..."


In [ ]:
data_GD_gold.to_excel('/content/LLM-GD-dat_final.xlsx')

In [ ]:
data_fi.to_excel('/content/LL')

In [ ]:
df_final.to_excel('/content/data_gold.xlsx')

In [ ]:
data_preds = pd.read_excel('/content/GPT-4-predictions-GD-data.xlsx',sheet_name = 'four-shot-removed')

In [ ]:
data_preds

,final
0,Importance Gestational diabetes is diabetes th...
1,<C>Summary of USPSTF RationaleAssessmentRation...
2,<C>There is inadequate evidence that commonly ...
3,<C>Benefits of early detection and interventio...
4,<C>There is adequate evidence that treatment o...
...,...
280,Clinical Preventive Service RecommendationDiab...
281,<C>The AAFP also agrees that there is insuffic...
282,"While AAFP acknowledges that <C>diabetes, like..."
283,The evidence did show a <E>reduction in the in...


In [ ]:
import re
#ce-rela = []
fullText = []
causal_relation = []
variables = []
effect=[]
cause=[]
signal=[]
action=[]
condition=[]
clean = re.compile('<.*?>')
for para in data_preds['final']:
  para=para.encode("ascii", "ignore")
  para=para.decode()
  #print(para)
  #tags = re.compile('<.*?>(.*?)</.*?>').findall(para)
  sent = re.sub(clean, '', para)
  tags = re.compile('(<C>|<E>|<A>|<S>|<CO>)(.*?)(</C>|</E>|</A>|</S>|</CO>)').findall(para)
 # print(tags)
  for i in tags:
    if i[0] == '<C>':
      variables.append([sent,i[1],'cause'])
    elif i[0] == '<E>':
      variables.append([sent, i[1], 'effect'])
    elif i[0] == '<A>':
      variables.append([sent,i[1], 'action'])
    elif i[0] == '<S>':
      variables.append([sent,i[1],'signal'])
    elif i[0] == '<CO>':
      variables.append([sent,i[1],'condition'])

In [ ]:
df_preds = pd.DataFrame((variables),columns =['text','variables','role'])
sent_id, unique = pd.factorize(df_preds['text'])
df_preds['id'] = sent_id
df_preds.to_excel('/content/data_preds.xlsx')

In [ ]:
df_preds

,text,variables,role,id
0,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0
1,Importance Gestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0
2,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause,0
3,Importance Gestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0
4,Summary of USPSTF RationaleAssessmentRationale...,Summary of USPSTF RationaleAssessmentRationale...,cause,1
...,...,...,...,...
380,"While AAFP acknowledges that diabetes, like al...",improvement in long term health outcomes for s...,effect,282
381,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes in indi...,effect,283
382,The evidence did show a reduction in the incid...,improvement in health outcomes.,effect,283
383,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284


In [ ]:
data_preds_sor = df_preds.sort_values(by=['id', 'role', 'variables'])

In [ ]:
data_preds_sor.to_excel('/content/data_preds.xlsx')

In [ ]:
data = df_final.merge(df_preds,on='id',how='outer')

In [ ]:
data

,text_x,variables_x,role_x,id,text_y,variables_y,role_y
0,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause
1,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Importance Gestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect
2,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause
3,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Importance Gestational diabetes is diabetes th...,increased risk of several long-term health out...,effect
4,ImportanceGestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause
...,...,...,...,...,...,...,...
1017,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,283,The evidence did show a reduction in the incid...,improvement in health outcomes.,effect
1018,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause
1019,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,Lifestyle interventions to prevent progression...,"reducing weight, blood pressure and cholesterol",effect
1020,Lifestyle interventions to prevent progression...,"reducing weight, blood pressure and cholesterol",effect,284,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause


In [ ]:
data.to_excel('/content/GPT-4-merged.xlsx')

In [ ]:
##############processing

In [ ]:
data_gold = pd.read_excel('/content/data_gold.xlsx')
data_preds = pd.read_excel('/content/data_preds.xlsx')

In [ ]:
data_gold

,Unnamed: 0,text,variables,role,id,variables_combined
0,10,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Gestational diabetes-cause
1,13,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Gestational diabetes-cause
2,11,ImportanceGestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,Pregnant persons with gestational diabetes-cause
3,12,ImportanceGestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,increased risk for maternal and fetal complica...
4,14,ImportanceGestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,increased risk of several long-term health out...
...,...,...,...,...,...,...
719,724,"While AAFP acknowledges that diabetes , like a...",risk factor,signal,282,risk factor-signal
720,731,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,283,received intense lifestyle interventions or me...
721,730,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,effect,283,reduction in the incidence of diabetes-effect
722,732,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,Lifestyle interventions to prevent progression...


In [ ]:
data_preds

,Unnamed: 0,text,variables,role,id
0,2,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause,0
1,0,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0
2,1,Importance Gestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0
3,3,Importance Gestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0
4,4,Summary of USPSTF RationaleAssessmentRationale...,Summary of USPSTF RationaleAssessmentRationale...,cause,1
...,...,...,...,...,...
380,380,"While AAFP acknowledges that diabetes, like al...",improvement in long term health outcomes for s...,effect,282
381,382,The evidence did show a reduction in the incid...,improvement in health outcomes.,effect,283
382,381,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes in indi...,effect,283
383,383,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284


In [ ]:
data_gold['Rank'] = data_gold.groupby(by=['id','role','variables'])['text'].transform(lambda x: x.rank(method='first'))

In [ ]:
data_preds['Rank'] = data_gold.groupby(by=['id','role','variables'])['text'].transform(lambda x: x.rank(method='first'))

In [ ]:
data_gold

,Unnamed: 0,text,variables,role,id,variables_combined,Rank
0,10,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Gestational diabetes-cause,1.0
1,13,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Gestational diabetes-cause,2.0
2,11,ImportanceGestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,Pregnant persons with gestational diabetes-cause,1.0
3,12,ImportanceGestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,increased risk for maternal and fetal complica...,1.0
4,14,ImportanceGestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,increased risk of several long-term health out...,1.0
...,...,...,...,...,...,...,...
719,724,"While AAFP acknowledges that diabetes , like a...",risk factor,signal,282,risk factor-signal,1.0
720,731,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,283,received intense lifestyle interventions or me...,1.0
721,730,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,effect,283,reduction in the incidence of diabetes-effect,1.0
722,732,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,Lifestyle interventions to prevent progression...,1.0


In [ ]:
data_gold['Conca'] = pd.concat([data_gold['role'],data_gold['variables']],axis=1).apply(lambda x:''.join(x), axis=1)

In [ ]:
data_preds['Conca'] = pd.concat([data_preds['role'],data_preds['variables']],axis=1).apply(lambda x:''.join(x), axis=1)

In [ ]:
data_gold['conca_num'] = data_gold['id'].astype(str) + data_gold['Conca'] + data_gold['Rank'].astype(str)

In [ ]:
data_preds['conca_num'] = data_preds['id'].astype(str) + data_preds['Conca'] + data_preds['Rank'].astype(str)

In [ ]:
data_gold_final = data_gold[['text','variables','role','id','conca_num']]

In [ ]:
data_gold_final.columns = ['text_g','variables_g','role_g','id_g','conca_num_g']

In [ ]:
data_gold_final

,text_g,variables_g,role_g,id_g,conca_num_g
0,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0
1,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes2.0
2,ImportanceGestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,0causePregnant persons with gestational diabet...
3,ImportanceGestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,0effectincreased risk for maternal and fetal c...
4,ImportanceGestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,0effectincreased risk of several long-term hea...
...,...,...,...,...,...
719,"While AAFP acknowledges that diabetes , like a...",risk factor,signal,282,282signalrisk factor1.0
720,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,283,283causereceived intense lifestyle interventio...
721,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,effect,283,283effectreduction in the incidence of diabete...
722,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,284causeLifestyle interventions to prevent pro...


In [ ]:
data_preds_final = data_preds[['text','variables','role','id','conca_num']]

In [ ]:
data_preds_final.columns = ['text_p','variables_p','role_p','id_p','conca_num_p']

In [ ]:
data_preds_final

,text_p,variables_p,role_p,id_p,conca_num_p
0,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0
1,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,0causePregnant persons with gestational diabet...
2,Importance Gestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,0effectincreased risk for maternal and fetal c...
3,Importance Gestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,0effectincreased risk of several long-term hea...
4,Summary of USPSTF RationaleAssessmentRationale...,Summary of USPSTF RationaleAssessmentRationale...,cause,1,1causeSummary of USPSTF RationaleAssessmentRat...
...,...,...,...,...,...
380,"While AAFP acknowledges that diabetes, like al...",improvement in long term health outcomes for s...,effect,282,282effectimprovement in long term health outco...
381,The evidence did show a reduction in the incid...,improvement in health outcomes.,effect,283,283effectimprovement in health outcomes.1.0
382,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes in indi...,effect,283,283effectreduction in the incidence of diabete...
383,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,284causeLifestyle interventions to prevent pro...


In [ ]:
data_gold.to_excel('/content/data_gold.xlsx')
data_preds.to_excel('/content/data_preds.xlsx')

In [ ]:
data_preds

,Unnamed: 0,text,variables,role,id,Rank,Conca,conca_num
0,2,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause,0,1.0,causeGestational diabetes,0causeGestational diabetes1.0
1,0,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,2.0,causePregnant persons with gestational diabetes,0causePregnant persons with gestational diabet...
2,1,Importance Gestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,1.0,effectincreased risk for maternal and fetal co...,0effectincreased risk for maternal and fetal c...
3,3,Importance Gestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,1.0,effectincreased risk of several long-term heal...,0effectincreased risk of several long-term hea...
4,4,Summary of USPSTF RationaleAssessmentRationale...,Summary of USPSTF RationaleAssessmentRationale...,cause,1,1.0,causeSummary of USPSTF RationaleAssessmentRati...,1causeSummary of USPSTF RationaleAssessmentRat...
...,...,...,...,...,...,...,...,...
380,380,"While AAFP acknowledges that diabetes, like al...",improvement in long term health outcomes for s...,effect,282,1.0,effectimprovement in long term health outcomes...,282effectimprovement in long term health outco...
381,382,The evidence did show a reduction in the incid...,improvement in health outcomes.,effect,283,1.0,effectimprovement in health outcomes.,283effectimprovement in health outcomes.1.0
382,381,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes in indi...,effect,283,1.0,effectreduction in the incidence of diabetes i...,283effectreduction in the incidence of diabete...
383,383,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,1.0,causeLifestyle interventions to prevent progre...,284causeLifestyle interventions to prevent pro...


In [ ]:
data_gold

,Unnamed: 0,text,variables,role,id,variables_combined,Rank,Conca,conca_num,join
0,10,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Gestational diabetes-cause,1.0,causeGestational diabetes,0causeGestational diabetes1.0,1
1,13,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,Gestational diabetes-cause,2.0,causeGestational diabetes,0causeGestational diabetes2.0,1
2,11,ImportanceGestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,Pregnant persons with gestational diabetes-cause,1.0,causePregnant persons with gestational diabetes,0causePregnant persons with gestational diabet...,1
3,12,ImportanceGestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,increased risk for maternal and fetal complica...,1.0,effectincreased risk for maternal and fetal co...,0effectincreased risk for maternal and fetal c...,1
4,14,ImportanceGestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,increased risk of several long-term health out...,1.0,effectincreased risk of several long-term heal...,0effectincreased risk of several long-term hea...,1
...,...,...,...,...,...,...,...,...,...,...
719,724,"While AAFP acknowledges that diabetes , like a...",risk factor,signal,282,risk factor-signal,1.0,signalrisk factor,282signalrisk factor1.0,1
720,731,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,283,received intense lifestyle interventions or me...,1.0,causereceived intense lifestyle interventions ...,283causereceived intense lifestyle interventio...,1
721,730,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,effect,283,reduction in the incidence of diabetes-effect,1.0,effectreduction in the incidence of diabetes,283effectreduction in the incidence of diabete...,1
722,732,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,Lifestyle interventions to prevent progression...,1.0,causeLifestyle interventions to prevent progre...,284causeLifestyle interventions to prevent pro...,1


In [ ]:
data_merged = pd.read_excel('/content/merged_data.xlsx')

In [ ]:
data_merged

,Unnamed: 0,ID,text,variables,role,Nikhil_pred,Nikhil_role
0,0,0,Pregnant persons with gestational diabetes are...,Pregnant persons,Signal,Pregnant persons,signal
1,1,0,Pregnant persons with gestational diabetes are...,with gestational diabetes,cause,with gestational diabetes,cause
2,2,0,Pregnant persons with gestational diabetes are...,increased risk for maternal and fetal complica...,effect,are at increased risk for maternal and fetal c...,effect
3,3,1,Gestational diabetes has also been associated ...,Gestational diabetes,cause,Gestational diabetes,cause
4,4,1,Gestational diabetes has also been associated ...,increased risk of several long-term health out...,effect,increased risk of several long-term health out...,effect
...,...,...,...,...,...,...,...
509,2493,1019,"While AAFP acknowledges that diabetes, like al...",insulin resistance,cause,insulin resistance,condition
510,2503,1021,The evidence did show a reduction in the incid...,did show a reduction in the incidence of diabe...,effect,reduction in the incidence of diabetes\t,effect
511,2504,1021,The evidence did show a reduction in the incid...,intense lifestyle interventions or metformin,cause,received intense lifestyle interventions or m...,cause
512,2507,1023,Lifestyle interventions to prevent progression...,Lifestyle interventions,cause,Lifestyle interventions to prevent progressio...,cause


In [ ]:
m = data_gold_final.merge(data_preds_final,left_on='id_g',right_on='id_p')

In [ ]:
m

,text_g,variables_g,role_g,id_g,conca_num_g,join,text_p,variables_p,role_p,id_p,conca_num_p
0,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0,1,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0
1,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0,1,Importance Gestational diabetes is diabetes th...,Pregnant persons with gestational diabetes,cause,0,0causePregnant persons with gestational diabet...
2,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0,1,Importance Gestational diabetes is diabetes th...,increased risk for maternal and fetal complica...,effect,0,0effectincreased risk for maternal and fetal c...
3,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0,1,Importance Gestational diabetes is diabetes th...,increased risk of several long-term health out...,effect,0,0effectincreased risk of several long-term hea...
4,ImportanceGestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes2.0,1,Importance Gestational diabetes is diabetes th...,Gestational diabetes,cause,0,0causeGestational diabetes1.0
...,...,...,...,...,...,...,...,...,...,...,...
1017,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,effect,283,283effectreduction in the incidence of diabete...,1,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes in indi...,effect,283,283effectreduction in the incidence of diabete...
1018,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,284causeLifestyle interventions to prevent pro...,1,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,284causeLifestyle interventions to prevent pro...
1019,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,284causeLifestyle interventions to prevent pro...,1,Lifestyle interventions to prevent progression...,"reducing weight, blood pressure and cholesterol",effect,284,"284effectreducing weight, blood pressure and c..."
1020,Lifestyle interventions to prevent progression...,"reducing weight, blood pressure and cholesterol",effect,284,"284effectreducing weight, blood pressure and c...",1,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,284,284causeLifestyle interventions to prevent pro...


In [ ]:
mask = m.apply(lambda x: str(x["variables_g"]) in x["variables_p"], axis=1)

In [ ]:
mask

0        True
1       False
2       False
3       False
4        True
        ...  
1017     True
1018     True
1019    False
1020    False
1021     True
Length: 1022, dtype: bool

In [ ]:
m = m[mask][["variables_g", "role_g", "variables_p","role_p","text_g","text_p"]]

In [ ]:
m

,variables_g,role_g,variables_p,role_p,text_g,text_p
0,Gestational diabetes,cause,Gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
4,Gestational diabetes,cause,Gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
9,Pregnant persons with gestational diabetes,cause,Pregnant persons with gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
14,increased risk for maternal and fetal complica...,effect,increased risk for maternal and fetal complica...,effect,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
19,increased risk of several long-term health out...,effect,increased risk of several long-term health out...,effect,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
...,...,...,...,...,...,...
1012,risk factor,signal,"diabetes, like all chronic conditions, exists ...",cause,"While AAFP acknowledges that diabetes , like a...","While AAFP acknowledges that diabetes, like al..."
1015,received intense lifestyle interventions or me...,cause,reduction in the incidence of diabetes in indi...,effect,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...
1017,reduction in the incidence of diabetes,effect,reduction in the incidence of diabetes in indi...,effect,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...
1018,Lifestyle interventions to prevent progression...,cause,Lifestyle interventions to prevent progression...,cause,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...


In [ ]:
####Jaccard_distance between the matched gold,preds############

In [ ]:
#data_final = m

In [ ]:
data_final = data_merged

In [ ]:
data_final.columns = ['Unnamed: 0','ID','text','variables_g','role_g','variables_p','role_p']

In [ ]:
!pip install textdistance

In [ ]:
import pandas as pd
data_final = pd.read_excel('/content/Merged_GPT-4results.xlsx',sheet_name = 'final')

In [ ]:
original_distance = []
import textdistance
for i,j in zip(data_final['variables_g'],data_final['variables_p']):
  original_distance.append(textdistance.jaccard.normalized_similarity(i, j))

In [ ]:
data_final['jaccard_similarity'] = original_distance

In [ ]:
data_final.loc[:,'jaccard_similarity'].mean()

0.4172308469843213

In [ ]:
#without signal
data_final.loc[:,'jaccard_similarity'].mean()

0.4375294755239286

In [ ]:
original_distance1 = []
import textdistance
for i,j in zip(data_final['variables_g'],data_final['variables_p']):
  original_distance1.append(1-textdistance.jaccard.normalized_similarity(i, j))

In [ ]:
data_final['jaccard_distance'] = original_distance1

In [ ]:
data_final.loc[:,'jaccard_distance'].mean()

0.5827691530156788

In [ ]:
###########exact match

In [ ]:
data_final['exact_match'] = ''

In [ ]:
match = []
vari_nospace = data_final['variables_g'].str.strip()
Nikhil_nospace = data_final['variables_p'].str.strip()
# print(vari_nospace)
# print(Nikhil_nospace)
for ind, (i,j) in enumerate(zip(vari_nospace,Nikhil_nospace)):
  if i == j:
    match.append(1)
  else:
    match.append(0)

In [ ]:
data_final['exact_match'] = match

In [ ]:
data_final.loc[:,'exact_match'].mean()

0.12136752136752137

In [ ]:
##########exact_match count#################
data_final.loc[:,'exact_match'].sum()

71

In [ ]:
###################Levenshtein distance

In [ ]:
#new package
!pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.9/172.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.3 MB/s eta 0:00:00


In [ ]:
#######Levenshtein_similarity##############
original_distance = []
import Levenshtein
for i,j in zip(data_final['variables_g'],data_final['variables_p']):
  original_distance.append(Levenshtein.distance(i, j))

In [ ]:
data_final['Levenshtein_distance'] = original_distance

In [ ]:
data_final

,variables_g,role_g,variables_p,role_p,text_g,text_p,jaccard_similarity,jaccard_distance,exact_match,Levenshtein_distance
0,Gestational diabetes,cause,Gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...,1.000000,0.000000,1,0
4,Gestational diabetes,cause,Gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...,1.000000,0.000000,1,0
9,Pregnant persons with gestational diabetes,cause,Pregnant persons with gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...,1.000000,0.000000,1,0
14,increased risk for maternal and fetal complica...,effect,increased risk for maternal and fetal complica...,effect,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...,0.961111,0.038889,0,7
19,increased risk of several long-term health out...,effect,increased risk of several long-term health out...,effect,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...,0.892308,0.107692,0,14
...,...,...,...,...,...,...,...,...,...,...
1012,risk factor,signal,"diabetes, like all chronic conditions, exists ...",cause,"While AAFP acknowledges that diabetes , like a...","While AAFP acknowledges that diabetes, like al...",0.085271,0.914729,0,118
1015,received intense lifestyle interventions or me...,cause,reduction in the incidence of diabetes in indi...,effect,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...,0.401515,0.598485,0,79
1017,reduction in the incidence of diabetes,effect,reduction in the incidence of diabetes in indi...,effect,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...,0.287879,0.712121,0,94
1018,Lifestyle interventions to prevent progression...,cause,Lifestyle interventions to prevent progression...,cause,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,1.000000,0.000000,1,0


In [ ]:
#####Levenshtein_distance_average######################
data_final.loc[:,'Levenshtein_distance'].mean()

158.52307692307693

In [ ]:
##################Cosine similarity

In [ ]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings = embed(list(data_final['variables_g']))

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
  #print("Message: {}".format(messages[i]))
  print("Embedding size: {}".format(len(message_embedding)))
  message_embedding_snippet = ", ".join(
      (str(x) for x in message_embedding[:3]))
  print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Embedding size: 512
Embedding: [0.023212315514683723, -0.06374122202396393, -0.06084660440683365, ...]

Embedding size: 512
Embedding: [0.023212315514683723, -0.06374122202396393, -0.06084660440683365, ...]

Embedding size: 512
Embedding: [-0.020788950845599174, -0.06280025839805603, -0.06727736443281174, ...]

Embedding size: 512
Embedding: [0.004136980976909399, 0.0516943596303463, -0.06260571628808975, ...]

Embedding size: 512
Embedding: [0.049528174102306366, 0.03359803929924965, -0.07376943528652191, ...]

Embedding size: 512
Embedding: [-0.05419808626174927, 0.01861204020678997, -0.03761434555053711, ...]

Embedding size: 512
Embedding: [-0.0007534283795394003, -0.03444522246718407, -0.040734488517045975, ...]

Embedding size: 512
Embedding: [-0.02619512751698494, -0.00995410606265068, -0.040948010981082916, ...]

Embedding size: 512
Embedding: [-0.031616147607564926, -0.002719090785831213, -0.07324771583080292, ...]

Embedding size: 512
Embedding: [-0.05419808626174927, 0.01861

In [ ]:
# Reduce logging output.
logging.set_verbosity(logging.ERROR)

message_embeddings_preds = embed(list(data_final['variables_p']))

# for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
#   #print("Message: {}".format(messages[i]))
#   print("Embedding size: {}".format(len(message_embedding)))
#   message_embedding_snippet = ", ".join(
#       (str(x) for x in message_embedding[:3]))
#   print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

In [ ]:
type(message_embeddings)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(message_embeddings,message_embeddings_preds)

In [ ]:
cos_sim.shape

(514, 514)

In [ ]:
cos_sim.mean()

0.20549457

In [ ]:
#without signal
cos_sim.mean()

0.22294267

In [ ]:
#df.groupby('group').apply(lambda x: cosine_similarity(np.array([i for i in x['vec']])))

In [ ]:
########Zero-shot

In [ ]:
text = f'''
Pregnant persons with gestational diabetes are at increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia.
Gestational diabetes has also been associated with an increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring.
EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a moderate net benefit to screening for gestational diabetes at 24 weeks of gestation or after to improve maternal and fetal outcomes.
RECOMMENDATION The USPSTF recommends screening for gestational diabetes in asymptomatic pregnant persons at 24 weeks of gestation or after.
IImportanceGestational diabetes is diabetes that develops during pregnancy.1-3 Prevalence of gestational diabetes in the US has been estimated at 5.8% to 9.2%, based on traditional diagnostic criteria, although it may be higher if more inclusive criteria are used.4-8 Pregnant persons with gestational diabetesincreased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia.3,9-11 Gestational diabetes has also been associated with an increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring.12-16Table 1.
Summary of USPSTF RationaleAssessmentRationalePregnant persons at 24 weeks of gestation or afterPregnant persons before 24 weeks of gestationDetectionThere is adequate evidence that commonly used screening tests can accurately detect gestational diabetes.
There is inadequate evidence that commonly used screening tests can accurately detect gestational diabetes earlier than 24 weeks of gestation.
Benefits of early detection and intervention and treatmentThere is inadequate direct evidence that screening for gestational diabetes improves health outcomes.
There is adequate evidence that treatment of screen-detected gestational diabetes is associated with moderate improvements in maternal and fetal outcomes, including primary (first) cesarean delivery, macrosomia, large for gestational age infants, shoulder dystocia, birth injury, and neonatal intensive care unit admissions.
Harms of early detection and intervention and treatmentThere is adequate evidence that the harms of screening for gestational diabetes, such as anxiety, depression, and labeling, are small.
There is adequate evidence that the harms of treatment of gestational diabetes, such as maternal hypoglycemia and low birth weight, are small.
USPSTF Assessment of Magnitude of Net BenefitPregnant Persons at 24 Weeks of Gestation or AfterThe US Preventive Services Task Force (USPSTF) concludes with moderate certainty that there is a moderate net benefit to screening for gestational diabetes at 24 weeks of gestation or after to improve maternal and fetal outcomes.
Definitions of Gestational DiabetesDuring pregnancy, insulin resistance increases, leading to higher glucose intolerance and development of gestational diabetes in some pregnant persons.
Assessment of RiskSeveral factors increase a persons risk for developing gestational diabetes, including obesity, increased maternal age, history of gestational diabetes, family history of diabetes, and belonging to a racial/ethnic group that is at increased risk for developing type 2 diabetes (Hispanic, Native American, South or East Asian, or Pacific Islander descent).4,19,20 Factors associated with a lower risk for developing gestational diabetes include age younger than 25 to 30 years, White race, a body mass index (BMI) of 25 or less (calculated as weight in kilograms divided by height in meters squared), no family history (in a first-degree relative) of diabetes, and no history of glucose intolerance or adverse pregnancy outcomes related to gestational diabetes.21-23 The risk in different racial/ethnic groups may be due in part to social risk factors such as low socioeconomic status or structural racism, although these associations are not examined in the current evidence.
Although a higher BMI increases the risk of gestational diabetes across racial/ethnic groups, the association varies.
In Asian American persons, the prevalence of gestational diabetes at a BMI of 22 to less than 25 is similar to the prevalence in Hispanic persons, non-Hispanic White persons, and Black persons with a higher (>28) BMI.24,25Screening TestsScreening for gestational diabetes in asymptomatic persons involves either a 2-step (screening test followed by a diagnostic test) or 1-step (diagnostic test used for all patients) approach.
In the US, a 2-step approach is commonly used.8,26 A 50-g oral glucose chal- lenge test (OGCT) is performed between 24 and 28 weeks of gestation in a nonfasting state.
If the screening threshold is met or exceeded, patients receive the oral glucose tolerance test (OGTT).
During the OGTT, a fasting glucose level is obtained, followed by administration of a 75-g or 100-g glucose load, then evaluation of glucose levels after 1, 2, and often 3 hours.
Pregnant persons at 24 weeks of gestation or after\nScreen for gestational diabetes.Grade: BPregnant persons before 24 weeks of gestationThe evidence is insufficient to assess the balance of benefits and harms for screening for gestational diabetes.
Screen: If the person is pregnant and is at least 24 weeks of gestation, screen for gestational diabetes by using 1 of several methods:A 2-step process that involves a screening test (oral glucose challenge test) followed by a diagnostic test (oral glucose tolerance test).
A diagnosis of gestational diabetes is made when 2 or more glucose values fall at or above the specified glucose thresholds.27In the 1-step approach, a 75-g glucose load is administered after a fasting glucose level is obtained, and plasma glucose levels are evaluated after 1 and 2 hours.
The American College of Obstetricians and Gynecologists recommends testing between 24 and 28 weeks of gestation.1 Pregnant persons whose first prenatal visit happens after 28 weeks of gestation (ie, late entry into prenatal care) should bescreened as soon as possible.
Potential Preventable BurdenBetween 2006 and 2016, there was an absolute increase of 3.6% in the prevalence of gestational diabetes.7 Pregnant persons with gestational diabetes are at increased risk for maternal and fetal complications and may benefit from early identification and treatment.
They are also at increased risk for developing type 2 diabetes after pregnancy.29 Pregnant persons who are diagnosed with gestational diabetes before 24 weeks of gestation may be at even greater risk for maternal and fetal complications.29Potential HarmsPotential harms of screening for gestational diabetes include psychological harms (anxiety, depression), intensive medical interventions (induction of labor, cesarean delivery, or admission to the neonatal intensive care unit [NICU]), and negative hospital experiences related to labeling (reduction in breastfeeding and fewer newborns staying in the mothers room) that may be associated with a diagnosis of gestational diabetes.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>Pregnant persons with gestational diabetes</C> <E>are at increased risk for maternal and fetal complications, including preeclampsia, fetal macrosomia (which can cause shoulder dystocia and birth injury), and neonatal hypoglycemia.</E>
<O>Gestational diabetes has also been associated with an increased risk of several long-term health outcomes in pregnant persons and intermediate outcomes in their offspring.</O>
<O>EVIDENCE ASSESSMENT The USPSTF concludes with moderate certainty that there is a moderate net benefit to screening for gestational diabetes at 24 weeks of gestation or after to improve maternal and fetal outcomes.</O>
<A><O>RECOMMENDATION The USPSTF recommends screening for gestational diabetes in asymptomatic pregnant persons at 24 weeks of gestation or after.</O></A>
<O>IImportanceGestational diabetes is diabetes that develops during pregnancy.1-3 Prevalence of gestational diabetes in the US has been estimated at 5.8% to 9.2%, based on traditiona

In [ ]:
text = f'''
Race/ethnicity was fairly diverse in several studies, although 2 of the largest studies with treatment at 24 weeks of gestation or after enrolled large proportions of White (75%) and Chinese (97%) persons.29,34,37,38Treatment of gestational diabetes at 24 weeks of gestation or after was associated with decreased risk of primary cesarean deliveries (relative risk [RR], 0.70 [95% CI, 0.54-0.91]; absolute risk difference [ARD], 5.3%; 3 trials) and preterm deliveries, although findings for the latter are not statistically significant (RR, 0.75 [95% CI, 0.56-1.01]; ARD, 2.3%; 4 trials).29,34 Treatment of gestational diabetes was not associated with reduced risk of preeclampsia in the included studies (RR, 0.99 [95% CI, 0.46-2.16]; 6 trials).29 Addi- tionally, treatment was not associated with reduced risk of gestational hypertension (2 trials), total cesarean deliveries (8 trials), emergency cesarean deliveries (1 trial), induction of labor (5 trials), or maternal birth trauma (2 trials).29,34For fetal/neonatal outcomes, treatment of gestational diabetes at 24 weeks of gestation or after was associated with reduced risk of shoulder dystocia (RR, 0.42 [95% CI, 0.23-0.77]; ARD, 1.3%;3 trials), macrosomia (RR, 0.53 [95% CI, 0.41-0.68]; ARD, 8.9%; 8 trials), LGA infants (RR, 0.56 [95% CI, 0.47-0.66]; ARD, 8.4%; 7 trials), birth injury (eg, fracture or nerve palsies) (odds ratio, 0.33 [95% CI, 0.11-0.99]; ARD, 0.2%), and NICU admissions (RR, 0.73 [95% CI, 0.53-0.99]; ARD, 2%; 5 trials).29,34 No association was found for several outcomes, including mortality (stillbirth, neona- tal, or perinatal), respiratory distress syndrome, hypoglycemia (any or clinical), and hyperbilirubinemia.
One study found that gestational diabetes diagnosis and labeling may be associated with higher rates of cesarean delivery.29,34Harms associated with the treatment of gestational diabetes were evaluated in 13 trials (11 RCTs and 2 nonrandomized clinical trials; n = 4235).29,34 Treatment at 24 weeks of gestation or after was not associated with an increased risk for severe maternal hypoglycemia, low birth weight, or small for gestational age infants.
Treatment of gestational diabetes was associated with a reduced risk of macrosomia (>4000 g) (RR, 0.53 [95% CI, 0.41 to 0.68]) but no difference in the risk of total number of cesarean deliveries (RR, 0.95 [95% CI, 0.83 to 1.08]).29,34 This suggests that a small proportion of pregnant persons may undergo cesarean delivery because of gestational diabetes diagnosis alone.
Pregnant persons who are unable to produce adequate insulin to overcome the increase in insulin resistance seen in pregnancy develop glucose intolerance and gestational diabetes.
The USPSTF found that several screening tests (OGCT, OGTT, fasting plasma glucose) and strategies (1- vs 2-step approach) can accurately detect gestational diabetes and does not recommend any single specific test.
Type 1 diabetes (due to autoimmune b-cell destruction, usually leading to absolute insulin deficiency)2.
Gestational diabetes mellitus (diabetes diagnosed in the second or third trimester of pregnancy that was not clearly overt diabetes prior to gestation)4.
Occasionally, patients with type 2 diabetes may present with DKA (6), particularly ethnic minorities (7).
In both type 1 and type 2 diabetes, various genetic and environmental factors can result in the progressive loss of b-cell mass and/or function that manifests clinically as hyperglycemia.
Once hyperglycemia occurs, patients with all forms of diabetes are at risk for developing the same chronic complications, although rates of progression may differ.
It is now clear from studies of first-degree relatives of patients with type 1 diabetes that the persistent presence of two or more islet autoantibodies is an almost certain predictor of clinical hyperglycemia and diabetes.
Type 2 diabetes is associated with insulin secretory defects related to infiammation and metabolic stress among other contributors, including genetic factors.
The same tests may be used to screen for and diagnose diabetes and to detect individuals with prediabetes (Table 2.2 and Table 2.5).
Fasting and 2-Hour Plasma Glucose The FPG and 2-h PG may be used to diagnose diabetes (Table 2.2).
"A1CRecommendations2.1	To avoid misdiagnosis or missed diagnosis, the A1C test should be performed using a method that is certied by the NGSP and stan- dardized to the Diabetes Control and Complications Trial (DCCT) assay."
Race/Ethnicity/HemoglobinopathiesHemoglobin variants can interfere with the measurement of A1C, although most assays in use in theU.S. are unaffected by the most common variants.
For patients with a hemoglobin variant but normal red blood cell turnover, such as those with the sickle cell trait, an A1C assay without interference from hemoglobin variants should be used.
African Americans heterozygous for the common hemoglobin variant HbS may have, for any given level of mean glycemia, lower A1C by about 0.3% than those without the trait (20).
Another genetic variant, X-linked glucose-6-phosphate dehydrogenase G202A, carried by 11% of African Americans, was associated with a decrease in A1C of about 0.8% in homozygous men and 0.7% in homozygous women compared with those without the variant (21).
Even in the absence of hemoglobin variants, A1C levels may vary with race/ ethnicity independently of glycemia (22-24).
For example, African Americans may have higher A1C levels than non- Hispanic whites with similar fasting and postglucose load glucose levels (25), and A1C levels may be higher for a given mean glucose concentration when measured with continuous glucose monitoring (26).
Be- cause of the potential for preanalytic variability, it is critical that samples for plasma glucose be spun and separated immediately after they are drawn.
TYPE 1 DIABETESRecommendationsScreening for type 1 diabetes risk with a panel of islet autoantibodies is currently recommended in the setting of a research trial or can be offered as an option for rst-degree family members of a proband with type 1 diabetes.
Although patients are not typically obese when they present with type 1 diabetes, obesity is increasingly common in the general population and there is evidence that it may also be a risk factor for type 1 diabetes.
People with type 1 diabetes are also prone to other autoimmune disorders such as Hashimoto thyroiditis, Graves disease, celiac disease, Addison disease, vitiligo, auto- immune hepatitis, myasthenia gravis, and pernicious anemia (see Section 4 "Comprehensive Medical Evaluation and Assessment of Comorbidities," https://doi.org/10.2337/dc20-S004).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>Race/ethnicity was fairly diverse in several studies, although 2 of the largest studies with treatment at 24 weeks of gestation or after enrolled large proportions of White (75%) and Chinese (97%) persons.29,34,37,38</C><A>Treatment of gestational diabetes at 24 weeks of gestation or after was associated with decreased risk of primary cesarean deliveries (relative risk [RR], 0.70 [95% CI, 0.54-0.91]; absolute risk difference [ARD], 5.3%; 3 trials) and preterm deliveries, although findings for the latter are not statistically significant (RR, 0.75 [95% CI, 0.56-1.01]; ARD, 2.3%; 4 trials).29,34 </A><O>Treatment of gestational diabetes was not associated with reduced risk of preeclampsia in the included studies (RR, 0.99 [95% CI, 0.46-2.16]; 6 trials).29 Addi- tionally, treatment was not associated with reduced risk of gestational hypertension (2 trials), total cesarean deliveries (8 trials), emergency cesarean deliveries (1 trial), induction of labor (5 trial

In [ ]:
text = f'''
Individuals with autoantibody-negative type 1 diabetes of African or Asian ancestry may suffer from episodic DKA and exhibit varying degrees of insulin deficiency between episodes.
Multiple studies indicate that measuring islet autoantibodies in individuals genetically at risk for type 1 diabetes (e.g., relatives of those with type 1 diabetes or indi- viduals from the general population with type 1 diabetes-associated genetic factors) identifies individuals who may develop type 1 diabetes (9).
Indeed, the risk of type 1 diabetes increases as the number of relevant autoantibodies detected increases (43-45).
PREDIABETES AND TYPE 2 DIABETES2.13 Risk based screening for prediabetes and/or type 2 diabetes should be considered after the onset of puberty or after 10 years of age, whichever occurs earlier, in children and adolescents with overweight (BMI >85th percen- tile) or obesity (BMI >95th per- centile) and who have one or more risk factor for diabetes.
If results are normal, testing should be repeated at a minimum of 3-year intervals, with consideration of more frequent testing depending on initial results and risk status.
It should be noted that theTable 2.4Risk-based screening for type 2 diabetes or prediabetes in asymptomatic children and adolescents in a clinical setting (163)Testingshould be considered in youth* who have overweight (>85th percentile) or obesity (>95th percentile) and who have one or more additional risk factors based on the strength of their association with diabetes:Maternal history of diabetes or GDM during the childs gestation AFamily history of type 2 diabetes in rst- or second-degree relative ARace/ethnicity (Native American, African American, Latino, Asian American, Pacic Islander) ASigns of insulin resistance or conditions associated with insulin resistance (acanthosis nigricans, hypertension, dyslipidemia, polycystic ovary syndrome, or small-for-gestational- age birth weight) BGDM, gestational diabetes mellitus.
If tests are normal, repeat testing at a minimum of 3-year intervals, or more frequently if BMI is increasing, is recommended.
Those with an A1C range of 6.0- 6.5% (42-48 mmol/mol) had a 5-year risk of developing diabetes between 25% and 50% and a relative risk 20 times higher compared with A1C of 5.0% (31 mmol/mol) (49).
Similar to those with IFG and/or IGT, individuals with A1C of 5.7-6.4% (39-47 mmol/mol) should be informed of their increased risk for diabetes and CVD and counseled about effective strategies to lower their risks (see Section 3 "Prevention or Delay of Type 2 Di- abetes," https://doi.org/10.2337/dc20- S003).
Similar to glucose measurements, the continuum of risk is curvilinear, so as A1C rises, the diabetes risk rises disproportionately (49).
Aggressive interventions and vigilant follow-up should be pursued for those considered at very high risk (e.g., those with A1C .6.0% [42 mmol/mol]).
Excess weight itself causes some degree of insulin resistance.
DKA seldom occurs spontaneously in type 2 diabetes; when seen, it usually arises in association with the stress of another illness such as infection or with the use of certain drugs (e.g., corticosteroids, atypical antipsychotics, and sodium-glucose cotransporter 2 inhibitors) (53,54).
The risk of developing type 2 diabetes increases with age, obesity, and lack of physical activity.
After 5.3 years of follow-up, CVD risk factors were modestly but significantly improved with intensive treatment compared with routine care, but the incidence of first CVD events or mortality was not significantly different between the groups (48).
Community ScreeningIdeally, testing should be carried out within a health care setting because of the need for follow-up and treatment.
CYSTIC FIBROSIS-RELATED DIABETESRecommendationsAnnual screening for cystic brosisrelated diabetes (CFRD) with an oral glucose tolerance test should begin by age 10 years in all patients with cystic brosis not previously diagnosed with CFRD.
BPatients with cystic brosis related diabetes should be treated with insulin to attain individualized glycemic goals.
ABeginning 5 years after the diagnosis of cystic brosisrelated diabetes, annual monitoring for complications of diabetes is recommended.
Diabetes in this population, compared with individuals with type 1 or type 2 diabetes, is associated with worse nutritional status, more severe infiammatory lung disease, and greater mortality.
Genetically determined b-cell function and insulin resistance associated with infection and infiammation may also contribute to the development of CFRD.
EThe oral glucose tolerance test is the preferred test to make a diagnosis of posttransplantation diabetes mellitus.
Most studies have reported that transplant patients with hyperglycemia and PTDM after transplantation have higher rates of rejection, infection, and rehospitalization (89,91,96).
Those with previously poor control or with persistent hyperglycemia should continue insulin with frequent home self-monitoring of blood glucose to determine when insulin dose reductions may be needed and when it may be appropriate to switch to non- insulin agents.
Drug dose adjustments may be required because of decreases in the glomerular filtration rate, a relatively common complication in transplant patients.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>Individuals with autoantibody-negative type 1 diabetes of African or Asian ancestry</C> <E>may suffer from episodic DKA and exhibit varying degrees of insulin deficiency between episodes</E>.
<A>Multiple studies indicate that measuring islet autoantibodies in individuals genetically at risk for type 1 diabetes</A> <E>identifies individuals who may develop type 1 diabetes (9)</E>.
<C>Indeed, the risk of type 1 diabetes</C> <E>increases as the number of relevant autoantibodies detected increases (43-45)</E>.
<CO>Risk based screening for prediabetes and/or type 2 diabetes should be considered after the onset of puberty or after 10 years of age, whichever occurs earlier, in children and adolescents with overweight (BMI >85th percen- tile) or obesity (BMI >95th per- centile) and who have one or more risk factor for diabetes</CO>.
<CO>If results are normal, testing should be repeated at a minimum of 3-year intervals, with consideration of more frequent testing dep

In [ ]:
text = f'''
Thiazolidinediones have been used successfully in patients with liver and kidney transplants, but side effects include fiuid retention, heart failure, and osteopenia (98,99).
Well-designed intervention trials examining the efficacy and safety of these and other antihyperglycemic agents in patients with PTDM are needed.
MONOGENIC DIABETES SYNDROMESRecommendationsAll children diagnosed with diabetes in the rst 6 months of life should have immediate genetic testing for neonatal diabetes.
AChildren and those diagnosed in early adulthood who have diabetes not characteristic of type 1 or type 2 diabetes that occurs in successive generations (suggestive of an autosomal dominant pattern of inheri- tance) should have genetic test- ing for maturity-onset diabetes of the young.
AIn both instances, consultation with a center specializing in diabetes genetics is recommended to understand the signicance of these mutations and how best to approach further evaluation, treatment, and genetic counseling.
EMonogenic defects that cause b-cell dysfunction, such as neonatal diabetes and MODY, represent a small fraction of patients with diabetes (,5%).
"Correct diagnosis has critical implications because mostTable 2.6-Most common causes of monogenic diabetes (102)Gene	Inheritance	Clinical featuresMODYNeonatal diabetesGCK	AD	GCK-MODY: stable, nonprogressive elevated fasting blood glucose; typically does notrequire treatment; microvascular complications are rare; small rise in 2-h PG level on OGTT (,54 mg/dL [3 mmol/L])HNF1A	AD	HNF1A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; lowered renal threshold for glucosuria; large rise in 2-h PG level on OGTT (.90 mg/dL [5 mmol/L]); sensitive to sulfonylureasHNF4A	AD	HNF4A-MODY: progressive insulin secretory defect with presentation in adolescence or earlyadulthood; may have large birth weight and transient neonatal hypoglycemia; sensitive to sulfonylureasHNF1B	AD	HNF1B-MODY: developmental renal disease (typically cystic); genitourinary abnormalities; atrophy of the pancreas; hyperuricemia; goutKCNJ11	AD	Permanent or transient: IUGR; possible developmental delay and seizures; responsive toSulfonylureasINS	AD	Permanent: IUGR; insulin requiringABCC8	AD	Permanent or transient: IUGR; rarely developmental delay; responsive to sulfonylureas6q24 (PLAGL1,HYMA1)AD for paternal duplicationsTransient: IUGR; macroglossia; umbilical hernia; mechanisms include UPD6, paternal duplication or maternal methylation defect; may be treatable with medications other than insulinGATA6	AD	Permanent: pancreatic hypoplasia; cardiac malformations; pancreatic exocrine insufficiency;insulin requiringEIF2AK3	AR	Permanent: Wolcott-Rallison syndrome: epiphyseal dysplasia; pancreatic exocrineinsufficiency; insulin requiringFOXP3	X-linked	Permanent: immunodysregulation, polyendocrinopathy, enteropathy X-linked (IPEX)syndrome: autoimmune diabetes; autoimmune thyroid disease; exfoliative dermatitis; insulin requiringAD, autosomal dominant; AR, autosomal recessive; IUGR, intrauterine growth restriction; OGTT, oral glucose tolerance test; 2-h PG, 2-h plasma glucose."
patients with KATP-related neonatal diabetes will exhibit improved glycemic control when treated with high-dose oral sulfonylureas instead of insulin.
Insulin gene (INS) mutations are the second most common cause of permanent neonatal diabetes, and, while intensive insulin management is currently the preferred treatment strategy, there are important genetic counseling considerations, as most of the mutations that cause diabetes are dominantly inherited.
Clinically, patients with GCK-MODY exhibit mild, stable, fasting hyperglycemia and do not require antihyperglycemic therapy except sometimes during pregnancy.
Mutations or deletions in HNF1B are associated with renal cysts and uterine malformations (renal cysts and diabetes [RCAD] syndrome).
Additionally, diagnosis can lead to identification of other affected family members.
A diagnosis of MODY should be considered in individuals who have atypical diabetes and multiple family members with diabetes not characteristic of type 1 or type 2 diabetes, although admittedly "atypical diabetes" is becoming increasingly difficult to precisely define in the absence of a definitive set of tests for either type of diabetes (104-110).
In most cases, the presence of autoantibodies for type 1 diabetes precludes further testing for monogenic diabetes, but the presence of autoantibodies in patients with monogenic diabetes has been reported (111).
Individuals in whom monogenic diabetes is suspected should be referred to a specialist for further evaluation if available, and consultation is available from several centers.
It is critical to correctly diagnose one of the monogenic forms of diabetes because these patients may be incorrectly diagnosed with type 1 or type 2 diabetes, leading to suboptimal, even potentially harmful, treatment regimens and delays in diagnosing other family members (114).
Genetic counseling is recommended to ensure that affected individuals understand the patterns of inheritance and the importance of a correct diagnosis.
The diagnosis of monogenic diabetes should be considered in children and adults diagnosed with diabetes in early adulthood with the following findings: Diabetes diagnosed within the first 6 months of life (with occasional cases presenting later, mostly INS and ABCC8 mutations) (103,116) Diabetes without typical features of type 1 or type 2 diabetes (negative diabetes-associated autoantibodies, non- obese, lacking other metabolic features especially with strong family history of diabetes) Stable, mild fasting hyperglycemia (100-150 mg/dL [5.5-8.5 mmol/L]),stable A1C between 5.6 and 7.6%(between 38 and 60 mmol/mol), especially if nonobesePANCREATIC DIABETES OR DIABETES IN THE CONTEXT OF DISEASE OF THE EXOCRINE PANCREASPancreatic diabetes includes both structural and functional loss of glucose- normalizing insulin secretion in the context of exocrine pancreatic dysfunction and is commonly misdiagnosed as type 2 diabetes.
Hyperglycemia due to general pancreatic dysfunction has been called "type 3c diabetes" and, more recently, diabetes in the context of disease of the exocrine pancreas has been termedpancreoprivic diabetes (1).
In the context of pancreatectomy, islet autotransplantation can be done to retain insulin secretion (123,124).
In some cases, this can lead to insulin independence.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the first prenatal visit in those with risk factors using standard diagnostic criteria.
GESTATIONAL DIABETES MELLITUSRecommendationsTest for undiagnosed prediabetes and diabetes at the rst prenatal visit in those with risk factors using standard diagnostic criteria.
BTest for gestational diabetes mellitus at 2428 weeks of gestation in pregnant women not previously found to have diabetes.
ATest women with gestational diabetes mellitus for prediabetes or diabetes at 412 weeks postpartum, using the 75-g oral glucose tolerance test and clinically appropriate nonpregnancy diagnostic criteria.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>Thiazolidinediones have been used successfully in patients with liver and kidney transplants,</C> <E>but side effects include fiuid retention, heart failure, and osteopenia (98,99).</E>
<O>Well-designed intervention trials examining the efficacy and safety of these and other antihyperglycemic agents in patients with PTDM are needed.</O>
<C>All children diagnosed with diabetes in the rst 6 months of life</C> <A>should have immediate genetic testing for neonatal diabetes.</A>
<C>Children and those diagnosed in early adulthood who have diabetes not characteristic of type 1 or type 2 diabetes that occurs in successive generations (suggestive of an autosomal dominant pattern of inheri- tance)</C> <A>should have genetic test- ing for maturity-onset diabetes of the young.</A>
<O>In both instances, consultation with a center specializing in diabetes genetics is recommended to understand the signicance of these mutations and how best to approach further evaluation, t

In [ ]:
text = f'''
BWomen with a history of gestational diabetes mellitus should have lifelong screening for the development of diabetes or pre- diabetes at least every 3 years.
BWomen with a history of gestational diabetes mellitus found to have prediabetes should receive intensive lifestyle interventions and/or metformin to prevent diabetes.
Because of the number of pregnant women with undiagnosed type 2 diabetes, it is reasonable to test women with risk factors for type 2 diabetes (133) (Table 2.3) at their initial prenatal visit, using standard diagnostic criteria (Table 2.2).
As effective prevention interventions are available (138,139), women diagnosed with GDM should receive lifelong screening for pre- diabetes to allow interventions to reduce diabetes risk and for type 2 diabetes to allow treatment at the earliest possible time (140).
DiagnosisGDM carries risks for the mother, fetus, and neonate.
The Hyperglycemia and Adverse Pregnancy Outcome (HAPO) study (141), a large-scale multinational cohort study completed by more than 23,000 pregnant women, demonstrated that risk of adverse maternal, fetal, and neonatal outcomes continuously increased as a function of maternal glycemia at 24-28 weeks of gestation, even within ranges previously considerednormal for pregnancy.
This one-step strategy was anticipated to significantly increase the incidence of GDM (from 5-6% to 15- 20%), primarily because only one abnormal value, not two, became sufficient to make the diagnosis (142).
A recent follow-up study of women participating in a blinded study of pregnancy OGTTs found that 11 years after their pregnancies, women who would have been diagnosed with GDM by the one-step approach, as compared with those without, were at 3.4-fold higher risk of developing prediabetes and type 2 diabetes and had children with a higher risk of obesity and increased body fat, suggesting that the larger group of women identified by the one-step approach would benefit from increased screening for diabetes and prediabetes that would accompany a history of GDM(144).
Table 2.7Screening for and diagnosis of GDMOne-step strategyPerform a 75-g OGTT, with plasma glucose measurement when patient is fasting and at 1 and 2 h, at 2428 weeks of gestation in women not previously diagnosed with diabetes.
The OGTT should be performed in the morning after an overnight fast of at least 8 h.
Additional well-designed clinical studies are needed to determine the optimal intensity of monitoring and treatment of women with GDM diagnosed by the one-step strategy (147,148).
The panel recommended a two- step approach to screening that used a 1-h 50-g glucose load test (GLT) followed by a 3-h 100-g OGTT for those who screened positive.
The American College of Obstetricians and Gynecologists (ACOG) recommends any of the commonly used thresholds of 130, 135, or 140 mg/dL for the 1-h 50-g GLT (150).
Treatment of higher-threshold maternal hyperglycemia, as identified by the two-step approach, reduces rates of neonatal macrosomia, large-for- gestational-age births (153), and shoulder dystocia, without increasing small-for- gestational-age births.
ACOG currently supports the two-step approach but notes that one elevated value, as opposed to two, may be used for the diagnosis of GDM (150).
IMPORTANCE Gestational diabetes is associated with several poor health outcomes.
At or after 24 weeks of gestation, oral glucose challenge tests with 140- and 135-mg/dL cutoffs had sensitivities of 82% and 93%, respectively, and specificities of 82% and 79%, respectively, against Carpenter and Coustan criteria, and a test with a 140-mg/dL cutoff had sensitivity of 85% and specificity of 81% against the National Diabetes Group Data criteria.
Based on 8 RCTs and 1 nonrandomized study (n = 3982), treatment was significantly associated with decreased risk of primary cesarean deliveries (relative risk [RR], 0.70 [95% CI, 0.54-0.91]; absolute risk difference [ARD], 5.3%), shoulder dystocia (RR, 0.42 [95% CI, 0.23-0.77]; ARD, 1.3%),macrosomia (RR, 0.53 [95% CI, 0.41-0.68]; ARD, 8.9%), large for gestational age (RR, 0.56[95% CI, 0.47-0.66]; ARD, 8.4%), birth injuries (odds ratio, 0.33 [95% CI, 0.11-0.99]; ARD,0.2%), and neonatal intensive care unit admissions (RR, 0.73 [95% CI, 0.53-0.99]; ARD, 2.0%).
Outcomes related to hyper- tension in pregnancy were classified as preeclampsia, gestational hypertension, or hypertensive disorders in pregnancy (mixed).
One new study (n = 1012)41 found 1-step screening of at-risk women was significantly associated with a reduction in late (>28 weeks of gestation) stillbirth (adjusted odds ratio [OR], 0.68 [95% CI, 0.47 to 0.97]).
The other new study (n = 2780)39 found universal 2-step screening, with early screening offered to women with risk factors, significantly associated with fewer cesarean deliveries (RR, 0.78 [95% CI, 0.66 to 0.92]; ARD, -4.8% [95% CI, -8.2% to -1.5%), birth injuries (fracture or dislocation; RR, 0.47 [95% CI, 0.23 to 0.97]; ARD, -0.9%[95% CI, -1.9% to 0.10%]), and admissions to the neonatal intensive care unit (NICU) (RR, 0.67 [95% CI, 0.58 to 0.78]; ARD, -8.7% [95% CI, -12.3% to -5.2%]) compared with historical controls.
Pre- specified analyses found screening in first trimester was significantly associated with decreased likelihood of NICU admissions vs second-trimester screening but with no significant difference for other outcomes.
No significant differences were found in anxiety and depressive symptoms before and after screening for those with negative or false-positive results in 2 cohort studies (n = 1015).44,48 One study (n = 100)42 found that anxiety symptoms scores were slightly higher (6 points on 60-point scale; P = .007) for women with vs without gestational diabetes immediately after receiving results but not significantly higher at gestational week 36 or 6 weeks postpartum.
One good-quality cohort study (n = 3778)46 found that the association between macrosomia and cesarean delivery in women with normoglycemia or untreated borderline gestational diabetes was not observed in those with treated gestational diabetes, suggesting that a gestational diabetes diagnosis may have increased the propensity to perform cesarean deliveries.
IADPSG vs Carpenter and Coustan ScreeningFive RCTs (n = 25 772)20-24 examined universal screening at 24 to 28 weeks of gestation with the 1-step IADPSG vs 2-step Carpenter and Coustan criteria (Table 1).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>Women with a history of gestational diabetes mellitus</C> <E>should have lifelong screening for the development of diabetes or pre- diabetes at least every 3 years</E>.
<C>Women with a history of gestational diabetes mellitus found to have prediabetes</C> <A>should receive intensive lifestyle interventions and/or metformin to prevent diabetes</A>.
<O>Because of the number of pregnant women with undiagnosed type 2 diabetes</O>, <CO>it is reasonable to test women with risk factors for type 2 diabetes (133) (Table 2.3) at their initial prenatal visit, using standard diagnostic criteria (Table 2.2)</CO>.
<O>As effective prevention interventions are available (138,139)</O>, <C>women diagnosed with GDM</C> <E>should receive lifelong screening for pre- diabetes to allow interventions to reduce diabetes risk and for type 2 diabetes to allow treatment at the earliest possible time (140)</E>.
<C>DiagnosisGDM carries risks for the mother, fetus, and neonate</C>.
<O>The

In [ ]:
text = f'''
In the largest trial, 1-step screening significantly increased risk for neonatal hypoglycemia vs 2-step screening, although this may have been in part due to the routine surveillance of neonates with risk factors including diagnosis of maternal gestational diabetes (eFigure 2 in the Supplement).
In 1 trial (n = 921)24 in which all women randomized to 2-step screening underwent the 100-g OGTT (to assist with blinding), 2-step screening was associated with significantly more testing- related adverse events than 1-step screening (eg, reactive hypoglycemia, vomiting, nausea).
Early vs Usual Timing for Carpenter and Coustan ScreeningOne good-quality RCT (n = 922)19 enrolling obese women found early (14 to 20 weeks) vs usual timing of screening with Carpenter and Coustan criteria potentially associated with increased risk of preeclampsia, but the difference was not statistically significant (RR, 1.42 [95% CI, 0.99 to 2.05]; ARD, 4.0% [95% CI, 0.0% to 8.0%]).
Treatment of gestational diabetes was significantly associated with lower risk of primary cesarean deliveries vs no treatment (3 studies; RR, 0.70 [95% CI, 0.54 to 0.91]; I2 = 0%; ARD, -5.3%[95% CI, -10.3% to -0.24%]).25,32,36 Treatment was also associated, though not significantly, with fewer preterm deliveries (4 studies; RR, 0.75 [95% CI, 0.56 to 1.01]; I2 = 0%; ARD, -2.6%[95% CI, -4.9% to 0.02%])31,32,35,36 (Figure 3).
Treatment was not significantly associated with reduced risk of gestational hypertension (2 trials32,35; some imprecision), total cesarean deliveries (8 trials25-29,31,32,35), emergency cesarean deliveries (1 trial27), induction of labor (5 trials25,27,28,32,35), or maternal birth trauma (2 studies27,36).
At or after 24 weeks of gestation, treatment of gestational diabetes was significantly associated with improved health outcomes.
We recommend that preconception counseling be provided to all women with diabetes who are considering pregnancy.
We suggest that women with diabetes seeking to conceive strive to achieve blood glucose and hemoglobin A1C (HbA1C) levels as close to normal as possible when they can be safely achieved without undue hypoglycemia.
We recommend that insulin-treated women with diabetes seeking to conceive be treated with multiple daily doses of insulin or continuous sc insulin infusion in preference to split-dose, premixed insulin therapy, because the former are more likely to allow for the achievement and maintenance of target blood glucose levels preconceptionally and, in the event of pregnancy, are more likely to allow for sufficient flexibility or precise adjustment of insulin therapy.
We suggest that a change to a woman's insulin regimen, particularly when she starts continuous sc in- sulin infusion, be undertaken well in advance of withdrawing contraceptive measures or otherwise trying to conceive to allow the patient to acquire expertise in, and the optimization of, the chosen insulin regimen.
We suggest that insulin-treated women with diabetes seeking to conceive be treated with rapid-acting insulin analog therapy (with insulin aspart or insulin lispro) in preference to regular (soluble) insulin.
(2IQQEE)Folic acid supplementation  1.4.We recommend that beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive, a woman with diabetes take a daily folic acid supplement to reduce the risk of neural tube defects.
We recommend that all women with diabetes who are seeking pregnancy have a detailed ocular assessment by a suitably trained and qualified eye care professional in advance of withdrawing contraceptive measures or otherwise trying to conceive (1IQQQQ), and if retinopathy is documented, the patient should be apprised of the specific risks to her of this worsening during pregnancy.
If the degree of retinopathy warrants therapy, we recommend deferring conception until the retinopathy has been treated and found to have stabilized.
We recommend that women with established retinopathy be seen by their eye specialist every trimester, then within 3 months of delivering, and then as needed.
(1IQEEE)  1.5 c. We suggest that pregnant women with diabetes not known to have retinopathy have ocular assessment performed soon after conception and then periodically as indicated during pregnancy.
(1IQEEE)  1.5 c. We suggest that pregnant women with diabetes not known to have retinopathy have ocular assessment performed soon after conception and then periodically as indicated during pregnancy. (2IQQEE) Renal function (preconception and during pregnancy)  1.6 a.
We suggest that all women with diabetes considering pregnancy have their renal function assessed (by measuring their urine albumin to creatinine ratio, serum creatinine, and estimated glomerular filtration rate [GFR]) in advance of withdrawing contraceptive measures or otherwise trying to conceive.
We recommend that a woman with diabetes who is seeking conception while taking an angiotensin-converting enzyme (ACE) inhibitor or angiotensin-receptor blocker in almost all cases should discontinue the medication before withdrawing contraceptive measures or otherwise trying to conceive.
(Ungraded recommendation)  1.7 d. We recommend when ACE inhibitors or angiotensin-receptor blockers have been continued up to the time of conception that the medication should be withdrawn immediately upon the confirmation of pregnancy.
We recommend that if a woman with diabetes is seeking pregnancy and has CAD, its severity should be ascertained, treatment instituted, and counseling provided as to the potential risks of pregnancy to the woman and fetus before the woman withdraws contraception or otherwise tries to conceive.
We suggest that bile acid-binding resins may be used in women with diabetes to treat hypercholesterolemia; however, this is seldom warranted.
For women with type 1 diabetes seeking conception, we recommend measurement of serum TSH and, if their thyroid peroxidase status is unknown, measurement of thyroid peroxidase antibodies before withdrawing contraceptive measures or otherwise trying to conceive.
We recommend weight reduction before pregnancy for overweight and obese women with diabetes.
(1IQQEE) In the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT) must be performed in the absence of symptoms of hyperglycemia and found to be abnormal on another day to confirm the diagnosis.
We recommend that pregnant women not previously identified (either during testing performed as per recommendation 2.1 or at some other time before 24 weeks gestation) with overt diabetes or gestational diabe- tes be tested for gestational diabetes (see Table 2) by having a 2-hour, 75-g oral glucose tolerance test (OGTT) performed at 24 to 28 weeks gestation.
(1IQQQE)  The 75-g OGTT should be performed after an over- night fast of at least 8 hours (but not more than 14 hours) and without having reduced usual carbohydrate intake for the preceding several days.
The test should be performed with the patient seated, and the patient should not smoke during the test.
One or more abnormal values establishes the diagnosis, with the exception that in the case of overt diabetes, but not gestational diabetes, a second test (either a fasting plasma glucose, untimed random plasma glucose, HbA1C, or OGTT), in the absence of symptoms of hyperglycemia, must be performed and found to be abnormal on another day to confirm the diagnosis of overt diabetes.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>In the largest trial, 1-step screening significantly increased risk for neonatal hypoglycemia vs 2-step screening,</C> <E>although this may have been in part due to the routine surveillance of neonates with risk factors including diagnosis of maternal gestational diabetes (eFigure 2 in the Supplement).</E>
<C>In 1 trial (n = 921)24 in which all women randomized to 2-step screening underwent the 100-g OGTT (to assist with blinding), 2-step screening was associated with significantly more testing- related adverse events than 1-step screening</C> <E>(eg, reactive hypoglycemia, vomiting, nausea).</E>
<C>Early vs Usual Timing for Carpenter and Coustan ScreeningOne good-quality RCT (n = 922)19 enrolling obese women found early (14 to 20 weeks) vs usual timing of screening with Carpenter and Coustan criteria potentially associated with increased risk of preeclampsia,</C> <E>but the difference was not statistically significant (RR, 1.42 [95% CI, 0.99 to 2.05]; ARD, 

In [ ]:
text = f'''
We recommend that women with gestational diabetes target blood glucose levels as close to normal as possible.
(1IQQQE)  2.3 c. We recommend using blood glucose-lowering pharmacological therapy if lifestyle therapy is insufficient to maintain normoglycemia in women with gestational diabetes.
We recommend that postpartum care for women who have had gestational diabetes should include measurement of fasting plasma glucose or fasting self-monitored blood glucose for 24 to 72 hours after delivery to rule out ongoing hyperglycemiaTable 1.
b Testing should use plasma glucose analyzed at a laboratory, not capillary blood glucose analyzed with a blood glucose meter.
We recommend that a 2-hour, 75-g OGTT should be undertaken 6 to 12 weeks after delivery in women with gestational diabetes to rule out prediabetes or diabetes.
(1IQQQE) If results are normal, we recommend this or other diagnostic tests for diabetes should be repeated periodically as well as before future pregnancies.
We recommend that all women who have had gestational diabetes receive counseling on lifestyle measures to reduce the risk of type 2 diabetes, the need for future pregnancies to be planned, and the need for regular diabetes screening, especially before any future pregnancies.
We recommend self-monitoring of blood glucose in all pregnant women with gestational or overt diabetes (1IQQQQ) and suggest testing before and either 1 or 2 hours after the start of each meal (choosing the postmeal time when it is estimated that peak postprandial blood glucose is most likely to occur) and, as indicated, at bed- time and during the night.
We recommend pregnant women with overt or gestational diabetes strive to achieve a target preprandial blood glucose .
We suggest pregnant women with overt or gestational diabetes strive to achieve target blood glucose levels 1 hour after the start of a meal  (2IQEEE) when these targets can be safely achieved without undue hypoglycemia.
We suggest that continuous glucose monitoring be used during pregnancy in women with overt or gestational diabetes when self-monitored blood glucose levels (or, in the case of the woman with overt diabetes, HbA1C values) are not sufficient to assess glycemic control (including both hyperglycemia and hypoglycemia).
We recommend medical nutrition therapy for all pregnant women with overt or gestational diabetes to help achieve and maintain desired glycemic control while pro- viding essential nutrient requirements.
We suggest that women with overt or gestational diabetes follow the Institute of Medicine revised guidelines for weight gain during pregnancy (1) (Table 4).
We suggest obese women with overt or gestational diabetes reduce their calorie intake by approximately one-third (compared with their usual intake before pregnancy) while maintaining a minimum intake of 1600 to 1800 kcal/d.
We suggest women with overt or gestational diabetes limit carbohydrate intake to 35% to 45% of total calories, distributed in 3 small- to moderate-sized meals and 2 to 4 snacks including an evening snack.
We recommend pregnant women with overt or gestational diabetes should follow the same guidelines for the intake of minerals and vitamins as for women without diabetes (1IQQEE), with the exception of taking folic acid 5 mg daily beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive (see Recommen- dation 1.4).
We suggest that at 12 weeks gestation, the dose of folic acid be reduced to 0.4 to 1.0 mg/d, which should be continued until the completion of breastfeeding.
We suggest that the long-acting insulin analog detemir may be initiated during pregnancy for those women who require basal insulin and for whom neutral protamine Hagedorn (NPH) insulin, in appropriate doses, has previously resulted in, or for whom it is thought NPH insulin may result in, problematic hypoglycemia; insulin detemir may be continued in those women with diabetes already successfully taking insulin detemir before pregnancy.
We suggest that those pregnant women success- fully using insulin glargine before pregnancy may continue it during pregnancy.
(2IQQQO)  5.1 d. We recommend the ongoing use of continuous sc insulin infusion during pregnancy in women with diabetes when this has been initiated before pregnancy (1IQQQE), but suggest that continuous sc insulin infusion not be initiated during pregnancy unless other insulin strategies including multiple daily doses of insulin have first been tried and proven unsuccessful.
We suggest that metformin therapy be used for glycemic control only for those women with gestational diabetes who do not have satisfactory glycemic control despite medical nutrition therapy and who refuse or can- not use insulin or glyburide and are not in the first trimester.
Evidence  Women with diabetes who receive preconception counselinghave better preconception glycemic control (8, 9) and are more likely to have favorable pregnancy outcomes, including lower rates of congenital anomalies (9, 10) and spontaneous abortions (11, 12).
Evidence  Maternal hyperglycemia in the first few weeks of pregnancy increases the risk of fetal malformations, spontaneous abortions, and perinatal mortality (14 -18).
We suggest that women with diabetes successfully using the long-acting insulin analogs insulin detemir or insulin glargine preconceptionally may continue with this therapy before and then during pregnancy.
(2IQQEE)1.3a- d. Evidence  Rapid-acting insulin analogs are likely more able than regular human insulin to help a woman achieve postprandial blood glucose targets and are less likely to cause hypoglycemia; fetal outcomes, however, seem comparable (28 -30).
We recommend that beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive, a woman with diabetes take a daily folic acid supplement to reduce the risk of neural tube defects.
Evidence  Taking a daily folic acid supplement preconceptionally reduces the risk of neural tube defects (38).
The absence of retinopathy before conception confers very small risk of development of significant retinopathy during pregnancy; nonetheless, significant retinopathy can develop and progress during pregnancy, even if not identified preconceptionally (40).
Additional risk factors for progression of retinopathy during pregnancy include preexisting hypertension (43), poorly controlled hypertension during pregnancy (44), preeclampsia (45), and poor glycemic control at the onset of and during pregnancy (41).
We suggest that all women with diabetes considering pregnancy have their renal function assessed (by measuring their urine albumin to creatinine ratio, serum creatinine, and estimated GFR) in advance of withdrawing contraceptive measures or otherwise trying to conceive.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are the extracted cause, effect, signal, condition and action from the given sentences:

1. <C>We recommend that women with gestational diabetes</C> <A>target blood glucose levels as close to normal as possible</A>.
2. <C>We recommend using blood glucose-lowering pharmacological therapy</C> <CO>if lifestyle therapy is insufficient to maintain normoglycemia in women with gestational diabetes</CO>.
3. <C>We recommend that postpartum care for women who have had gestational diabetes</C> <A>should include measurement of fasting plasma glucose or fasting self-monitored blood glucose for 24 to 72 hours after delivery to rule out ongoing hyperglycemia</A>.
4. <O>b Testing should use plasma glucose analyzed at a laboratory, not capillary blood glucose analyzed with a blood glucose meter</O>.
5. <C>We recommend that a 2-hour, 75-g OGTT</C> <A>should be undertaken 6 to 12 weeks after delivery in women with gestational diabetes to rule out prediabetes or diabetes</A>.

In [ ]:
text  = f'''
(Ungraded recommendation) We suggest that a woman with diabetes who has a significantly reduced GFR be assessed by a nephrologist before pregnancy, both for baseline renal assessment and to review the woman's specific risk of worsening renal function in the event of pregnancy.
We suggest that all women with diabetes and preconceptional renal dysfunction have their renal function monitored regularly during pregnancy.
Evidence  Renal dysfunction in a pregnant woman with type 1 diabetes is associated with an increased risk of adverse maternal and fetal outcomes, including an increased risk of preeclampsia (46 - 48).
(1IQQEE)  1.7 c. We suggest that in the exceptional case where the degree of renal dysfunction is severe and there is uncertainty about when conception will occur, physicians and patients be engaged in shared decision-making about whether to continue ACE inhibitors or angiotensin-receptor blockers.
(Ungraded recommendation)  1.7 d. We recommend that when ACE inhibitors or angiotensin-receptor blockers have been continued up to the time of conception that the medication should be withdrawn immediately upon the confirmation of pregnancy.
Hypertension in a preconceptional woman increases the risk of adverse outcomes during pregnancy, especially her risk of developing preeclampsia (55).
We recommend that if a woman with diabetes has sufficient numbers of vascular risk factors (particularly the duration of the woman's diabetes and her age), screening studies for CAD be undertaken in advance of withdrawing contraceptive measures or otherwise trying to conceive.
Evidence  Myocardial infarction during pregnancy is associated with adverse maternal and fetal outcomes including maternal and fetal demise (57, 58).
Evidence  Autoimmune thyroid disease is common among women of childbearing age with type 1 diabetes with prevalence rates as high as 44% (62).
A systematic review and meta-analysis (70) demonstrated that abnormal screening test results were associated with worse maternal and fetal outcomes.
Remarks  We acknowledge that with universal testing for diabetes in early pregnancy, there will be a high rate of false- positive results (70) and that women with positive testing may have anxiety and will suffer the burden of additional testing.
Nevertheless, we recommended universal testing because we place the highest value on preventing fetal complications.
We recommend that pregnant women not previously identified (either during testing performed as per recommendation 2.1 or at some other time before 24 weeks gestation) with overt diabetes or gestational diabetes be tested for gestational diabetes (see Table 2) by having a 2-hour, 75-g OGTT performed at 24 to 28 weeks gestation.
Evidence  Pregnant women who develop gestational diabetes are at risk of adverse pregnancy outcomes, which may be prevented by adequate treatment (6, 71).
To exclude from the definition of gestational diabetes those women with overt diabetes, most of our committee supports redefining gestational diabetes as defined in the Hyperglycemia and Adverse Pregnancy Outcome study; that is, gestational diabetes is "the condition associated with degrees of maternal hyperglycemia less severe than those found in overt diabetes but associated with an increased risk of adverse pregnancy outcomes" (6).
It is recognized that implementation of the IADPSG criteria will lead to a substantial increase in the numbers of pregnant women being diagnosed with gestational diabetes with the attendant medicalization of pregnancies and with a concomitant increase in healthcare costs both to individuals and to society.
We recommend using blood glucose-lowering pharmacological therapy if lifestyle therapy is insufficient to maintain normoglycemia in women with gestational diabetes.
Lifestyle therapy for gestational diabetes results in a lower incidence of reduced birth weight, large-for-gestational-age births, and preeclampsia (71, 82).
Both aerobic exercise (83- 86) and non-weight-bearing exercise (87) have been shown to lower blood glucose levels in women with gestational diabetes.
(2IQQEE)2.4a- e. Evidence  Women who have had gestational diabetes are at high risk for the later development of impaired fasting glucose, impaired glucose tolerance, overt diabetes (91-93), and the metabolic syndrome (91, 94 -106).
Infants born to mothers with gestational diabetes are at increased risk of the later development of obesity or type 2 diabetes (107).
Pregnant women with type 1 diabetes are at increased risk of hypoglycemia including severe hypoglycemia, especially during the first trimester (21-25).
There is also an increased risk of hypoglycemia in pregnant women with type 2 diabetes (26).
Nevertheless, nutrition therapy has been shown to improve glycemic control for people living with overt diabetes (119, 120) and for women with gestational diabetes (121).
Evidence  In the absence of definitive evidence regarding optimal weight gain for women with gestational or overt diabetes-and with evidence both that women who gain excess weight during pregnancy may retain it after childbirth(123) and that women who are overweight or obese before pregnancy are at an increased risk for complications during pregnancy (including hypertensive complications, stillbirth, and increased risk for cesarean section) (124 - 127)-and with the reassurance that limiting maternal weight gain is not associated with a decrease in fetal birth weight (128), we conclude that following the Institute of Medicine recommendations for weight gain during pregnancy, although not written specifically for women with overt or gestational diabetes, is nonetheless appropriate for women with these conditions (Table 4) (129).
Moderate energy restriction (1600 -1800 kcal/d) in pregnant women with overt diabetes improves mean glycemia and fasting insulinemia without inhibiting fetal growth or birth weight or inducing ketosis (128).
Energy intake of approximately 2050 kcal in all BMI categories in women with gestational diabetes has been reported to limit maternal weight gain, maintain euglycemia, avoid ketonuria, and maintain an average birth weight of 3542 g (130).
Some authorities suggest that a minimum of 175 g/d carbohydrate should be provided, which is higher than the 130 g/d recommended for nonpregnant women (1).
We recommend pregnant women with overt or gestational diabetes should follow the same guidelines for the intake of minerals and vitamins as for women without diabetes (1IQQEE), with the exception of taking folic acid 5 mg daily beginning 3 months before withdrawing contraceptive measures or otherwise trying to conceive (see Recommendation 1.4).
We suggest that at 12 weeks gestation, the dose of folic acid be reduced to 0.4 to 1.0 mg/d, which should be continued until the completion of breastfeeding (2IQQEE).
We suggest that the long-acting insulin analog detemir may be initiated during pregnancy for those women who require basal insulin and for whom NPH insulin, in appropriate doses, has previously resulted in, or for whom it is thought NPH insulin may result in, problematic hypoglycemia; insulin detemir may be continued in those women with diabetes already successfully taking insulin detemir before pregnancy.
We suggest that those pregnant women successfully using insulin glargine before pregnancy may continue it during pregnancy.
We suggest that the rapid-acting insulin analogs lispro and aspart be used in preference to regular (soluble) insulin in pregnant women with diabetes.
We recommend the ongoing use of continuous sc insulin infusion during pregnancy in women with diabetes when this has been initiated before pregnancy (1IQQQE) but suggest that continuous sc insulin infusion not be initiated during pregnancy unless other insulin strategies including multiple daily doses of insulin have first been tried and proven unsuccessful.
Evidence  In nonpregnant women, insulin detemir is associated with less hypoglycemia than NPH insulin (141-143).
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>(Ungraded recommendation) We suggest that a woman with diabetes who has a significantly reduced GFR be assessed by a nephrologist before pregnancy, both for baseline renal assessment and to review the woman's specific risk of worsening renal function in the event of pregnancy.</C>
<A><O>We suggest that all women with diabetes and preconceptional renal dysfunction have their renal function monitored regularly during pregnancy.</O></A>
<E>Evidence  Renal dysfunction in a pregnant woman with type 1 diabetes is associated with an increased risk of adverse maternal and fetal outcomes, including an increased risk of preeclampsia (46 - 48).</E>
<C>(1IQQEE)  1.7 c. We suggest that in the exceptional case where the degree of renal dysfunction is severe and there is uncertainty about when conception will occur, physicians and patients be engaged in shared decision-making about whether to continue ACE inhibitors or angiotensin-receptor blockers.</C>
<A>(Ungraded recomm

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>Additionally, compared with multiple daily doses of insulin, continuous sc insulin in- fusion provides greater lifestyle flexibility, easier blood glucose management in women experiencing morning nausea, less blood glucose variability, and facilitates man- aging glucose control in the peridelivery setting (170).</C>
<A>5.1 d. Remarks  Owing to the potential risk of temporarily worsened blood glucose control, ketoacidosis, and hypoglycemia when continuous sc insulin infusion is initiated, its use during pregnancy should be limited to those patients already successfully using this method of insulin administration before pregnancy and to those women who, during pregnancy, have not succeeded with other insulin strategies including multiple daily doses of insulin.</A>
<C>We suggest that glyburide (glibenclamide) is a suitable alternative to insulin therapy for glycemic control in women with gestational diabetes who fail to achieve sufficient glycemic control afte

In [ ]:
text = f'''
Additionally, compared with multiple daily doses of insulin, continuous sc insulin in- fusion provides greater lifestyle flexibility, easier blood glucose management in women experiencing morning nausea, less blood glucose variability, and facilitates man- aging glucose control in the peridelivery setting (170).
5.1 d. Remarks  Owing to the potential risk of temporarily worsened blood glucose control, ketoacidosis, and hypoglycemia when continuous sc insulin infusion is initiated, its use during pregnancy should be limited to those patients already successfully using this method of insulin administration before pregnancy and to those women who, during pregnancy, have not succeeded with other insulin strategies including multiple daily doses of insulin.
We suggest that glyburide (glibenclamide) is a suitable alternative to insulin therapy for glycemic control in women with gestational diabetes who fail to achieve sufficient glycemic control after a 1-week trial of medical nutrition therapy and exercise except for those women with a diagnosis of gestational diabetes before 25 weeks gestation and for those women with fasting plasma glucose levels >110 mg/dL (6.1 mmol/L), in which case insulin therapy is preferred.
Evidence  In pregnant women taking glyburide, the umbilical cord glyburide concentration is undetectable (90, 173- 176) or, at most, very low (177).
Glyburide is effective in controlling blood glucose in women with gestational diabetes and has been associated with favorable neonatal outcomes including the rate of large-for-gestational-age infants, macrosomia, neonatal intensive care unit admission, and neonatal hypoglycemia (90).
Compared with women with gestational diabetes taking insulin, those taking metformin have no difference in maternal glycemic control, significantly lower rates of neonatal hypoglycemia, and no increased risk of congenital anomalies or other serious maternal or neonatal adverse events.
Additionally, metformin-treated women with gestational diabetes have increased rates of preterm birth (89).
Evidence  Elevated maternal blood glucose during labor and delivery increases the risk of neonatal hypoglycemia and fetal distress (195-200) as well as birth asphyxia and ab- normal fetal heart rate (200, 201), albeit with these associations having been mainly demonstrated in obser- vational studies of women with type 1 diabetes.
We recommend whenever possible women with overt or gestational diabetes should breastfeed their infant.
We recommend that breastfeeding women with overt diabetes successfully using metformin or glyburide therapy during pregnancy should continue to use these medications, when necessary, during breastfeeding.
Evidence  The increased risk of infants born to women with diabetes for childhood obesity and the later development of impaired glucose intolerance and diabetes (81) is reduced by breastfeeding (202-211).
Breastfeeding may also facilitate postpartum weight loss and reduce maternal and neonatal risk for the later development of type 2 diabetes (212, 213).
Evidence  Combined oral contraceptive use by women with type 1 diabetes does not affect their glycemic control or increase their risk of end-organ injury (217-219).
We suggest women with type 1 diabetes be screened for postpartum thyroiditis with a TSH at 3 and 6 months postpartum.
SUMMARY OF RECOMMENDATION AND EVIDENCE    The USPSTF recommends screening for abnormal blood glucose as part of cardiovascular risk assessment in adults aged 40 to 70 years who are overweight or obese.
Recommendation: The USPSTF recommends screening for abnormal blood glucose as part of cardiovascular risk assessment in adults aged 40 to 70 years who are overweight or obese.
Modifiable cardiovascular risk factors include abnormal blood glucose, hypertension, hyperlipidemia or dyslipidemia, smoking, overweight and obesity, physical inactivity, and an unhealthy diet.
PopulationAdults aged 40 to 70 y who are overweight or obeseRecommendationScreen for abnormal blood glucose.
Treatment and InterventionsEffective behavioral interventions combine counseling on a healthful diet and physical activity and involve multiple contacts over extended periods.
The USPSTF previously found adequate evidence that intensive behavioral counseling interventions for persons at increased risk for CVD have moderate benefits in lowering CVD risk.
Populations in which these benefits have been shown include persons who are obese or overweight and have hypertension, hyperlipidemia or dyslipidemia, and/or IFG or IGT.
Harms of Early Detection and Treatment    The USPSTF found that measuring blood glucose is associated with short-term anxiety but not long-term psychological harms.
The harms of drug therapy for the prevention of diabetes are small to moderate, depending on the drug and dosage used.
Persons who have a family history of diabetes, have a history of gestational diabetes or polycystic ovarian syndrome, or are members of certain racial/ ethnic groups (that is, African Americans, American In- dians or Alaskan Natives, Asian Americans, Hispanics or Latinos, or Native Hawaiians or Pacific Islanders) may be at increased risk for diabetes at a younger age or at a lower body mass index.
Although direct evidence that preventing a diagnosis of type 2 diabetes results in improved health outcomes is limited, primary prevention that reduces the chances of a diagnosis may reduce the adverse consequences of disease management.
Cohort and modeling studies suggest that re- screening every 3 years may be a reasonable approach for adults with normal blood glucose levels (3-7).
Other Approaches to Prevention    Because overweight and obesity, physical inactivity, abnormal lipid levels, high blood pressure, and smoking are all modifiable risk factors for cardiovascular events, the USPSTF recommends screening and appropriate interventions for these conditions (available at www.uspreventiveservicestaskforce.org).
The USPSTF recommends screening for obesity in adults and offering or referring those with a body mass index of 30 kg/m2 or greater to intensive, multicomponent behavioral interventions.
The USPSTF recommends offering or referring adults who are overweight (body mass index >25 kg/ m2) and have additional cardiovascular risk factors to intensive behavioral counseling interventions to promote a healthful diet and physical activity for CVD prevention.
The USPSTF recommends screening for lipid disorders in men aged 35 years or older and women aged 45 years or older who are at increased risk for coronary heart disease.
'''

In [ ]:
text = f'''
The USPSTF also recommends screening for hypertension in adults aged 18 years or older and that clinicians ask all adults about tobacco use and provide tobacco cessation interventions to those who use tobacco products.
Useful Resources    The Community Preventive Services Task Force recommends combined diet and physical activity promotion programs for persons who are at increased risk for type 2 diabetes.
Clinical trials and additional modeling studies are needed to better elucidate the optimal frequency of screening and the age at which to start screening.
More U.S. data are also needed on the benefits and harms of lifestyle interventions and medical treatments for screen-detected IFG, IGT, and diabetes over a longer follow-up period.
Uncontrolled diabetes is a leading cause of cardiovascular mortality and morbidity and may also result in other complications, such as vision loss, renal failure, and amputation.
The USPSTF determined that there is adequate evidence that lifestyle interventions can prevent or delay progression to type 2 diabetes.
Metformin, thiazolidinediones, and aglucosidase inhibitors were all found to be effective in preventing or delaying progression.
The Diabetes Prevention Program (11), conducted in the United States in 3234 participants, found that an intensive lifestyle modification intervention was associated with better quality of life at 3-year follow-up.
Intensive glycemic control was associated with reduced risk for nonfatal myocardial infarction and microvascular disease but increased risk for severe hypoglycemia (15, 16).
However, a consistent benefit of stroke reduction was found in study groups with intensive versus standard blood pressure control (23, 24).
The ACCORD study (25) found that intensive lipid-lowering therapy was associated with reduced risk for a composite cardiovascular outcome in men with diabetes but possible harm in women (interaction P = 0.01) (25).
One trial found that acarbose was associated with higher risk for withdrawal because of adverse events compared with placebo (42).
One large trial found that nateglinide (43) was associated with increased risk for hypoglycemia compared with placebo (RR, 1.73 [CI, 1.57 to 1.92]); further, valsartan was associated (44) with hypotension related adverse events (RR, 1.16 [CI, 1.11 to 1.23]).
One trial (45) found that rosiglitazone was associated with increased risk for congestive heart failure (HR, 7.04 [CI, 1.6 to 31]), but the estimate was imprecise.
UPDATE OF PREVIOUS USPSTF RECOMMENDATION    This is an update of the 2008 USPSTF recommendation statement in which the USPSTF recommended screening for diabetes in asymptomatic adults with hypertension (defined as sustained blood pressure of>135/80 mm Hg) (B recommendation).
RECOMMENDATIONS OF OTHERS    The American Diabetes Association (46) recommends screening for diabetes in adults aged 45 years or older and screening in persons with multiple risk factors regardless of age.
(27) found that maternal plasma glucose concentration during pregnancy (measured 2 h after a 75-g load) had a continuous association with adverse pregnancy outcomes (LGA and cesarean section).
(16) found risk of severe macrosomia, neonatal hypoglycemia, and hyperbilirubinemia increased with increasing number of abnormal glucose values according to current American Diabetes Association cut points (2,5) among women who did not meet National Diabetes Data Group criteria for GDM (31).
Adiposity in offspring at 5-7 years of age was significantly associated with measures of maternal glycemia (50-g glucose challenge and/or 100-g OGTT) during pregnancy.
Fetal macrosomia (LGA) is a major indicator of the effects of hyperglycemia during pregnancy (12,35,36).
Long-term risks associated with fetal macrosomia in infants of women with GDM (independent of confounders) include childhood overweight (43,44) and metabolic factors that may increase risk of cardiovascular disease (CVD) (45).
Measuring FPG alone identified 8.3% of the cohort as having GDM.
In addition to the outcomes used to deter- mine the thresholds, there were strong associations between maternal glucose and preeclampsia (ORs 1.40 -1.57) and shoulder dystocia and/or birth injury (1.30 -1.43).
The frequency of birth weight, C-peptide, or percent infant body fat >90th percentile was approximately twofold greater when any of the glucose values were greater than or equal to the threshold.
The frequency of preeclampsia was twofold higher when one or more glucose values met or exceeded threshold, and frequencies of preterm delivery and primary cesarean section were >45% higher.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>The USPSTF also recommends screening for hypertension in adults aged 18 years or older</C> <O>and that</O> <A>clinicians ask all adults about tobacco use and provide tobacco cessation interventions to those who use tobacco products</A>.
<C>Useful Resources    The Community Preventive Services Task Force recommends combined diet and physical activity promotion programs for persons who are at increased risk for type 2 diabetes</C>.
<C>Clinical trials and additional modeling studies are needed</C> <O>to</O> <A>better elucidate the optimal frequency of screening and the age at which to start screening</A>.
<C>More U.S. data are also needed</C> <O>on the</O> <A>benefits and harms of lifestyle interventions and medical treatments for screen-detected IFG, IGT, and diabetes over a longer follow-up period</A>.
<C>Uncontrolled diabetes is a leading cause of cardiovascular mortality and morbidity</C> <E>and may also result in other complications, such as vision loss, r

In [ ]:
text = f'''
In both randomized controlled trials, treatment, achieved primarily by diet/lifestyle modification, resulted in reduced birth weight and frequency of LGA births and pre- eclampsia.
As ongoing epidemics of obesity and diabetes result in more type 2 diabetes in young women, the number who are undiagnosed (before pregnancy) is increasing (49,50).
IADPSG Consensus Panel members favored use of any available certified laboratory measure of glucose (FPG, random plasma glucose, or A1C) for initial detection of possible cases.
An expert committee recently recommended that an A1C value >6.5% (measured in a laboratory standardized/aligned with the Diabetes Control and Complications Trial [DCCT]/UK Prospective Diabetes Study [UKPDS] assay) be used for diagnosis of diabetes outside pregnancy (55).
It is desirable to detect overt diabetes in pregnancy as early as possible to provide an opportunity to optimize pregnancy outcome.
However, if enrollment is at 24 weeks' gestation or later and overt diabetes is not found, the initial test should be followed by a 75-g OGTT.
It was recognized that any assessment of glycemia in early pregnancy would also result in detection of milder degrees of hyperglycemia short of overt diabetes.
Recently, it was reported that higher first trimester FPG levels (lower than those diagnostic of diabetes) are associated with increased risks of later diagnosis of GDM and adverse pregnancy outcomes (57).
Table 2-Strategy for the detection and diagnosis of hyperglycemic disorders in pregnancy*First prenatal visitMeasure FPG, A1C, or random plasma glucose on all or only high-risk women If results indicate overt diabetes as per Table 1  Treatment and follow-up as for preexisting diabetes If results not diagnostic of overt diabetesand fasting plasma glucose >5.1 mmol/l (92 mg/dl) but , diagnose as GDMand fasting plasma glucose , test for GDM from 24 to 28 weeks' gestation with a 75-g OGTT24-28 weeks' gestation: diagnosis of GDM2-h 75-g OGTT: perform after overnight fast on all women not previously found to have overt diabetes or GDM during testing earlier in this pregnancyOvert diabetes if fasting plasma glucose >7.0 mmol/l (126 mg/dl)GDM if one or more values equals or exceeds thresholds indicated in Table 1 Normal if all values on OGTT less than thresholds indicated in Table 1*To be applied to women without known diabetes antedating pregnancy.
Universal early testing in populations with a high prevalence of type 2 diabetes is recommended, especially if metabolic testing in this age-group is not commonly performed outside of pregnancy.
Well-designed studies should be conducted to determine whether it is beneficial and cost-effective to perform an OGTT in women who do not have overt diabetes at early testing but have indeterminate nondiagnostic results.
Glucose testing early in pregnancy to detect overt diabetes and again with a 75-g OGTT at 24 -28 weeks' of gestation in all pregnancies not already diagnosed with overt diabetes or GDM by early testing represents fundamental changes in strategies for detection and diagnosis of hyperglycemia in pregnancy.
Clinical Preventive Service RecommendationDiabetes Screening, AdultsType 2 Diabetes, AdultsThe AAFP recommends screening for type 2 diabetes in adults aged 40 to 70 years who have overweight or obesity.
The AAFP also agrees that there is insufficient evidence for screening prior to 24 weeks gestation.1The AAFP has also reviewed the US Preventive Services Task Force evidence report and agrees that nonpregnant adults aged 40 to 70 who have obesity or overweight be screened for type 2 diabetes as part of risk assessment for cardiovascular disease.2  Screening in this setting refers to measuring fasting plasma glucose, HbA1c level, or using an oral glucose tolerance test.
While AAFP acknowledges that diabetes, like all chronic conditions, exists on a continuum and that insulin resistance is a risk factor for developing diabetes, the current evidence does not show improvement in long term health outcomes for screening for prediabetes in adults who have obesity or overweight.
The evidence did show a reduction in the incidence of diabetes in individuals at risk for diabetes who received intense lifestyle interventions or metformin, however there was not an associated improvement in health outcomes.
Lifestyle interventions to prevent progression for prediabetes to diabetes are effective in reducing weight, blood pressure and cholesterol and would likely be recommended and beneficial for these individuals, even in the absence of screening.
'''

In [ ]:
import openai

openai.api_key = "sk-**********************"

prompt_1 = f"""
Perform the following actions:
1 - You will be provided with text delimited by triple quotes. Extract the cause, effect,signal, condition and action from the given sentence. Enclose the begnining and the end with tags. Use <A> for beginning of an action and </A> for end of action, <C></C> for cause, <CO></CO> for condition, <E></E> for effect and <O></O> for others.

2 - Output a table that contains the following columns \
keys: Sentence , prediction
Separate your answers with line breaks  and output a excel file.

Text:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
<C>In both randomized controlled trials, treatment, achieved primarily by diet/lifestyle modification,</C> <E>resulted in reduced birth weight and frequency of LGA births and pre- eclampsia.</E>
<C>As ongoing epidemics of obesity and diabetes result in more type 2 diabetes in young women,</C> <E>the number who are undiagnosed (before pregnancy) is increasing (49,50).</E>
<O>IADPSG Consensus Panel members favored use of any available certified laboratory measure of glucose (FPG, random plasma glucose, or A1C) for initial detection of possible cases.</O>
<O>An expert committee recently recommended that an A1C value >6.5% (measured in a laboratory standardized/aligned with the Diabetes Control and Complications Trial [DCCT]/UK Prospective Diabetes Study [UKPDS] assay) be used for diagnosis of diabetes outside pregnancy (55).</O>
<CO>It is desirable to detect overt diabetes in pregnancy as early as possible</CO> <A>to provide an opportunity to optimize pregnancy ou

In [ ]:
import pandas as pd
data_preds = pd.read_excel('/content/GPT-4-predictions-GD-data.xlsx',sheet_name = 'Zero-shot')

In [ ]:
data_preds

,final,Unnamed: 1
0,<C>Pregnant persons with gestational diabetes<...,NaN
1,<O>Gestational diabetes has also been associat...,NaN
2,<O>EVIDENCE ASSESSMENT The USPSTF concludes wi...,NaN
3,<A><O>RECOMMENDATION The USPSTF recommends scr...,NaN
4,<O>IImportanceGestational diabetes is diabetes...,NaN
...,...,...
284,<O>Clinical Preventive Service RecommendationD...,NaN
285,<O>The AAFP also agrees that there is insuffic...,NaN
286,"<O>While AAFP acknowledges that diabetes, like...",NaN
287,<E>The evidence did show a reduction in the in...,NaN


In [ ]:
import re
#ce-rela = []
fullText = []
causal_relation = []
variables = []
effect=[]
cause=[]
signal=[]
action=[]
condition=[]
clean = re.compile('<.*?>')
for para in data_preds['final']:
  para=para.encode("ascii", "ignore")
  para=para.decode()
  #print(para)
  #tags = re.compile('<.*?>(.*?)</.*?>').findall(para)
  sent = re.sub(clean, '', para)
  tags = re.compile('(<C>|<E>|<A>|<S>|<CO>)(.*?)(</C>|</E>|</A>|</S>|</CO>)').findall(para)
 # print(tags)
  for i in tags:
    if i[0] == '<C>':
      variables.append([sent,i[1],'cause'])
    elif i[0] == '<E>':
      variables.append([sent, i[1], 'effect'])
    elif i[0] == '<A>':
      variables.append([sent,i[1], 'action'])
    elif i[0] == '<S>':
      variables.append([sent,i[1],'signal'])
    elif i[0] == '<CO>':
      variables.append([sent,i[1],'condition'])

In [ ]:
df_preds = pd.DataFrame((variables),columns =['text','variables','role'])
sent_id, unique = pd.factorize(df_preds['text'])
df_preds['id'] = sent_id
df_preds.to_excel('/content/data_preds.xlsx')

In [ ]:
df_preds

,text,variables,role,id
0,Pregnant persons with gestational diabetes are...,Pregnant persons with gestational diabetes,cause,0
1,Pregnant persons with gestational diabetes are...,are at increased risk for maternal and fetal c...,effect,0
2,RECOMMENDATION The USPSTF recommends screening...,<O>RECOMMENDATION The USPSTF recommends screen...,action,1
3,Pregnant persons at 24 weeks of gestation or a...,<O>Screen for gestational diabetes.Grade: BPre...,action,2
4,Screen: If the person is pregnant and is at le...,<O>Screen: If the person is pregnant and is at...,action,3
...,...,...,...,...
293,"However, if enrollment is at 24 weeks' gestati...","However, if enrollment is at 24 weeks' gestati...",condition,203
294,"However, if enrollment is at 24 weeks' gestati...",the initial test should be followed by a 75-g ...,action,203
295,"Recently, it was reported that higher first tr...","Recently, it was reported that higher first tr...",cause,204
296,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...,effect,205


In [ ]:
data_gold = pd.read_excel('/content/data_GD_final.xlsx',sheet_name='Sheet2')

In [ ]:
data_gold

,final
0,<S>Pregnant persons</S> <C>with gestational di...
1,<C>Gestational diabetes</C> has also been ass...
2,EVIDENCE ASSESSMENT The USPSTF concludes with ...
3,RECOMMENDATION The USPSTF recommends <A>screen...
4,Importance<C>Gestational diabetes</C> is diab...
...,...
284,Clinical Preventive Service RecommendationDiab...
285,The AAFP also agrees that there is insufficien...
286,"While AAFP acknowledges that <E>diabetes</E> ,..."
287,The evidence did show a <E>reduction in the in...


In [ ]:
import re
#ce-rela = []
fullText = []
causal_relation = []
variables = []
effect=[]
cause=[]
signal=[]
action=[]
condition=[]
clean = re.compile('<.*?>')
for para in data_gold['final']:
  para=para.encode("ascii", "ignore")
  para=para.decode()
  #print(para)
  #tags = re.compile('<.*?>(.*?)</.*?>').findall(para)
  sent = re.sub(clean, '', para)
  tags = re.compile('(<C>|<E>|<A>|<S>|<CO>)(.*?)(</C>|</E>|</A>|</S>|</CO>)').findall(para)
 # print(tags)
  for i in tags:
    if i[0] == '<C>':
      variables.append([sent,i[1],'cause'])
    elif i[0] == '<E>':
      variables.append([sent, i[1], 'effect'])
    elif i[0] == '<A>':
      variables.append([sent,i[1], 'action'])
    elif i[0] == '<S>':
      variables.append([sent,i[1],'signal'])
    elif i[0] == '<CO>':
      variables.append([sent,i[1],'condition'])

In [ ]:
df_gold = pd.DataFrame((variables),columns =['text','variables','role'])
sent_id, unique = pd.factorize(df_gold['text'])
df_gold['id'] = sent_id
df_gold.to_excel('/content/data_gold.xlsx')

In [ ]:
df_gold

,text,variables,role,id
0,Pregnant persons with gestational diabetes ar...,Pregnant persons,signal,0
1,Pregnant persons with gestational diabetes ar...,with gestational diabetes,cause,0
2,Pregnant persons with gestational diabetes ar...,increased risk for maternal and fetal complica...,effect,0
3,Gestational diabetes has also been associated...,Gestational diabetes,cause,1
4,Gestational diabetes has also been associated...,increased risk of several long-term health out...,effect,1
...,...,...,...,...
729,"While AAFP acknowledges that diabetes , like a...",adults who have obesity or overweight,condition,286
730,The evidence did show a reduction in the incid...,reduction in the incidence of diabetes,effect,287
731,The evidence did show a reduction in the incid...,received intense lifestyle interventions or me...,cause,287
732,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...,cause,288


In [ ]:
#four-shot prompting F1-score correct (final)

In [ ]:
import pandas as pd

In [ ]:
report = pd.read_excel('/content/GPT4_gold_tokens.xlsx',sheet_name = 'Sheet2')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(report['gold'],report['preds']))

              precision    recall  f1-score   support

           A       0.50      0.69      0.58      2126
           C       0.24      0.83      0.37      1080
          CO       0.46      0.06      0.11      1584
           E       0.78      0.36      0.49      1805
           O       0.49      0.29      0.37      2499
           S       0.00      0.00      0.00        75

    accuracy                           0.42      9169
   macro avg       0.41      0.37      0.32      9169
weighted avg       0.51      0.42      0.39      9169



In [ ]:
import pandas as pd

In [ ]:
data_4shot = pd.read_excel('/content/Merged_GPT-4results.xlsx')

In [ ]:
data_4shot

,Unnamed: 0,variables_g,role_g,variables_p,role_p,text_g,text_p
0,0,Gestational diabetes,cause,Gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
1,4,Gestational diabetes,cause,Gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
2,9,Pregnant persons with gestational diabetes,cause,Pregnant persons with gestational diabetes,cause,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
3,14,increased risk for maternal and fetal complica...,effect,increased risk for maternal and fetal complica...,effect,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
4,19,increased risk of several long-term health out...,effect,increased risk of several long-term health out...,effect,ImportanceGestational diabetes is diabetes th...,Importance Gestational diabetes is diabetes th...
...,...,...,...,...,...,...,...
580,1012,risk factor,signal,"diabetes, like all chronic conditions, exists ...",cause,"While AAFP acknowledges that diabetes , like a...","While AAFP acknowledges that diabetes, like al..."
581,1015,received intense lifestyle interventions or me...,cause,reduction in the incidence of diabetes in indi...,effect,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...
582,1017,reduction in the incidence of diabetes,effect,reduction in the incidence of diabetes in indi...,effect,The evidence did show a reduction in the incid...,The evidence did show a reduction in the incid...
583,1018,Lifestyle interventions to prevent progression...,cause,Lifestyle interventions to prevent progression...,cause,Lifestyle interventions to prevent progression...,Lifestyle interventions to prevent progression...


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_true=data_4shot['role_g'], y_pred=data_4shot['role_p']))

              precision    recall  f1-score   support

      action       0.49      0.76      0.59       116
       cause       0.42      0.87      0.56       156
   condition       0.57      0.06      0.11       134
      effect       0.79      0.34      0.47       145
      signal       0.00      0.00      0.00        34

    accuracy                           0.48       585
   macro avg       0.45      0.41      0.35       585
weighted avg       0.53      0.48      0.41       585

